__[Open and try this file online (Colab, Pro / Extended RAM needed)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x34_d01_tag_und_member.ipynb)__

# tag und member
Verarbeitungsschritt Nr.3. Setzt das Verzeichnis data_00 voraus, in dem die vorherige Verarbeitungsstufe (d00_mwparserfromhell) die bisherigen Ergebnisse gespeichert hat. 
* speichert in data_01
* erstellt die Felder tag und member 
* integriert lexeme_manuell, named_entities und Zahlwörter
* Löschen laut löschliste
* behandelt Partizipien
* formt die grundlegende Datenstruktur (s.u.)
* testet Lexeme gemäß lex_test level 1
* Datei wiktionary ist danach überflüssig
* Dauer: Colab 37 min, Altrechner 26 min


### Doku
* __wiktionary_lemma__ enthält alle Grundformen. Ihr Identifikator ist die lemma_id.
* __wiktionary_merkmal__ enthält Merkmale, die einer Grundform zugeordnet werden. Manche Merkmale sind Lexeme, andere nicht. Merkmale eines Adjektivs sind z.B. seine Steigerungen (das sind Lexeme der Grundform) oder ein Gegenteilswort zur Grundform (das sind keine Lexeme der Grundform).
* __wiktionary_merkmal_text__ enthält ebenfalls Merkmale zu einer lemma_id. Allerdings keine einzelnen Wörter, sondern Textabsätze. Z..B. Definitionen oder Beispielsätze.

* __Abkürzungen__ stehen immer in wiktionary_merkmal (merkmal abk) und manchmal auch als eigenständiges Lemma in wiktionary_lemma.
  In wiktionary_lemma werden Abkürzungen durch das member 'Abkürzung' gekennzeichnet. 
  Das gibt die Information, dass es sich lohnt, in wiktionary_merkmal nachzuschlagen, um die Bedeutung einer Abkürzung zu erfahren.

* __Partizipien__ stehen in wiktionary_merkmal. Ihr lemma ist das Verb im Infinitiv (das ist oft falsch eingetragen). Ihr tag lautet 'VVPP ADJA' für Ptz2 bzw. 'VVFIN ADJA' für Ptz1, ihr member ist Ptz1 bzw. Ptz2.
  Zusätzlich sind alle Partizipien aber auch als Adjektiv als eigenständiges lemma registriert. Dort weist data_id auf das Verb zurück.

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 20:03:37


In [2]:
# pandas
if environment['in_colab']:
    !pip install --upgrade pandas
import pandas as pd

In [3]:
# alle anderen
try:
    import pandasklar as pak, bpyth as bpy, bj_nlp

except ImportError as e:
    !pip install pandasklar bj_nlp
    import pandasklar as pak, bpyth as bpy, bj_nlp
    
grid       = pak.grid                     # dTale-Nutzung gemäß Config
from functools import partial
gridt = partial(grid, backend='dtale')    # dTale-Nutzung erzwungen

check_mask = pak.check_mask      

## Einstellungen

In [4]:
# Einstellungen

# Was debuggen?
suche_debug = ['Euro','€','tugendhaftestes','zu viele','zuviele', 'zu viel']  

gründlich          = True    # einige Arbeitsschritte können weggelassen werden, wenn gründlich = False
speicher_freigeben = False    # sollen alte Ergebnisse gelöscht werden
version            = '_all'
#version           = '_100K'


# verbose
pak.Config.set('VERBOSE', True)

# Darstellung der Dataframes
pak.Config.set('GRID_BACKEND', '')   # standardmäßig kein dTale
pd.set_option('max_colwidth', 2000)  # aber dafür breiter

# Load
verzeichnis_load                      = 'data_00'
wiktionary_filename                   = verzeichnis_load + '/wiktionary'              + version + '.pickle'  
wiktionary_trash_filename             = verzeichnis_load + '/wiktionary_trash'        + version + '.pickle'
wiktionary_merkmal_filename           = verzeichnis_load + '/wiktionary_merkmal'      + version + '.pickle'  
wiktionary_merkmal_text_filename      = verzeichnis_load + '/wiktionary_merkmal_text' + version + '.pickle'  

# Steuertabellen, externe und manuelle Daten
löschliste_filename                   = 'Steuertabellen/löschliste.xlsx'     
lexeme_manuell_filename               = 'Steuertabellen/lexeme_manuell.xlsx'       # manuell eingepflegte Lexeme 
named_entities_filename               = 'Steuertabellen/named_entities_02.xlsx'    # Vornamen, Nachnamen, Orte
lex_test_filename                     = 'Steuertabellen/lex_test.xlsx'     

# Save
verzeichnis_save                      = 'data_01'
wiktionary_lemma_filename_save        = verzeichnis_save + '/wiktionary_lemma'        + version + '.pickle'  
wiktionary_nolemma_filename_save      = verzeichnis_save + '/wiktionary_nolemma'      + version + '.pickle'  
wiktionary_merkmal_filename_save      = verzeichnis_save + '/wiktionary_merkmal'      + version + '.pickle'  
wiktionary_merkmal_text_filename_save = verzeichnis_save + '/wiktionary_merkmal_text' + version + '.pickle'  

#
# Einige hardcodierte Tags finden sich unter wiktionary Einzelkorrekturen
# !!!

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

GRID_BACKEND = 
--> setting backend= as default for all pandasklar functions



## Vorbereiten

In [5]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

In [6]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

In [7]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt 
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

In [8]:
# Weitere Imports
from x32_Funktionen                    import *
from Steuertabellen.s01_Steuertabellen import *
from collections                       import defaultdict

## Daten laden

In [9]:
# wiktionary laden     
wiktionary               = pak.load_pickle(wiktionary_filename)
wiktionary               = pak.drop_cols(wiktionary,['wikitext','section_id'])

wiktionary_trash         = pak.load_pickle(wiktionary_trash_filename)
wiktionary_trash         = pak.drop_cols(wiktionary_trash,['msg','section_id','wikitext'])
wiktionary_trash         = pak.rename_col(wiktionary_trash,'error','msg')

wiktionary_merkmal       = pak.load_pickle(wiktionary_merkmal_filename)
wiktionary_merkmal       = pak.drop_cols(wiktionary_merkmal,['section_id'])

wiktionary_merkmal_trash = wiktionary_merkmal.head(0)

wiktionary_merkmal_text  = pak.load_pickle(wiktionary_merkmal_text_filename)

837593 rows loaded
4152 rows loaded
5171417 rows loaded
654677 rows loaded


In [10]:
# laden

# pak.nnan(wiktionary_merkmal_text)

In [11]:
# Keine NaN
assert not pak.any_nan(wiktionary, without=['syllables','rhymes','flexion','ipa'])

In [12]:
# Keine Leerzeichen in der ID
mask = wiktionary.section_id2.str.contains(' ')
check_mask(wiktionary,mask,0)

check_mask:  0 rows


In [13]:
pak.memory_consumption(locals())

,name,rtype,size
0,wiktionary_merkmal,"(DataFrame, Series)",2.1 GB
1,wiktionary,"(DataFrame, Series)",769.9 MB
2,wiktionary_merkmal_text,"(DataFrame, Series)",326.5 MB
3,wiktionary_trash,"(DataFrame, Series)",4.0 MB
4,mask,"(Series,)",1.6 MB
5,translate_tag,"(DataFrame, Series)",37.3 KB
6,translate_tagZ,"(DataFrame, Series)",20.9 KB
7,_i4,"(str,)",6.1 KB
8,Stop,"(type,)",1.0 KB
9,Markdown,"(type,)",1.0 KB


### wiktionary anschauen

In [14]:
# suche_debug 
mask = wiktionary.title.isin(suche_debug)
wiktionary[mask].sort_values(['section_id2',])

,page_id,page_part,section_id2,title,lemma,inflected,pos,rhymes,section_len,flexion,ipa,syllables,lang,lang_code,error
6707,11127,0,Euro,Euro,Euro,False,{'Substantiv': []},[ɔɪ̯ʁo],4588,"{'Genus': 'm', 'Nominativ Singular': 'Euro', 'Nominativ Plural 1': 'Euros', 'Nominativ Plural 2': 'Euro', 'Genitiv Singular': 'Euro', 'Genitiv Singular*': 'Euros', 'Genitiv Plural 1': 'Euros', 'Genitiv Plural 2': 'Euro', 'Dativ Singular': 'Euro', 'Dativ Plural 1': 'Euros', 'Dativ Plural 2': 'Euro', 'Akkusativ Singular': 'Euro', 'Akkusativ Plural 1': 'Euros', 'Akkusativ Plural 2': 'Euro'}",[ˈɔɪ̯ʁo],"[Eu, ro]",Deutsch,de,
6708,11127,1,Euro_1,Euro,Euro,False,{'Substantiv': []},[ɔɪ̯ʁo],1502,"{'Genus': 'f', 'Nominativ Singular': 'Euro', 'Nominativ Plural': 'Euros', 'Genitiv Singular': 'Euro', 'Genitiv Plural': 'Euros', 'Dativ Singular': 'Euro', 'Dativ Plural': 'Euros', 'Akkusativ Singular': 'Euro', 'Akkusativ Plural': 'Euros'}",[ˈɔɪ̯ʁo],"[Eu, ro]",Deutsch,de,
129287,284496,0,tugendhaftestes,tugendhaftestes,tugendhaft,True,{'Deklinierte Form': []},[uːɡn̩thaftəstəs],720,NaN,[ˈtuːɡn̩thaftəstəs],"[tu, gend, haf, tes, tes]",Deutsch,de,
31948,89575,0,zu_viel,zu viel,zu viel,False,"{'Adverb': [], 'Wortverbindung': []}",[iːl],1394,NaN,[ˈt͡suː fiːl],"[zu, viel]",Deutsch,de,
558818,820148,0,zu_viele,zu viele,zu viel,True,{'Wortverbindung': []},[iːlə],914,NaN,[t͡su ˈfiːlə],"[zu, vie, le]",Deutsch,de,


In [15]:
#pak.sample(wiktionary)

In [16]:
# suche_debug 
mask = wiktionary_trash.title.isin(suche_debug)
grid(wiktionary_trash[mask].sort_values(['section_id2',]))

No rows


In [17]:
# wiktionary_trash
a = pak.analyse_freqs(wiktionary_trash, 'msg')
a

,msg,msg_count,msg_percent,graph
0,title zu viele Wörter,3910,94.2,###############################################
1,title zu kurz,206,5.0,##
2,lemma zu kurz,26,0.6,
3,lang_code ist nan,10,0.2,


### wiktionary_merkmal anschauen

In [18]:
# suche_debug 
mask = wiktionary_merkmal.data.isin(suche_debug)
a = wiktionary_merkmal[mask].sort_values(['section_id2','merkmal'])
#a

In [19]:
pak.sample(wiktionary_merkmal)

,section_id2,merkmal,node_debug,node_kontext,data,num,meta
0,Hallo,übersicht,Deutsch Substantiv Übersicht,Genus,n,NaN,<NA>
446,April_1,istName,<NA>,<NA>,exists,NaN,<NA>
1031,Liebe_1,übersicht,Deutsch Substantiv Übersicht,Nominativ Plural,—,NaN,<NA>
10413,Haus,übersicht,Deutsch Substantiv Übersicht,Genus,n,NaN,<NA>
18515,Rauch,nurWortart,Wortart fehlt,1,Substantiv,NaN,<NA>
81347,spulgen,übersicht,Deutsch Verb Übersicht,Präteritum_ich,,NaN,<NA>
175097,ein-,Präfix,<NA>,<NA>,exists,NaN,<NA>
230371,Joch,herkunft,,Ü,𒄿𒌑𒃷,[],
1308756,μm,up,,,Einheit,[1],
1314648,&c.,def,,,et cetera,[1],


In [20]:
pak.analyse_cols(wiktionary_merkmal)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,int64,np.int64,int64,np.int32,True,False,True,False,80.0 B,1,5171417,0,0,5171417,0.0,2585708.0,2585708.0,5171416.0,1.337177e+13
1,section_id2,str,pd.string,string,,False,True,True,True,782.0 B,1,836708,0,4334709,5171417,&c.,NaN,NaN,μm,NaN
2,merkmal,str,pd.string,string,,False,True,True,True,684.0 B,1,36,0,5171381,5171417,Präfix,NaN,NaN,übersicht,NaN
3,node_debug,str,pd.string,string,,False,True,True,True,676.0 B,1,31,11465,5159921,5171417,,NaN,NaN,Wortart fehlt,NaN
4,node_kontext,str,pd.string,string,,False,True,True,True,593.0 B,1,235,11465,5159717,5171417,,NaN,NaN,Ü,NaN
5,data,str,pd.string,string,,False,True,True,True,783.0 B,1,725566,0,4445851,5171417,,NaN,NaN,𒄿𒌑𒃷,NaN
6,num,list,object,object,,False,True,False,True,544.0 B,1,224,1133103,4038090,5171417,[],NaN,NaN,[1225],NaN
7,meta,str,pd.string,string,,False,True,True,True,547.0 B,1,18571,1133103,4019743,5171417,,NaN,NaN,παθοs,NaN


In [21]:
# Häufigkeit der Merkmale
a = pak.analyse_freqs(wiktionary_merkmal,['merkmal','data'])
gridt(a)

36 rows


In [22]:
# ursprüngliche_merkmale merken
ursprüngliche_merkmale = wiktionary_merkmal.merkmal.unique()

## Vorverarbeitung
* Feld num in String wandeln, um dups entfernen zu können
* Dups entfernen
* wiktionary: flexion in defaultdict wandeln

In [23]:
# num ist leere Liste 
mask = (wiktionary_merkmal.num.str.len() == 0)  
#wiktionary_merkmal[mask]
wiktionary_merkmal.loc[mask,'num'] = ''

In [24]:
# fillna
wiktionary_merkmal['num'] = wiktionary_merkmal.num.fillna('')

In [25]:
# num ist Liste mit einem Eintrag >> diesen verwenden
mask = (wiktionary_merkmal.num.str.len() == 1)
wiktionary_merkmal.loc[mask,'num'] = wiktionary_merkmal[mask].num.str[0]
#wiktionary_merkmal[mask]

In [26]:
?pak.list_to_string

Signature: pak.list_to_string(series, sep=',')
Docstring:
Converts a Series of lists of strings into a Series of strings.
* sep: The separator, default is ','
Example:
df = pak.people()
df['history2'] = pak.list_to_string(df.history)    
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/pandas.py
Type:      function


In [27]:
# Liste in String wandeln, damit wir Dups entfernen können
# wiktionary_merkmal: list_to_string
wiktionary_merkmal['num'] = pak.list_to_string(wiktionary_merkmal.num)

In [28]:
# Dups entfernen. 
# Die Duplikate entstehen z.B. dadurch, dass für manche Merkmale nur der Linktext verwendet wird. 
# Beispiel Wolf_1, Merkmal kombi: die Wolf _mündet_, in die Wolf _mündet_ führt zu einem Duplikat.
mask = wiktionary_merkmal.duplicated()
wiktionary_merkmal = pak.drop_rows(wiktionary_merkmal, mask)

Delete 1902922 rows from 5171417


In [29]:
# lemma_lower
wiktionary['lemma_lower'] = wiktionary.lemma.str.lower() 

In [30]:
# wiktionary: flexion in defaultdict wandeln
# 1 min
wiktionary = pak.cols_to_defaultdict( wiktionary, col_dict='flexion')

In [31]:
# change_datatype
wiktionary = pak.change_datatype(wiktionary, verbose=False)

## score

In [32]:
# Basis ist erstmal die Länge des Wiktionary-Eintrags
wiktionary['score_base'] = pak.scale( wiktionary.section_len, 'compare_median') + 0.001

In [33]:
# Lemmas sind besser
wiktionary['f0'] = 0.5
mask = (wiktionary.title == wiktionary.lemma)
wiktionary.loc[mask,'f0'] = 2.0

In [34]:
# Je mehr _ die ID enthält, desto schlechter
wiktionary['f1'] = 1 / (1 + wiktionary.section_id2.str.count('_'))  

In [35]:
# Je weiter unten auf der Seite, desto schlechter
wiktionary['f2'] = 1 / (1 + wiktionary.page_part) 

mask = wiktionary.page_part == 0
wiktionary.loc[mask,'f2'] += 1

In [36]:
# Merkmale zählen. Je mehr Merkmale, desto besser
df = pak.group_and_agg( wiktionary_merkmal,
                        col_origins = ['section_id2','data',],
                        col_funcs   = ['',           'size'],
                        col_names   = ['',           'm_count'] )
wiktionary = pak.update_col(wiktionary, df, on='section_id2',col='m_count')
wiktionary['f3'] = (wiktionary.m_count**0.5).fillna(0.1)

2431787 rows less, now 836708 rows
update_col: 836708 cells written into new column


In [37]:
# alles zusammenrechnen
wiktionary['score'] = wiktionary.score_base * wiktionary.f0 * wiktionary.f1 * wiktionary.f2 * wiktionary.f3

In [38]:
spalten = ['page_id','page_part','section_id2','title','lemma','section_len','score_base','f0','f1','f2','m_count','f3','score']
pak.sample(wiktionary[spalten])

,page_id,page_part,section_id2,title,lemma,section_len,score_base,f0,f1,f2,m_count,f3,score
0,0,0,Hallo,Hallo,Hallo,1935,2.752282,2.0,1.0,2.0,10.0,3.162278,34.81392
27,50,0,ich,ich,ich,86973,111.775359,2.0,1.0,2.0,29.0,5.385165,2407.714918
135,217,0,Haus,Haus,Haus,70143,90.198436,2.0,1.0,2.0,1126.0,33.555923,12106.767249
2394,3305,0,a._Rh.,a. Rh.,a. Rh.,339,0.52786,2.0,0.5,2.0,2.0,1.414214,1.493012
2643,3628,0,frz.,frz.,frz.,343,0.536124,2.0,1.0,2.0,1.0,1.000000,2.144496
13086,21980,1,North_Carolina_1,North Carolina,North Carolina,1055,1.624077,2.0,0.333333,0.5,9.0,3.000000,1.624077
17464,34601,0,Gotte,Gotte,Gott,309,0.465876,0.5,1.0,2.0,1.0,1.000000,0.465876
18971,47220,2,Riß_2,Riß,Riß,84,0.001,2.0,0.5,0.333333,NaN,0.100000,0.000033
83135,173070,0,Papas,Papas,Papas,1041,1.606128,2.0,1.0,2.0,16.0,4.000000,25.698051
95827,239700,0,μm,μm,μm,1007,1.562538,2.0,1.0,2.0,5.0,2.236068,13.975769


In [40]:
# Zwischenwerte löschen
wiktionary = pak.drop_cols(wiktionary, ['section_len','score_base','f0','f1','f2','m_count','f3'])

In [41]:
# Häufigkeiten score
pak.analyse_freqs( (wiktionary.score/50).round(0) * 50 ).head(10)

,item,item_count,item_percent,graph
0,0.0,727445,86.8,###########################################
1,50.0,88335,10.5,#####
2,100.0,12370,1.5,
3,150.0,3912,0.5,
4,200.0,1909,0.2,
5,250.0,1083,0.1,
6,300.0,660,0.1,
7,350.0,430,0.1,
8,400.0,283,0.0,
9,450.0,191,0.0,


In [42]:
# Einträge mit dem höchsten Score
a = wiktionary.sort_values('score',ascending=False).head(10)
spalten = ['section_id2','title','lemma','pos','flexion','score']
a[spalten]

,section_id2,title,lemma,pos,flexion,score
135,Haus,Haus,Haus,{'Substantiv': []},"{'Genus': ['n'], 'Nominativ Singular': ['Haus'], 'Nominativ Plural': ['Häuser'], 'Genitiv Singular': ['Hauses'], 'Genitiv Plural': ['Häuser'], 'Dativ Singular': ['Haus'], 'Dativ Singular*': ['Hause'], 'Dativ Plural': ['Häusern'], 'Akkusativ Singular': ['Haus'], 'Akkusativ Plural': ['Häuser']}",12106.767249
6315,schwimmen,schwimmen,schwimmen,{'Verb': []},"{'Präsens_ich': ['schwimme'], 'Präsens_du': ['schwimmst'], 'Präsens_er, sie, es': ['schwimmt'], 'Präteritum_ich': ['schwamm'], 'Präteritum_ich*': ['schwomm'], 'Partizip II': ['geschwommen'], 'Konjunktiv II_ich': ['schwämme'], 'Konjunktiv II_ich*': ['schwömme'], 'Imperativ Singular': ['schwimm'], 'Imperativ Plural': ['schwimmt'], 'Hilfsverb': ['sein'], 'Hilfsverb*': ['haben']}",8328.625148
12146,Schnecke,Schnecke,Schnecke,{'Substantiv': []},"{'Genus': ['f'], 'Nominativ Singular': ['Schnecke'], 'Nominativ Plural': ['Schnecken'], 'Genitiv Singular': ['Schnecke'], 'Genitiv Plural': ['Schnecken'], 'Dativ Singular': ['Schnecke'], 'Dativ Plural': ['Schnecken'], 'Akkusativ Singular': ['Schnecke'], 'Akkusativ Plural': ['Schnecken']}",7218.973809
862,Katze,Katze,Katze,{'Substantiv': []},"{'Genus': ['f'], 'Nominativ Singular': ['Katze'], 'Nominativ Plural': ['Katzen'], 'Genitiv Singular': ['Katze'], 'Genitiv Plural': ['Katzen'], 'Dativ Singular': ['Katze'], 'Dativ Plural': ['Katzen'], 'Akkusativ Singular': ['Katze'], 'Akkusativ Plural': ['Katzen']}",6405.19632
128,Wasser,Wasser,Wasser,{'Substantiv': []},"{'Genus': ['n'], 'Nominativ Singular': ['Wasser'], 'Nominativ Plural 1': ['Wasser'], 'Nominativ Plural 2': ['Wässer'], 'Genitiv Singular': ['Wassers'], 'Genitiv Plural 1': ['Wasser'], 'Genitiv Plural 2': ['Wässer'], 'Dativ Singular': ['Wasser'], 'Dativ Plural 1': ['Wassern'], 'Dativ Plural 2': ['Wässern'], 'Akkusativ Singular': ['Wasser'], 'Akkusativ Plural 1': ['Wasser'], 'Akkusativ Plural 2': ['Wässer']}",6229.017
2735,Papagei,Papagei,Papagei,{'Substantiv': []},"{'Genus': ['m'], 'Nominativ Singular': ['Papagei'], 'Nominativ Plural': ['Papageien'], 'Genitiv Singular': ['Papageien'], 'Genitiv Plural': ['Papageien'], 'Dativ Singular': ['Papageien'], 'Dativ Plural': ['Papageien'], 'Akkusativ Singular': ['Papageien'], 'Akkusativ Plural': ['Papageien']}",5875.737306
466,Krankenhaus,Krankenhaus,Krankenhaus,{'Substantiv': []},"{'Genus': ['n'], 'Nominativ Singular': ['Krankenhaus'], 'Nominativ Plural': ['Krankenhäuser'], 'Genitiv Singular': ['Krankenhauses'], 'Genitiv Plural': ['Krankenhäuser'], 'Dativ Singular': ['Krankenhaus'], 'Dativ Singular*': ['Krankenhause'], 'Dativ Plural': ['Krankenhäusern'], 'Akkusativ Singular': ['Krankenhaus'], 'Akkusativ Plural': ['Krankenhäuser']}",5780.693006
6139,Esel,Esel,Esel,{'Substantiv': []},"{'Genus': ['m'], 'Nominativ Singular': ['Esel'], 'Nominativ Plural': ['Esel'], 'Genitiv Singular': ['Esels'], 'Genitiv Plural': ['Esel'], 'Dativ Singular': ['Esel'], 'Dativ Plural': ['Eseln'], 'Akkusativ Singular': ['Esel'], 'Akkusativ Plural': ['Esel']}",4922.37385
24531,schräg,schräg,schräg,{'Adjektiv': []},"{'Positiv': ['schräg'], 'Komparativ': ['schräger'], 'Superlativ': ['schrägsten']}",4879.18033
7045,Band,Band,Band,{'Substantiv': []},"{'Genus': ['n'], 'Nominativ Singular': ['Band'], 'Nominativ Plural 1': ['Bänder'], 'Nominativ Plural 2': ['Bande'], 'Genitiv Singular': ['Bandes'], 'Genitiv Singular*': ['Bands'], 'Genitiv Plural 1': ['Bänder'], 'Genitiv Plural 2': ['Bande'], 'Dativ Singular': ['Band'], 'Dativ Singular*': ['Bande'], 'Dativ Plural 1': ['Bändern'], 'Dativ Plural 2': ['Banden'], 'Akkusativ Singular': ['Band'], 'Akkusativ Plural 1': ['Bänder'], 'Akkusativ Plural 2': ['Bande']}",4616.801168


In [43]:
#grid(wiktionary, pak.sample)

## Steuertabellen laden

In [44]:
blab.help(plan_merkmal_erstellen)

<class 'IPython.core.display.Markdown'>


<span style="font-size:larger;">plan_merkmal_erstellen():</span>

* definiert in Steuertabellen/s01_Steuertabellen.py
* `merkmal`: Name des Merkmals
* `template`: Name des Mediawiki-Templates, das diese Information enthält
* `collect`: Extractionsmethode
   * L extrahiert nur Links 
   * T extrahiert komplette Text-Absätze
   * Ü extrahiert Übersetzungen
   * P extrahiert Template-Parameter (Substring reicht)
   * E checkt nur auf Existenz
* `plan`:
   * 1: die Merkmale, die in wiktionary_merkmal gespeichert werden
   * 2: die Merkmale, die in wiktionary_merkmal_text gespeichert werden   
   * 3: Reihenfolge und Liste aller Merkmale aus flexion / übersicht <br>
        sowie die Merkmale aus named_entities und lexeme_manuell
* `sort`: Irgendwas zum Sortieren der Merkmale
* `is_lex`: Ist das Merkmal ein Lexem?    

In [45]:
# plan_merkmal Steuertabelle
try:
    del plan_merkmal
except:
    pass

from Steuertabellen.s01_Steuertabellen import *
plan_merkmal = plan_merkmal_erstellen()
#grid(plan_merkmal)
pak.sample(plan_merkmal)

,merkmal,template,collect,plan,sort,is_lex
0,noSteig,kSt.,E,1,,False
12,def,Bedeutungen,L,1,,False
17,klein,Verkleinerungsformen,L,1,,True
102,syllables,,,3,z1 syllables,False


In [46]:
# translate_tag: Wortarten >> tags

try:
    del translate_tag
except:
    pass

from Steuertabellen.s01_Steuertabellen import *
#grid(translate_tag)
pak.sample(translate_tag)

,quelle,ziel,notiz
0,Konjugierte_Form,LEX_V,Keine Grundform
76,Präfix,TRUNC,Wortartenlisten
100,LEX_V LEX_V,LEX_V,Vorrang Durchlauf2
176,ADJA ADJA,ADJA,Doppelung1
201,Vergleichspartikel,,löschmich


In [47]:
# translate_tagZ: tag >> tagZ und tagZZ

try:
    del translate_tagZ
except:
    pass

from Steuertabellen.s01_Steuertabellen import *
#grid(translate_tagZ)
pak.sample(translate_tagZ)

,tag,wortart_lang,tagZ,tagZZ,is_mutable,to_lemma
0,AFIX,Präfix,FIX,Z,False,W
19,VAFIN,Hilfsverb,VER_hilf,V,True,V
23,PPER,Personalpronomen,PRO,P,True,W
53,$(,Satzzeichen,SYM,Y,False,W
61,TODO,(todo!),X_todo,X,False,W


In [48]:
# lex_test, lemma_test
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=1)
#grid(lemma_test)
pak.sample(lemma_test)

180 rows loaded
Delete 84 rows from 180


,level,lemma,tag_soll,member_soll,check_tag
0,1,…,$(,manuell,
1,1,Ackermann,NE,Person_nachname,
4,1,angucken,VVFIN,manuell,
16,1,Bussi,NN,,
82,1,zurecht,PTKVZ,,


## Löschliste
* Excel-Datei aufbereiten
* wiktionary belöschen
* wiktionary_merkmal belöschen
* Falls löschliste und lexeme_manuell in Konflikt stehen, siegt lexeme_manuell. Es wird erst gelöscht und dann manuell ergänzt.

In [49]:
# Laden
löschliste = pak.load_excel( löschliste_filename )

607 rows loaded


In [50]:
# Format prüfen
löschliste['check'] = löschliste.isnull().sum(axis=1)
mask = löschliste.check != 3
error = check_mask(löschliste,mask,0, stop=False)
grid(löschliste,mask, color='green')

check_mask:  0 rows
No rows, mask filters them all away


In [51]:
löschliste = pak.drop_cols(löschliste,'check')
bpy.raise_if(error)

In [52]:
# Einzelne löschlisten

mask = löschliste.section_id2.notnull()
löschliste_section_id2 = list(löschliste[mask].drop_duplicates().section_id2)

mask = löschliste.lemma.notnull()
löschliste_lemma = list(löschliste[mask].drop_duplicates().lemma)

mask = löschliste.lex.notnull()
löschliste_lex = list(löschliste[mask].drop_duplicates().lex)
löschliste_lex += löschliste_lemma

mask = löschliste.data.notnull()
löschliste_data = list(löschliste[mask].drop_duplicates().data)

In [53]:
löschliste

,lex,lemma,section_id2,data
0,g'rad,<NA>,<NA>,<NA>
1,hapax legomenon,<NA>,<NA>,<NA>
2,Hapaxlegomenon,<NA>,<NA>,<NA>
3,Jacob Ruff,<NA>,<NA>,<NA>
4,SARS CoV 2 AMVV,<NA>,<NA>,<NA>
...,...,...,...,...
602,<NA>,<NA>,<NA>,endbetonte Form
603,<NA>,<NA>,<NA>,Laut Online-Duden kann es auch ein weiblicher (Berg-)Steiger sein.
604,<NA>,<NA>,<NA>,Plural der maskulinen Form
605,<NA>,<NA>,<NA>,homm


In [54]:
# löschliste_lemma nach löschliste_section_id2 übertragen
mask1 =  wiktionary.title.isin(löschliste_lemma)   &   (wiktionary.title == wiktionary.lemma)
mask2 = ~wiktionary.section_id2.isin(löschliste_section_id2)
mask = mask1  &  mask2
#wiktionary[mask]
löschliste_section_id2 += list(wiktionary[mask].section_id2)

In [55]:
# wiktionary belöschen nach section_id2
mask = wiktionary.section_id2.isin(löschliste_section_id2)   
wiktionary, wiktionary_trash = pak.move_rows( df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg='löschliste_section_id2' )       # Löschen

Move 504 rows from 837593


In [56]:
# wiktionary belöschen nach lex
mask = wiktionary.title.isin(löschliste_lex)   
wiktionary, wiktionary_trash = pak.move_rows( df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg='löschliste_lex' )       # Löschen

Move 51 rows from 837089


In [57]:
# wiktionary_merkmal belöschen nach section_id2
mask = wiktionary_merkmal.section_id2.isin(löschliste_section_id2) 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='löschliste_section_id2')

mask = wiktionary_merkmal_trash.msg == 'löschliste_section_id2'
grid(wiktionary_merkmal_trash[mask],pak.sample)

Move 11008 rows from 3268495
13 rows out of 11008


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,msg
446,April_1,istName,<NA>,<NA>,exists,,<NA>,löschliste_section_id2
5236,Mutter_2,übersicht,Deutsch Substantiv Übersicht,Nominativ Plural,—,,<NA>,löschliste_section_id2
5962,Boot_1,herkunft,,Ü,boot,,,löschliste_section_id2
81347,spulgen,übersicht,Deutsch Verb Übersicht,Präteritum_ich,,,<NA>,löschliste_section_id2
344673,ABl.EG,def,,,Amtsblatt der Europäischen Gemeinschaft,1,,löschliste_section_id2
479758,daselbst,abk,,,das,1,,löschliste_section_id2
592682,festkehren,übersicht,Deutsch Verb Übersicht,Präsens_ich,festkehre,,<NA>,löschliste_section_id2
670864,γ-Aminobuttersäure,übersicht,Deutsch Substantiv Übersicht,Genus,f,,<NA>,löschliste_section_id2
737003,fettarme_Milch,übersicht,Deutsch adjektivisch Übersicht,Nominativ Plural stark,——,,<NA>,löschliste_section_id2
1233898,geheien,def,,,werfen,5,alemannisch,löschliste_section_id2


In [58]:
# wiktionary_merkmal belöschen nach lex
mask = wiktionary_merkmal.data.isin(löschliste_lex) 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='löschliste_lex')

mask = wiktionary_merkmal_trash.msg == 'löschliste_lex'
grid(wiktionary_merkmal_trash[mask], pak.sample)

Move 2882 rows from 3257487
13 rows out of 2882


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,msg
1950,sein,alt,,,Sebastian Brant,,,löschliste_lex
49302,Geld,herkunft,,Ü,gelt,,,löschliste_lex
50817,Kreuzer,abk,,,Xer,2,,löschliste_lex
177755,Morphem,def,,,grammatisches Morphem,2,der,löschliste_lex
311374,fladern_1,übersicht,Deutsch Verb Übersicht,Partizip II,gefladerd,,<NA>,löschliste_lex
534518,absolut,kombi,,,absolute Dichtung,9,,löschliste_lex
633262,Piperidinsäure,def,,,γ-Aminobuttersäure,1,Chemie,löschliste_lex
1216474,-loge,def,,,gebundenes Lexem,1,,löschliste_lex
1432136,β-Ketobuttersäuren,gm,,,β-Ketobuttersäure,,,löschliste_lex
2196656,rußen,alt_1,,,russen,,Österreich,löschliste_lex


In [59]:
# aufräumen
del löschliste_lemma
del löschliste

## tag und member
* Aus dem Feld pos entstehen die Felder tag (standardisiert) und member (vollständig)

In [60]:
# Kaputte Datenstruktur in str wandeln
wiktionary['tag'] = wiktionary.pos.apply( lambda x: str(x) )
wiktionary = pak.move_cols(wiktionary,'tag','pos')

# translate
translate_chars = str.maketrans("{}'[]:", "     ,")
wiktionary.tag = wiktionary.tag.str.translate(translate_chars)

# Liste ohne Dups draus machen
wiktionary.tag = wiktionary.tag.str.split(",")
wiktionary.tag = wiktionary.tag.apply( lambda x: list(set(x)) )

# Strings pflegen
wiktionary.tag = wiktionary.tag.apply( lambda x: [ e.strip().replace(' ','_')  for e in x ] )

# Leere Strings entfernen
wiktionary.tag = wiktionary.tag.apply( lambda x: [ e for e in x  if e != ''] )

In [61]:
#grid(wiktionary, pak.sample)

In [62]:
# nach Priorität sortieren (laut Steuertabellen)
wiktionary.loc[:,'tag'] = member_sortieren(wiktionary.tag) 

In [63]:
# In string wandeln
wiktionary.tag = wiktionary.tag.str.join(' ')

In [64]:
# translate_first

print(translate_first1)
print(translate_first2)

wiktionary['tag'] = pak.replace_str( wiktionary.tag, translate_first1).str.strip()
wiktionary['tag'] = pak.replace_str( wiktionary.tag, translate_first2).str.strip()

[('  ', ' '), ('Partizip II', 'Ptz2'), ('Partizip_II', 'Ptz2'), ('PartizipII', 'Ptz2'), ('Konjunktiv II', 'Konj2'), ('Konjunktiv_II', 'Konj2'), ('KonjunktivII', 'Konj2'), ('  ', ' ')]
[('  ', ' '), ('Partizip I', 'Ptz1'), ('Partizip_I', 'Ptz1'), ('PartizipI', 'Ptz1'), ('Konjunktiv I', 'Konj1'), ('Konjunktiv_I', 'Konj1'), ('KonjunktivI', 'Konj1'), ('  ', ' ')]


In [65]:
#mask = (wiktionary.tag.str.count(' ') + 1)  > 2   
#wiktionary[mask]

In [66]:
# Spalte member bewahrt die ursprüngliche Klassifizierung vollständig
wiktionary['member'] = wiktionary.tag.copy()
wiktionary = pak.move_cols(wiktionary,'member','tag')

In [67]:
# wiktionary anschauen
#mask = wiktionary.tag == 'ADJA ADV PRO'
mask = wiktionary.title.isin( suche_debug )
grid(wiktionary,mask)

5 rows out of 837038


,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,member,rhymes,flexion,ipa,syllables,lang,lang_code,error,lemma_lower,score
6707,11127,0,Euro,Euro,Euro,False,{'Substantiv': []},Substantiv,Substantiv,[ɔɪ̯ʁo],"{'Genus': ['m'], 'Nominativ Singular': ['Euro'], 'Nominativ Plural 1': ['Euros'], 'Nominativ Plural 2': ['Euro'], 'Genitiv Singular': ['Euro'], 'Genitiv Singular*': ['Euros'], 'Genitiv Plural 1': ['Euros'], 'Genitiv Plural 2': ['Euro'], 'Dativ Singular': ['Euro'], 'Dativ Plural 1': ['Euros'], 'Dativ Plural 2': ['Euro'], 'Akkusativ Singular': ['Euro'], 'Akkusativ Plural 1': ['Euros'], 'Akkusativ Plural 2': ['Euro']}",[ˈɔɪ̯ʁo],"[Eu, ro]",Deutsch,de,,euro,153.715982
6708,11127,1,Euro_1,Euro,Euro,False,{'Substantiv': []},Substantiv,Substantiv,[ɔɪ̯ʁo],"{'Genus': ['f'], 'Nominativ Singular': ['Euro'], 'Nominativ Plural': ['Euros'], 'Genitiv Singular': ['Euro'], 'Genitiv Plural': ['Euros'], 'Dativ Singular': ['Euro'], 'Dativ Plural': ['Euros'], 'Akkusativ Singular': ['Euro'], 'Akkusativ Plural': ['Euros']}",[ˈɔɪ̯ʁo],"[Eu, ro]",Deutsch,de,,euro,4.110498
31948,89575,0,zu_viel,zu viel,zu viel,False,"{'Adverb': [], 'Wortverbindung': []}",Adverb Wortverbindung,Adverb Wortverbindung,[iːl],{},[ˈt͡suː fiːl],"[zu, viel]",Deutsch,de,,zu viel,14.845441
129287,284496,0,tugendhaftestes,tugendhaftestes,tugendhaft,True,{'Deklinierte Form': []},Deklinierte_Form,Deklinierte_Form,[uːɡn̩thaftəstəs],{},[ˈtuːɡn̩thaftəstəs],"[tu, gend, haf, tes, tes]",Deutsch,de,,tugendhaft,1.19459
558818,820148,0,zu_viele,zu viele,zu viel,True,{'Wortverbindung': []},Wortverbindung,Wortverbindung,[iːlə],{},[t͡su ˈfiːlə],"[zu, vie, le]",Deutsch,de,,zu viel,1.020573


In [68]:
#pak.sample(wiktionary)

In [69]:
pak.sample(translate_tag)

,quelle,ziel,notiz
0,Konjugierte_Form,LEX_V,Keine Grundform
76,Präfix,TRUNC,Wortartenlisten
100,LEX_V LEX_V,LEX_V,Vorrang Durchlauf2
176,ADJA ADJA,ADJA,Doppelung1
201,Vergleichspartikel,,löschmich


In [70]:
# anwenden
wiktionary['tag'] = pak.replace_str( wiktionary.tag, translate_tag )
wiktionary['tag'] = wiktionary.tag.str.replace('  ',' ').str.strip()    

In [71]:
# Durchlauf2: translate_tag erneut anwenden 
mask = translate_tag.notiz.str.contains('Durchlauf2')
wiktionary['tag'] = pak.replace_str( wiktionary.tag, translate_tag[mask] )
wiktionary['tag'] = wiktionary.tag.str.replace('  ',' ').str.strip()    

In [72]:
# tag TODO für alles, was noch weiterverarbeitet werden muss
mask = (wiktionary.tag.str.len() == 0)
wiktionary.loc[mask,'tag'] = 'TODO'
#grid(wiktionary,mask)

In [73]:
# wiktionary anschauen
#mask = wiktionary.tag == 'ADJA ADV PRO'
mask = wiktionary.title.isin( suche_debug )
grid(wiktionary,mask, backend='')

5 rows out of 837038


,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,member,rhymes,flexion,ipa,syllables,lang,lang_code,error,lemma_lower,score
6707,11127,0,Euro,Euro,Euro,False,{'Substantiv': []},NN,Substantiv,[ɔɪ̯ʁo],"{'Genus': ['m'], 'Nominativ Singular': ['Euro'], 'Nominativ Plural 1': ['Euros'], 'Nominativ Plural 2': ['Euro'], 'Genitiv Singular': ['Euro'], 'Genitiv Singular*': ['Euros'], 'Genitiv Plural 1': ['Euros'], 'Genitiv Plural 2': ['Euro'], 'Dativ Singular': ['Euro'], 'Dativ Plural 1': ['Euros'], 'Dativ Plural 2': ['Euro'], 'Akkusativ Singular': ['Euro'], 'Akkusativ Plural 1': ['Euros'], 'Akkusativ Plural 2': ['Euro']}",[ˈɔɪ̯ʁo],"[Eu, ro]",Deutsch,de,,euro,153.715982
6708,11127,1,Euro_1,Euro,Euro,False,{'Substantiv': []},NN,Substantiv,[ɔɪ̯ʁo],"{'Genus': ['f'], 'Nominativ Singular': ['Euro'], 'Nominativ Plural': ['Euros'], 'Genitiv Singular': ['Euro'], 'Genitiv Plural': ['Euros'], 'Dativ Singular': ['Euro'], 'Dativ Plural': ['Euros'], 'Akkusativ Singular': ['Euro'], 'Akkusativ Plural': ['Euros']}",[ˈɔɪ̯ʁo],"[Eu, ro]",Deutsch,de,,euro,4.110498
31948,89575,0,zu_viel,zu viel,zu viel,False,"{'Adverb': [], 'Wortverbindung': []}",ADV,Adverb Wortverbindung,[iːl],{},[ˈt͡suː fiːl],"[zu, viel]",Deutsch,de,,zu viel,14.845441
129287,284496,0,tugendhaftestes,tugendhaftestes,tugendhaft,True,{'Deklinierte Form': []},LEX_A,Deklinierte_Form,[uːɡn̩thaftəstəs],{},[ˈtuːɡn̩thaftəstəs],"[tu, gend, haf, tes, tes]",Deutsch,de,,tugendhaft,1.19459
558818,820148,0,zu_viele,zu viele,zu viel,True,{'Wortverbindung': []},TODO,Wortverbindung,[iːlə],{},[t͡su ˈfiːlə],"[zu, vie, le]",Deutsch,de,,zu viel,1.020573


In [74]:
# Dekliniertes_Gerundivum: beizubehaltende
# VMaßeinheiten

## Bindestrich

### Gebundene Adjektiv-Lexeme

In [75]:
# Gebundene Adjektiv-Lexeme in wiktionary
suche = ['ober-','unter-','äußer-','inner-',]
mask = wiktionary.section_id2.isin(suche)

check_mask(wiktionary,mask,0,4)
wiktionary[mask]

check_mask:  4 rows


,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,member,rhymes,flexion,ipa,syllables,lang,lang_code,error,lemma_lower,score
25872,68346,0,unter-,unter-,unter-,False,{'Affix': ['Präfix']},TRUNC,Affix Präfix,NaN,{},[ʊntɐ],"[un, ter]",Deutsch,de,,unter-,34.020112
214016,380261,0,ober-,ober-,ober-,False,"{'Affix': ['Präfix', 'Präfixoid']}",TRUNC,Affix Präfixoid Präfix,NaN,{},[ˈoːbɐ],[ober],Deutsch,de,,ober-,55.630255
457256,681179,0,äußer-,äußer-,äußer-,False,{'Adjektiv': []},ADJA,Adjektiv,NaN,"{'Positiv': ['äußer(er)'], 'Superlativ': ['äußersten']}",[ˈɔɪ̯səʁɐ],NaN,Deutsch,de,,äußer-,101.480792
457269,681193,0,inner-,inner-,inner-,False,{'Adjektiv': []},ADJA,Adjektiv,NaN,"{'Positiv': ['inner(er)'], 'Superlativ': ['innersten']}",[ˈɪnɐ],NaN,Deutsch,de,,inner-,84.878232


In [76]:
wiktionary.loc[mask,'title']       = wiktionary[mask].title.str.replace('-','')
wiktionary.loc[mask,'lemma']       = wiktionary[mask].lemma.str.replace('-','')
wiktionary.loc[mask,'lemma_lower'] = wiktionary[mask].lemma_lower.str.replace('-','')
wiktionary.loc[mask,'member']      = 'Adjektiv'
wiktionary.loc[mask,'tag']         = 'ADJA'
wiktionary.loc[mask,'pos']         = None # {'Adjektiv': []}
wiktionary.loc[mask,'section_id2'] = wiktionary[mask].section_id2.str.replace('-','_glex')
wiktionary[mask]

,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,member,rhymes,flexion,ipa,syllables,lang,lang_code,error,lemma_lower,score
25872,68346,0,unter_glex,unter,unter,False,None,ADJA,Adjektiv,NaN,{},[ʊntɐ],"[un, ter]",Deutsch,de,,unter,34.020112
214016,380261,0,ober_glex,ober,ober,False,None,ADJA,Adjektiv,NaN,{},[ˈoːbɐ],[ober],Deutsch,de,,ober,55.630255
457256,681179,0,äußer_glex,äußer,äußer,False,None,ADJA,Adjektiv,NaN,"{'Positiv': ['äußer(er)'], 'Superlativ': ['äußersten']}",[ˈɔɪ̯səʁɐ],NaN,Deutsch,de,,äußer,101.480792
457269,681193,0,inner_glex,inner,inner,False,None,ADJA,Adjektiv,NaN,"{'Positiv': ['inner(er)'], 'Superlativ': ['innersten']}",[ˈɪnɐ],NaN,Deutsch,de,,inner,84.878232


In [77]:
#pak.analyse_datatypes(wiktionary)

In [78]:
# Gebundene Adjektiv-Lexeme in wiktionary_merkmal
suche = ['ober-','unter-','äußer-','inner-',]
mask = wiktionary_merkmal.section_id2.isin(suche)
check_mask(wiktionary_merkmal,mask,80)
wiktionary_merkmal.loc[mask,'section_id2'] = wiktionary_merkmal[mask].section_id2.str.replace('-','_glex')
#wiktionary_merkmal[mask]

check_mask:  80 rows


In [79]:
# Gebundene Adjektiv-Lexeme in wiktionary_merkmal_text
suche = ['ober-','unter-','äußer-','inner-',]
mask = wiktionary_merkmal_text.section_id2.isin(suche)
check_mask(wiktionary_merkmal_text,mask,46)
wiktionary_merkmal_text.loc[mask,'section_id2'] = wiktionary_merkmal_text[mask].section_id2.str.replace('-','_glex')
#wiktionary_merkmal_text[mask]

check_mask:  46 rows


In [80]:
# Gebundene Adjektiv-Lexeme in wiktionary
suche = ['ober-','unter-','äußer-','inner-',]
mask = wiktionary.lemma.isin(suche)
check_mask(wiktionary,mask,36)

wiktionary.loc[mask,'title']       = wiktionary[mask].title.str.replace('-','')
wiktionary.loc[mask,'lemma']       = wiktionary[mask].lemma.str.replace('-','')
wiktionary.loc[mask,'lemma_lower'] = wiktionary[mask].lemma_lower.str.replace('-','')
# wiktionary[mask]

check_mask:  36 rows


In [81]:
# Gebundene Adjektiv-Lexeme in wiktionary_merkmal
suche = ['ober-','unter-','äußer-','inner-',]
mask = wiktionary_merkmal.data.isin(suche)
check_mask(wiktionary_merkmal,mask,0,10)
wiktionary_merkmal.loc[mask,'data'] = wiktionary_merkmal[mask].data.str.replace('-','')
wiktionary_merkmal[mask]

check_mask:  1 rows


,section_id2,merkmal,node_debug,node_kontext,data,num,meta
652252,unter-_1,übersicht,Deutsch Adjektiv Übersicht,Positiv,unter,,<NA>


### Präfix

In [82]:
# AFIX 551
mask0 = ~wiktionary.lemma.str.startswith('-')   
mask1 =  wiktionary.lemma.str.endswith('-')  
mask2 = ~wiktionary.lemma.str[0].str.isdigit()
mask3 =  wiktionary.lemma.str.len() > 1
mask4 = ~wiktionary.tag.str.contains(' ')   
mask = mask0  &  mask1  &  mask2  &  mask3  &  mask4 
pak.check_mask(wiktionary, mask, 551)

wiktionary.loc[mask,'tag']    = 'AFIX ' + wiktionary[mask].tag
wiktionary.loc[mask,'member'] = pak.remove_str(wiktionary[mask].member, ['Affix'])
grid(wiktionary, mask)

check_mask:  551 rows
551 rows out of 837038


,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,member,rhymes,flexion,ipa,syllables,lang,lang_code,error,lemma_lower,score
161,248,0,auto-,auto-,auto-,False,{'Gebundenes Lexem': []},AFIX TRUNC,Gebundenes_Lexem,NaN,{},[aʊ̯toː],"[au, to]",Deutsch,de,,auto-,64.162652
269,388,0,ambi-,ambi-,ambi-,False,{'Gebundenes Lexem': []},AFIX TRUNC,Gebundenes_Lexem,NaN,{},[ambi],"[am, bi]",Deutsch,de,,ambi-,15.632934
2694,3700,0,stief-,stief-,stief-,False,{'Gebundenes Lexem': []},AFIX TRUNC,Gebundenes_Lexem,NaN,{},[ʃtiːf],[stief],Deutsch,de,,stief-,22.996615
4142,6082,0,plagio-,plagio-,plagio-,False,{'Gebundenes Lexem': []},AFIX TRUNC,Gebundenes_Lexem,NaN,{},[plaɡi̯o-],"[pla, gio]",Deutsch,de,,plagio-,30.408737
4144,6084,0,ortho-,ortho-,ortho-,False,{'Gebundenes Lexem': []},AFIX TRUNC,Gebundenes_Lexem,NaN,{},[ˈɔʁto],"[or, tho]",Deutsch,de,,ortho-,55.665153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806614,1078889,0,nord-,nord-,nord-,False,{'Gebundenes Lexem': []},AFIX TRUNC,Gebundenes_Lexem,NaN,{},[nɔʁt],[nord],Deutsch,de,,nord-,57.463897
808534,1080874,0,Nobel-,Nobel-,Nobel-,False,{'Affix': ['Präfixoid']},AFIX TRUNC,Präfixoid,NaN,{},[ˈnoːbl̩],"[No, bel]",Deutsch,de,,nobel-,22.77493
809124,1081469,0,Operetten-,Operetten-,Operetten-,False,{'Affix': ['Präfixoid']},AFIX TRUNC,Präfixoid,NaN,{},[opəˈʁɛtn̩],"[Ope, ret, ten]",Deutsch,de,,operetten-,16.970527
814132,1086984,0,divers-,divers-,divers-,False,{'Gebundenes Lexem': []},AFIX TRUNC,Gebundenes_Lexem,NaN,{},[diˈvɛʁs],"[di, vers]",Deutsch,de,,divers-,29.66041


In [83]:
# dar: Präfix ohne Bindestrich
mask1 = wiktionary.lemma == 'dar'
mask2 = wiktionary.member.str.contains('Präfix')
mask = mask1  &  mask2
pak.check_mask(wiktionary, mask, 1)

wiktionary.loc[mask,'tag']    = 'AFIX TRUNC'
wiktionary.loc[mask,'member'] = 'Präfix'
wiktionary[mask]

check_mask:  1 rows


,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,member,rhymes,flexion,ipa,syllables,lang,lang_code,error,lemma_lower,score
20014,50798,0,dar,dar,dar,False,{'Affix': ['Präfix']},AFIX TRUNC,Präfix,[aːɐ̯],{},[daːɐ̯],[dar],Deutsch,de,,dar,0.507067


In [84]:
# Kontrolle und Statistik AFIX
mask_AFIX = wiktionary.tag.str.contains('AFIX')
a = wiktionary[mask_AFIX].sort_values(['section_id2'])
mask = a.tag != 'AFIX TRUNC'
check_mask(a,0)

df = pak.analyse_freqs(a, ['member','lemma'])
df = pak.drop_cols(df,'lemma_count')
grid(df)

check_mask:  552 rows
3 rows


,member,member_count,member_percent,lemma
0,Gebundenes_Lexem,378,68.5,"[tele-, Allround-, Atto-, Breitband-, Dezi-, Exa-, Femto-, Giga-, Hekto-, Kilo-, Kurzzeit-, Langzeit-, Lieblings-, Möchtegern-, Nachkriegs-, Nano-, Nullachtfünfzehn-, Peta-, Piko-, Polit-]"
1,Präfix,144,26.1,"[a-, ab-, an-, über-, Haupt-, Ingen-, abs-, ac-, ad-, af-, ag-, ak-, al-, anti-, ap-, ar-, as-, at-, auf-, aus-]"
2,Präfixoid,30,5.4,"[Allerwelts-, Bilderbuch-, Duodez-, General-, Heiden-, Mammut-, Nobel-, Operetten-, Pilot-, Quadrat-, Schmalspur-, Vollblut-, Westentaschen-, arsch-, bomben-, bären-, erz-, granaten-, grund-, hunde-]"


### Suffix

In [85]:
# ZFIX 480
mask0 =  wiktionary.lemma.str.startswith('-')
mask1 = ~wiktionary.lemma.str.endswith('-')
mask2 = ~wiktionary.lemma.str[-1].str.isdigit()
mask3 =  wiktionary.lemma.str.len() > 1
mask4 = ~wiktionary.tag.str.contains(' ')   
mask = mask0  &  mask1  &  mask2  &  mask3  &  mask4 
pak.check_mask(wiktionary, mask, 480)

wiktionary.loc[mask, 'tag']   = 'ZFIX ' + wiktionary[mask].tag
wiktionary.loc[mask,'member'] = pak.remove_str(wiktionary[mask].member, ['Affix'])
grid(wiktionary, mask)

check_mask:  480 rows
480 rows out of 837038


,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,member,rhymes,flexion,ipa,syllables,lang,lang_code,error,lemma_lower,score
1483,2140,0,-ow,-ow,-ow,False,{'Affix': ['Suffix']},ZFIX TRUNC,Suffix,NaN,{},[oː],[ow],Deutsch,de,,-ow,41.706392
3151,4533,0,-in,-in,-in,False,"{'Substantiv': [], 'Affix': ['Suffix']}",ZFIX TRUNC,Substantiv Suffix,NaN,"{'Genus': ['f'], 'Nominativ Singular': ['-in'], 'Nominativ Plural': ['-innen'], 'Genitiv Singular': ['-in'], 'Genitiv Plural': ['-innen'], 'Dativ Singular': ['-in'], 'Dativ Plural': ['-innen'], 'Akkusativ Singular': ['-in'], 'Akkusativ Plural': ['-innen']}",[ɪn],[in],Deutsch,de,,-in,54.969846
3152,4533,1,-in_1,-in,-in,False,{'Affix': ['Suffix']},ZFIX TRUNC,Suffix,NaN,{},"[ˈiːn, in]",[in],Deutsch,de,,-in,2.844091
3153,4533,2,-in_2,-in,-in,False,{'Affix': ['Suffix']},ZFIX TRUNC,Suffix,NaN,{},[iːn],[in],Deutsch,de,,-in,2.398567
4875,7314,0,-zid,-zid,-zid,False,{'Gebundenes Lexem': []},ZFIX TRUNC,Gebundenes_Lexem,NaN,{},[-t͡siːt],[zid],Deutsch,de,,-zid,44.211765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818059,1092226,0,-hältig,-hältig,-hältig,False,{'Affix': ['Suffix']},ZFIX TRUNC,Suffix,NaN,{},"[ˌhɛltɪç, ˌhɛltɪk]","[häl, tig]",Deutsch,de,,-hältig,77.039135
818239,1092548,0,-halber,-halber,-halber,False,{'Affix': ['Suffix']},ZFIX TRUNC,Suffix,NaN,{},[ˈhalbɐ],"[hal, ber]",Deutsch,de,,-halber,6.837396
828638,1106151,0,-biont,-biont,-biont,False,{'Gebundenes Lexem': []},ZFIX TRUNC,Gebundenes_Lexem,NaN,{},[ˈbi̯ɔnt],"[bi, ont]",Deutsch,de,,-biont,16.294325
829642,1107769,0,-seits,-seits,-seits,False,{'Affix': ['Suffix']},ZFIX TRUNC,Suffix,NaN,{},[zaɪ̯ts],[seits],Deutsch,de,,-seits,29.78781


In [86]:
# Kontrolle und Statistik ZFIX
mask_ZFIX = wiktionary.tag.str.contains('ZFIX')
a = wiktionary[mask_ZFIX].sort_values(['section_id2'])
mask = a.tag != 'ZFIX TRUNC'
check_mask(a,0)

df = pak.analyse_freqs(a, ['member','lemma'])
df = pak.drop_cols(df,'lemma_count')
grid(df)

check_mask:  480 rows
15 rows


,member,member_count,member_percent,lemma
0,Suffix,137,28.5,"[-en, -in, -ab, -aceae, -ade, -affin, -age, -al, -ales, -an, -ar, -arisch, -arm, -at, -ation, -au, -bar, -basiert, -blütig, -brüstig]"
1,Gebundenes_Lexem,102,21.2,"[-gon, -and, -arch, -archie, -arium, -artig, -artige, -ase, -bedürftig, -biont, -bräu, -cele, -chinesisch, -chorie, -coel, -cöl, -derm, -dermie, -drom, -echt]"
2,Ortsnamengrundwort,94,19.6,"[-a, -ach, -au, -aue, -bach, -beck, -bek, -berg, -borg, -born, -broich, -brunn, -brunnen, -brück, -brücke, -brügge, -burg, -busch, -by, -dal]"
3,Deklinierte_Form,77,16.0,"[-abel, -ibel, -jährig, -sal, -ant, -ier, -ling, -tum, -er, -meter, -algie, -anz, -chen, -elchen, -enz, -erchen, -ette, -gate, -heit, -in]"
4,Substantiv Suffix,23,4.8,"[-ier, -ant, -anz, -chen, -elchen, -enz, -er, -erchen, -ette, -heit, -in, -ismus, -keit, -kunde, -le, -lein, -ling, -sal, -schaft, -stan]"
5,Substantiv Gebundenes_Lexem,13,2.7,"[-meter, -algie, -loge, -logie, -manie, -mantie, -phagie, -philie, -phobie, -schrecke, -skopie, -urie]"
6,Suffixoid,7,1.5,"[-fritze, -hai, -heini, -mäßig, -schwanger, -tiger, -trächtig]"
7,Superlativ Adjektiv,6,1.2,"[-abel, -ant, -ell, -fähig, -ibel, -lastig]"
8,Komparativ Adjektiv,6,1.2,"[-abel, -ant, -ell, -fähig, -ibel, -lastig]"
9,Verb Suffix,5,1.0,"[-el, -fizier, -ier, -ig, -zen]"


### Sonstige

In [87]:
# MFIX >> löschen
mask0 =  wiktionary.lemma.str.startswith('-')
mask1 =  wiktionary.lemma.str.endswith('-') 
mask2 =  wiktionary.lemma.str.len() > 2
mask3 = ~wiktionary.tag.str.contains(' ')   
mask = mask0  &  mask1  &  mask2  &  mask3
check_mask(wiktionary, mask, 4)

msg = 'MFIX'
wiktionary, wiktionary_trash = pak.move_rows(df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg=msg) 
grid(wiktionary_trash, msg)

check_mask:  4 rows
Move 4 rows from 837038
4 rows out of 4711


,page_id,page_part,section_id2,title,lemma,inflected,pos,rhymes,section_len,flexion,ipa,syllables,lang,lang_code,msg,error,lemma_lower,score,tag,member
44323,116824,0,-acid-,-acid-,-acid-,False,{'Gebundenes Lexem': []},NaN,<NA>,{},[at͡siːt],[acid],Deutsch,de,MFIX,,-acid-,12.438816,TRUNC,Gebundenes_Lexem
44332,116859,0,-azid-,-azid-,-azid-,False,{'Gebundenes Lexem': []},NaN,<NA>,{},[at͡siːt],[azid],Deutsch,de,MFIX,,-azid-,14.687436,TRUNC,Gebundenes_Lexem
369954,556685,0,-o-,-o-,-o-,False,{'Affix': ['Interfix']},NaN,<NA>,{},[o],[o],Deutsch,de,MFIX,,-o-,12.735281,TRUNC,Affix Interfix
369955,556686,0,-i-,-i-,-i-,False,{'Affix': ['Interfix']},NaN,<NA>,{},[iː],[i],Deutsch,de,MFIX,,-i-,12.140166,TRUNC,Affix Interfix


In [88]:
# FIX_ZU_KURZ >> löschen
mask1A = wiktionary.tag.str.contains('AFIX')
mask1B = wiktionary.tag.str.contains('ZFIX')
mask2  = wiktionary.lemma.str.len() <= 2
mask = (mask1A | mask1B)  &  mask2
check_mask(wiktionary, mask, 9)

msg = 'FIX_ZU_KURZ'
wiktionary, wiktionary_trash = pak.move_rows(df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg=msg) 
grid(wiktionary_trash, msg)

check_mask:  10 rows
Move 10 rows from 837034
10 rows out of 4721


,page_id,page_part,section_id2,title,lemma,inflected,pos,rhymes,section_len,flexion,ipa,syllables,lang,lang_code,msg,error,lemma_lower,score,tag,member
9582,16406,0,a-,a-,a-,False,{'Affix': ['Präfix']},NaN,<NA>,{},[a(ː)],[a],Deutsch,de,FIX_ZU_KURZ,,a-,37.716729,AFIX TRUNC,Präfix
9583,16406,1,a-_1,a-,a-,False,{'Affix': ['Präfix']},NaN,<NA>,{},NaN,NaN,Deutsch,de,FIX_ZU_KURZ,,a-,3.875997,AFIX TRUNC,Präfix
25834,68299,0,-a,-a,-a,False,{'Ortsnamengrundwort': []},NaN,<NA>,{},[-a],[a],Deutsch,de,FIX_ZU_KURZ,,-a,22.750476,ZFIX NE,Ortsnamengrundwort
27408,71752,0,e-,e-,e-,False,{'Affix': ['Präfix']},NaN,<NA>,{},[e],[e],Deutsch,de,FIX_ZU_KURZ,,e-,21.455757,AFIX TRUNC,Präfix
35053,98034,0,-s,-s,-s,False,{'Affix': ['Suffix']},NaN,<NA>,{},[s],[s],Deutsch,de,FIX_ZU_KURZ,,-s,9.618256,ZFIX TRUNC,Suffix
35054,98034,1,-s_1,-s,-s,False,{'Affix': ['Interfix']},NaN,<NA>,{},[s],[s],Deutsch,de,FIX_ZU_KURZ,,-s,1.315103,ZFIX TRUNC,Interfix
39772,108322,0,-n,-n,-n,False,{'Affix': []},NaN,<NA>,{},[n],[n],Deutsch,de,FIX_ZU_KURZ,,-n,17.671247,ZFIX TRUNC,
39780,108330,0,-e,-e,-e,False,{'Affix': []},NaN,<NA>,{},[ə],[e],Deutsch,de,FIX_ZU_KURZ,,-e,15.925795,ZFIX TRUNC,
39781,108330,1,-e_1,-e,-e,False,{'Pronomen': ['Personalpronomen']},NaN,<NA>,{},[ə],[e],Deutsch,de,FIX_ZU_KURZ,,-e,0.510115,ZFIX PPER,Pronomen Personalpronomen
734488,1003815,0,-i,-i,-i,False,{'Affix': ['Suffix']},NaN,<NA>,{},[i],[i],Deutsch,de,FIX_ZU_KURZ,,-i,43.584072,ZFIX TRUNC,Suffix


In [89]:
# Zirkumfix >> löschen
maskA = wiktionary.lemma.str.contains('-…-')
maskB = wiktionary.member.str.contains('Zirkumfix')
mask = maskA | maskB
check_mask(wiktionary, mask, 3)

msg = 'Zirkumfix'
wiktionary, wiktionary_trash = pak.move_rows(df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg=msg) 
grid(wiktionary_trash, msg)

check_mask:  3 rows
Move 3 rows from 837024
3 rows out of 4724


,page_id,page_part,section_id2,title,lemma,inflected,pos,rhymes,section_len,flexion,ipa,syllables,lang,lang_code,msg,error,lemma_lower,score,tag,member
788475,1059723,0,Ge-…-e,Ge-…-e,Ge-…-e,False,{'Affix': ['Zirkumfix']},NaN,<NA>,{},[ɡəˈə],NaN,Deutsch,de,Zirkumfix,,ge-…-e,19.605901,TRUNC,Affix Zirkumfix
828474,1105850,0,be-…-t,be-…-t,be-…-t,False,{'Affix': ['Zirkumfix']},NaN,<NA>,{},[bəˈt],NaN,Deutsch,de,Zirkumfix,,be-…-t,36.23397,TRUNC,Affix Zirkumfix
828507,1105899,0,zer-…-t,zer-…-t,zer-…-t,False,{'Affix': ['Zirkumfix']},NaN,<NA>,{},[t͡sɛɐ̯ˈt],NaN,Deutsch,de,Zirkumfix,,zer-…-t,11.93464,TRUNC,Affix Zirkumfix


In [90]:
# dar ist ein Präfix ohne Bindestrich
mask = wiktionary.member.str.contains('Affix')
check_mask(wiktionary, mask, 1)

wiktionary.loc[mask,'member'] = pak.remove_str(wiktionary[mask].member, ['Affix'])
grid(wiktionary, mask)

check_mask:  0 rows
No rows, mask filters them all away


In [91]:
# Kontrolle und Statistik TRUNC
mask = wiktionary.tag.str.contains('TRUNC')
a = pak.analyse_freqs(wiktionary[mask], ['tag','member','lemma'])
assert a.shape[0] == 2
grid(a)

2 rows


,tag,tag_count,tag_percent,member,member_count,lemma,lemma_count
0,AFIX TRUNC,549,65.4,"[Gebundenes_Lexem, Präfix, Präfixoid]","[378, 141, 30]","[ab-, an-, über-, tele-, top-, auto-, ambi-, stief-, plagio-, ortho-, ein-, anglo-, hemi-, un-, poly-, erz-, infra-, anti-, hypo-, Ingen-]","[2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,ZFIX TRUNC,290,34.6,"[Suffix, Gebundenes_Lexem, Substantiv Suffix, Substantiv Gebundenes_Lexem, Suffixoid, Verb Suffix, Adjektiv Suffix, , Adjektiv Gebundenes_Lexem]","[135, 102, 23, 13, 7, 5, 2, 2, 1]","[-in, -ier, -el, -ig, -meter, -en, -gon, -ow, -zid, -kunde, -fizier, -erl, -stat, -schrecke, -au, -trop, -eck, -egk, -legen, -mar]","[3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


## Einzelkorrekturen

In [92]:
# member Häufigkeitsanalyse
a = pak.analyse_freqs(wiktionary,['member','lemma'], splits=[' ',None])
a = pak.drop_cols(a,'lemma_count')
gridt(a)

66 rows


In [93]:
# Einzelkorrekturen (das sind section_id2)       
einzelkorrekturen = { 'viel'        : 'ADJA PIAT',  # war: ADJA ADV PIAT CARD
                      'zig'         : 'ADJA',
                      'bis'         : 'KOUS APPR',      
                      'derentwegen' : 'PDAT PRELAT',
                      'Rheinland'   : 'NE',          # war: NN       
}

In [94]:
# anschauen (alle title, nicht nur die section_id2)
mask = wiktionary.title.isin(einzelkorrekturen.keys())
df = wiktionary[mask].sort_values('section_id2')
spalten = ['section_id2','title','lemma','tag','member','score']
grid(df[spalten])

6 rows


,section_id2,title,lemma,tag,member,score
22599,Rheinland,Rheinland,Rheinland,NN,Substantiv,71.044581
45932,bis,bis,bis,ITJ KON APPR,Partikel Konjunktion Adposition Präposition,99.658977
45933,bis_1,bis,bis,KOUS,Konjunktion Subjunktion,0.929987
58741,derentwegen,derentwegen,derentwegen,ADV PDAT PRELAT,Adverb Pronomen Demonstrativpronomen Relativpronomen,51.809136
4128,viel,viel,viel,ADJA ADV PIAT CARD,Adjektiv Adverb Pronomen Indefinitpronomen Numerale,430.210959
29519,zig,zig,zig,NN ADJA,Substantiv Adjektiv Zahlklassifikator,36.099143


In [95]:
# ändern lt. section_id2
for i,t in einzelkorrekturen.items():
    mask = (wiktionary.section_id2 == i)
    wiktionary.loc[mask,'tag'] = t

In [96]:
# pflege_tag_felder (schreibe tagZZ usw.)
wiktionary = pflege_tag_felder(wiktionary, translate_tagZ)

In [97]:
# anschauen (alle title, nicht nur die section_id2)
mask = wiktionary.title.isin(einzelkorrekturen.keys())
df = wiktionary[mask].sort_values('section_id2')
spalten = ['section_id2','title','lemma','tag','tagZZ','member','score']
grid(df[spalten])

6 rows


,section_id2,title,lemma,tag,tagZZ,member,score
22523,Rheinland,Rheinland,Rheinland,NE,N,Substantiv,71.044581
45671,bis,bis,bis,KOUS APPR,X,Partikel Konjunktion Adposition Präposition,99.658977
45672,bis_1,bis,bis,KOUS,X,Konjunktion Subjunktion,0.929987
58467,derentwegen,derentwegen,derentwegen,PDAT PRELAT,P,Adverb Pronomen Demonstrativpronomen Relativpronomen,51.809136
4109,viel,viel,viel,ADJA PIAT,A,Adjektiv Adverb Pronomen Indefinitpronomen Numerale,430.210959
29324,zig,zig,zig,ADJA,A,Substantiv Adjektiv Zahlklassifikator,36.099143


## named_entities

In [98]:
# laden
named_entities = pak.load_excel(named_entities_filename, tabcol='quelle')

4544 rows loaded


In [99]:
# Überblick
a = pak.analyse_freqs(named_entities, cols=['quelle','member','lemma'], splits=[None,' '])
a = pak.drop_cols(a,'lemma_count')
grid(a)

3 rows


,quelle,quelle_count,quelle_percent,member,member_count,lemma
0,Vorname,3825,84.2,"[Vorname, Nachname]","[3825, 11]","[Hagen, Jordan, Menzel, Oswald, Reinhardt, Sander, Timm, Urban, Wichmann, Wieland, Wulf, Anna, Elisabeth, Johanna, Marie, Julia, Katharina, Christina, Eva, Charlotte]"
1,Toponym,389,8.6,"[Toponym, Stadt, Land]","[389, 293, 103]","[Berlin, Frankfurt, London, Paris, Tokio, Wien, München, New York, Amsterdam, Madrid, Zürich, Sydney, Hamburg, Toronto, Los Angeles, Stockholm, Chicago, Seoul, Brüssel, Peking]"
2,Nachname,330,7.3,Nachname,330,"[Ackermann, Ahrens, Albers, Altmann, Appel, Bachmann, Bader, Baier, Bartels, Barth, Barthel, Bartsch, Baumann, Baur, Bayer, Beck, Becker, Beckmann, Beer, Behrendt]"


In [100]:
# score anpassen
# es gilt nahezu: wiktionary.score.quantile(0.5)  == named_entities.score.quantile(0.5) 
named_entities.score *= wiktionary.score.quantile(0.5) / named_entities.score.quantile(0.5) 

In [101]:
# je mehr Funktionen, desto besser
named_entities.score *= (1 + named_entities.member.str.count(' '))  

In [102]:
pak.sample(named_entities)

,lemma,tag,member,score,Genus,sex_f,sex_m,Land,Stadt,Nom Sg,Gen Sg,quelle
0,Hagen,NE,Vorname Nachname,73.790413,m,0.0,1.0,<NA>,<NA>,Hagen,Hagens,Vorname
140,Doris,NE,Vorname,19.764399,f,1.0,0.0,<NA>,<NA>,Doris,<NA>,Vorname
1875,Aytac,NE,Vorname,0.809541,x,0.5,0.5,<NA>,<NA>,Aytac,Aytacs,Vorname
3824,Ünal,NE,Vorname,0.745434,m,0.0,1.0,<NA>,<NA>,Ünal,Ünals,Vorname
3830,Wien,NE,Toponym Stadt,10.590842,<NA>,0.0,0.0,Österreich,<NA>,Wien,Wiens,Toponym
3867,Aachen,NE,Toponym Stadt,2.869799,<NA>,0.0,0.0,Deutschland,<NA>,Aachen,Aachens,Toponym
3970,Monterrey,NE,Toponym Stadt,2.123776,<NA>,0.0,0.0,Mexiko,<NA>,Monterrey,Monterreys,Toponym
4059,Tirana,NE,Toponym Stadt,1.687473,<NA>,0.0,0.0,Albanien,<NA>,Tirana,Tiranas,Toponym
4111,Singapur,NE,Toponym Land Stadt,11.577466,<NA>,0.0,0.0,Singapur,Singapur,Singapur,Singapurs,Toponym
4118,Deutschland,NE,Toponym Land,149.086761,<NA>,0.0,0.0,<NA>,Berlin,Deutschland,Deutschlands,Toponym


In [103]:
# Felder ergänzen
named_entities['title']       = named_entities.lemma
named_entities['lemma_lower'] = named_entities.lemma.str.lower() 
named_entities = pak.move_cols( named_entities, 'lemma_lower','lemma')

In [104]:
# section_id2
named_entities['section_id2'] = named_entities.lemma + '_' + named_entities.quelle.str[0].str.upper().copy()
named_entities['section_id2'] = named_entities.section_id2.str.replace(' ','_').copy()
named_entities = pak.move_cols(named_entities, ['section_id2','title'])

In [105]:
# Bekannte section_id2 aus wiktionary übertragen. Aber nur wenn der tag wirklich gleich ist.
mask = (wiktionary.lemma == wiktionary.title)  # wichtig
named_entities = pak.update_col(named_entities, wiktionary[mask], on=['lemma_lower','tag'], col='section_id2',  col_score='score' )

update_col: col_score score applied, 2288 records less!
update_col: 1251 cells written into existing column


In [106]:
pak.sample(named_entities)

,section_id2,title,lemma,lemma_lower,tag,member,score,Genus,sex_f,sex_m,Land,Stadt,Nom Sg,Gen Sg,quelle
0,Hagen,Hagen,Hagen,hagen,NE,Vorname Nachname,73.790413,m,0.0,1.0,<NA>,<NA>,Hagen,Hagens,Vorname
140,Doris,Doris,Doris,doris,NE,Vorname,19.764399,f,1.0,0.0,<NA>,<NA>,Doris,<NA>,Vorname
1875,Aytac_V,Aytac,Aytac,aytac,NE,Vorname,0.809541,x,0.5,0.5,<NA>,<NA>,Aytac,Aytacs,Vorname
3824,Ünal_V,Ünal,Ünal,ünal,NE,Vorname,0.745434,m,0.0,1.0,<NA>,<NA>,Ünal,Ünals,Vorname
3830,Wien,Wien,Wien,wien,NE,Toponym Stadt,10.590842,<NA>,0.0,0.0,Österreich,<NA>,Wien,Wiens,Toponym
3867,Aachen,Aachen,Aachen,aachen,NE,Toponym Stadt,2.869799,<NA>,0.0,0.0,Deutschland,<NA>,Aachen,Aachens,Toponym
3970,Monterrey_T,Monterrey,Monterrey,monterrey,NE,Toponym Stadt,2.123776,<NA>,0.0,0.0,Mexiko,<NA>,Monterrey,Monterreys,Toponym
4059,Tirana,Tirana,Tirana,tirana,NE,Toponym Stadt,1.687473,<NA>,0.0,0.0,Albanien,<NA>,Tirana,Tiranas,Toponym
4111,Singapur,Singapur,Singapur,singapur,NE,Toponym Land Stadt,11.577466,<NA>,0.0,0.0,Singapur,Singapur,Singapur,Singapurs,Toponym
4118,Deutschland,Deutschland,Deutschland,deutschland,NE,Toponym Land,149.086761,<NA>,0.0,0.0,<NA>,Berlin,Deutschland,Deutschlands,Toponym


In [107]:
# drop_cols
named_entities = pak.drop_cols(named_entities,['sex_f', 'sex_m', 'Land', 'Stadt','quelle'])

In [108]:
grid(named_entities, pak.sample)

7 rows out of 4544


,section_id2,title,lemma,lemma_lower,tag,member,score,Genus,Nom Sg,Gen Sg
0,Hagen,Hagen,Hagen,hagen,NE,Vorname Nachname,73.790413,m,Hagen,Hagens
140,Doris,Doris,Doris,doris,NE,Vorname,19.764399,f,Doris,<NA>
1875,Aytac_V,Aytac,Aytac,aytac,NE,Vorname,0.809541,x,Aytac,Aytacs
3824,Ünal_V,Ünal,Ünal,ünal,NE,Vorname,0.745434,m,Ünal,Ünals
3867,Aachen,Aachen,Aachen,aachen,NE,Toponym Stadt,2.869799,<NA>,Aachen,Aachens
4118,Deutschland,Deutschland,Deutschland,deutschland,NE,Toponym Land,149.086761,<NA>,Deutschland,Deutschlands
4543,Ziegler,Ziegler,Ziegler,ziegler,NE,Nachname,36.895206,<NA>,Ziegler,Zieglers


In [109]:
# Dups?
mask = named_entities.section_id2.duplicated()
check_mask(named_entities,mask,0,0)

check_mask:  0 rows


In [110]:
blab.help(pak.cols_to_dict)

<class 'IPython.core.display.Markdown'>


<span style="font-size:larger;">cols_to_dict(df, col_dict='', cols_add=[], use_defaultdict=False, drop=True):</span>

Moves columns into a dict or defaultdict.
This is 
* col_dict:         name of the target column. Can be empty, but may already contain a dict or defaultdict. 
* cols_add:         Columns to be packed.
* use_defaultdict:  Should a defaultdict be used as data structure? Otherwise keys can only occur once.
* drop:             Should the packed columns be dropped (>> move) or not (>> copy)?

In [111]:
named_entities = pak.cols_to_dict(named_entities,col_dict='flexion',cols_add=['Genus','Nom Sg','Gen Sg',], use_defaultdict=True)
named_entities

,section_id2,title,lemma,lemma_lower,tag,member,score,flexion
0,Hagen,Hagen,Hagen,hagen,NE,Vorname Nachname,73.790413,"{'Genus': ['m'], 'Nom Sg': ['Hagen'], 'Gen Sg': ['Hagens']}"
1,Jordan,Jordan,Jordan,jordan,NE,Vorname Nachname,73.790413,"{'Genus': ['m'], 'Nom Sg': ['Jordan'], 'Gen Sg': ['Jordans']}"
2,Menzel_V,Menzel,Menzel,menzel,NE,Vorname Nachname,73.790413,"{'Genus': ['m'], 'Nom Sg': ['Menzel'], 'Gen Sg': ['Menzels']}"
3,Oswald_V,Oswald,Oswald,oswald,NE,Vorname Nachname,73.790413,"{'Genus': ['m'], 'Nom Sg': ['Oswald'], 'Gen Sg': ['Oswalds']}"
4,Reinhardt_V,Reinhardt,Reinhardt,reinhardt,NE,Vorname Nachname,73.790413,"{'Genus': ['m'], 'Nom Sg': ['Reinhardt'], 'Gen Sg': ['Reinhardts']}"
...,...,...,...,...,...,...,...,...
4539,Wolff,Wolff,Wolff,wolff,NE,Nachname,36.895206,"{'Nom Sg': ['Wolff'], 'Gen Sg': ['Wolffs']}"
4540,Wolter_N,Wolter,Wolter,wolter,NE,Nachname,36.895206,"{'Nom Sg': ['Wolter'], 'Gen Sg': ['Wolters']}"
4541,Zander_1,Zander,Zander,zander,NE,Nachname,36.895206,"{'Nom Sg': ['Zander'], 'Gen Sg': ['Zanders']}"
4542,Zeller_N,Zeller,Zeller,zeller,NE,Nachname,36.895206,"{'Nom Sg': ['Zeller'], 'Gen Sg': ['Zellers']}"


In [112]:
grid(named_entities, pak.sample)

5 rows out of 4544


,section_id2,title,lemma,lemma_lower,tag,member,score,flexion
0,Hagen,Hagen,Hagen,hagen,NE,Vorname Nachname,73.790413,"{'Genus': ['m'], 'Nom Sg': ['Hagen'], 'Gen Sg': ['Hagens']}"
1002,Abelina_V,Abelina,Abelina,abelina,NE,Vorname,0.745434,"{'Genus': ['f'], 'Nom Sg': ['Abelina'], 'Gen Sg': ['Abelinas']}"
3867,Aachen,Aachen,Aachen,aachen,NE,Toponym Stadt,2.869799,"{'Nom Sg': ['Aachen'], 'Gen Sg': ['Aachens']}"
4118,Deutschland,Deutschland,Deutschland,deutschland,NE,Toponym Land,149.086761,"{'Nom Sg': ['Deutschland'], 'Gen Sg': ['Deutschlands']}"
4543,Ziegler,Ziegler,Ziegler,ziegler,NE,Nachname,36.895206,"{'Nom Sg': ['Ziegler'], 'Gen Sg': ['Zieglers']}"


In [113]:
# anfügen
wiktionary = pak.add_rows(wiktionary, named_entities, assert_subset=True)

4544 rows added, now a total of 841565


In [114]:
# flexion type korrekt?
assert pak.ntypes(wiktionary.flexion) == 1

In [115]:
# wiktionary anschauen
mask = wiktionary.title.isin(suche_debug) 
r = wiktionary[mask].sort_values(['title','section_id2'])
grid(r)

5 rows


,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,tag_0,tag_1,...,member,rhymes,flexion,ipa,syllables,lang,lang_code,error,lemma_lower,score
6687,11127,0,Euro,Euro,Euro,False,{'Substantiv': []},NN,NN,Q34cMU8Jzm,...,Substantiv,[ɔɪ̯ʁo],"{'Genus': ['m'], 'Nominativ Singular': ['Euro'], 'Nominativ Plural 1': ['Euros'], 'Nominativ Plural 2': ['Euro'], 'Genitiv Singular': ['Euro'], 'Genitiv Singular*': ['Euros'], 'Genitiv Plural 1': ['Euros'], 'Genitiv Plural 2': ['Euro'], 'Dativ Singular': ['Euro'], 'Dativ Plural 1': ['Euros'], 'Dativ Plural 2': ['Euro'], 'Akkusativ Singular': ['Euro'], 'Akkusativ Plural 1': ['Euros'], 'Akkusativ Plural 2': ['Euro']}",[ˈɔɪ̯ʁo],"[Eu, ro]",Deutsch,de,,euro,153.715982
6688,11127,1,Euro_1,Euro,Euro,False,{'Substantiv': []},NN,NN,vhokubi30Y,...,Substantiv,[ɔɪ̯ʁo],"{'Genus': ['f'], 'Nominativ Singular': ['Euro'], 'Nominativ Plural': ['Euros'], 'Genitiv Singular': ['Euro'], 'Genitiv Plural': ['Euros'], 'Dativ Singular': ['Euro'], 'Dativ Plural': ['Euros'], 'Akkusativ Singular': ['Euro'], 'Akkusativ Plural': ['Euros']}",[ˈɔɪ̯ʁo],"[Eu, ro]",Deutsch,de,,euro,4.110498
128971,284496,0,tugendhaftestes,tugendhaftestes,tugendhaft,True,{'Deklinierte Form': []},LEX_A,LEX_A,Zx6kMHkYfH,...,Deklinierte_Form,[uːɡn̩thaftəstəs],{},[ˈtuːɡn̩thaftəstəs],"[tu, gend, haf, tes, tes]",Deutsch,de,,tugendhaft,1.19459
31740,89575,0,zu_viel,zu viel,zu viel,False,"{'Adverb': [], 'Wortverbindung': []}",ADV,ADV,wpQ70rl7by,...,Adverb Wortverbindung,[iːl],{},[ˈt͡suː fiːl],"[zu, viel]",Deutsch,de,,zu viel,14.845441
558334,820148,0,zu_viele,zu viele,zu viel,True,{'Wortverbindung': []},TODO,TODO,sWEsmc8dOR,...,Wortverbindung,[iːlə],{},[t͡su ˈfiːlə],"[zu, vie, le]",Deutsch,de,,zu viel,1.020573


## Zahlwörter
* werden anschließend lexeme_manuell zugeschlagen

In [116]:
# Wandelt eine Zahl in Worte um
def zahl_in_worten(n, s=True, z=False):
    
    w1 = "null ein zwei drei vier fünf sechs sieben acht neun zehn elf zwölf dreizehn vierzehn fünfzehn sechzehn siebzehn achtzehn neunzehn".split()
    w2 = "zwanzig dreißig vierzig fünfzig sechzig siebzig achtzig neunzig".split()    
    if n < 0: raise ValueError
    if n == 0 and z: return ""
    if n == 1 and s: return "eins"
    if n < 20: return w1[n]
    if n < 100:
        w = w2[(n - 20) // 10]
        if n % 10:
            w = w1[n % 10] + "und" + w
        return w
    if n < 1000:
        return w1[n // 100] + "hundert" + zahl_in_worten(n % 100, z=True)
    if n < 1000000:
        return zahl_in_worten(n // 1000, s=False) + "tausend" + zahl_in_worten(n % 1000, z=True)
    raise ValueError

In [117]:
print(zahl_in_worten(3107))

dreitausendeinhundertsieben


In [118]:
def zahlwörter_liste(range_): 
    return [ 
    {'lex' : [zahl_in_worten(z), str(z)], 
     'lemma' : zahl_in_worten(z),      
     'def'   : str(z),
    } for z in range_]

zahlwörter_liste(range(0,20,10))

[{'lex': ['null', '0'], 'lemma': 'null', 'def': '0'},
 {'lex': ['zehn', '10'], 'lemma': 'zehn', 'def': '10'}]

In [119]:
# 0..99 zahlwörter_a erzeugen
range_ = range(0,100)
zahlwörter_a = pd.DataFrame(zahlwörter_liste( range_ ))
zahlwörter_a['score'] = wiktionary.score.quantile(0.8)  +  1/(zahlwörter_a.index.astype(int)+1)**0.5*3+1
#zahlwörter_a
# wiktionary.score.quantile(0.7) / 

In [120]:
# Hunderter Zahlwörter erzeugen
range_ = range(100,1000,100)
zahlwörter_b = pd.DataFrame(zahlwörter_liste( range_ ))
zahlwörter_b['score'] = wiktionary.score.quantile(0.8)  +  1/(zahlwörter_b.index.astype(int)+1)**0.5*3+1
#zahlwörter_b

In [121]:
# Tausender Zahlwörter erzeugen
range_ = range(1000,99001,1000)
zahlwörter_c = pd.DataFrame(zahlwörter_liste( range_ ))
zahlwörter_c['score'] = wiktionary.score.quantile(0.6)  +  1/(zahlwörter_c.index.astype(int)+1)**0.5*3+1
#zahlwörter_c

In [122]:
# Zehntausender Zahlwörter erzeugen
range_ = range(10000,99001,10000)
zahlwörter_d = pd.DataFrame(zahlwörter_liste( range_ ))
zahlwörter_d['score'] = wiktionary.score.quantile(0.5)  +  1/(zahlwörter_d.index.astype(int)+1)**0.5*3+1
#zahlwörter_d

In [123]:
# Hunderttausender Zahlwörter erzeugen
range_ = range(100000,990001,100000)
zahlwörter_e = pd.DataFrame(zahlwörter_liste( range_ ))
zahlwörter_e['score'] = wiktionary.score.quantile(0.7)  +  1/(zahlwörter_e.index.astype(int)+1)**0.5*3+1
#zahlwörter_e

In [124]:
# concat
zahlwörter_all = pd.concat([  zahlwörter_a, zahlwörter_b, zahlwörter_c, zahlwörter_d, zahlwörter_e,  ])

In [125]:
# explode lex
zahlwörter_all = zahlwörter_all.explode('lex')

In [126]:
# Sonstige Spalten
zahlwörter_all['tag']         = 'CARD'
zahlwörter_all['tag_lex']     = 'lex' # ja
zahlwörter_all['genus']       = 'f'
zahlwörter_all['member']      = 'Zahlwort'
zahlwörter_all['tab']         = 'Zahlwort' # Wird später wieder gelöscht, dient der Kompatibilität zu lexeme_manuell

In [127]:
# Substantive ergänzen
mask = zahlwörter_all.lex == zahlwörter_all.lemma
df = zahlwörter_all[mask].copy()
df['lex']   = df.lex.str.capitalize()
df['lemma'] = df.lex.str.capitalize()
df['tag']   = 'NN'
zahlwörter_all = pak.add_rows(zahlwörter_all, df)
#df

226 rows added, now a total of 678


In [128]:
#zahlwörter_all['score'] = wiktionary.score.quantile(0.5)

In [129]:
# zahlwörter_all anschauen
pak.sample(zahlwörter_all)

,lex,lemma,def,score,tag,tag_lex,genus,member,tab
0,null,null,0,6.239462,CARD,lex,f,Zahlwort,Zahlwort
16,acht,acht,8,4.239462,CARD,lex,f,Zahlwort,Zahlwort
237,10000,zehntausend,10000,3.154812,CARD,lex,f,Zahlwort,Zahlwort
376,achtzigtausend,achtzigtausend,80000,2.541538,CARD,lex,f,Zahlwort,Zahlwort
414,neunundneunzigtausend,neunundneunzigtausend,99000,2.507640,CARD,lex,f,Zahlwort,Zahlwort
677,Neunhunderttausend,Neunhunderttausend,900000,3.705723,NN,lex,f,Zahlwort,Zahlwort


## lexeme_manuell

In [130]:
# laden
lexeme_manuell = pak.load_excel( lexeme_manuell_filename, tabcol='tab' )
lexeme_manuell = pak.move_cols(  lexeme_manuell, 'tag_lex')

1794 rows loaded


In [131]:
#pak.sample(lexeme_manuell)

In [132]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

,tag_lex,lex,lemma,tag,member,genus,tab
561,ADV,zu viele,zu viel,ADV,<NA>,<NA>,Diverses
564,ADV,zuviele,zu viel,ADV,<NA>,<NA>,Diverses
1608,<NA>,€,Euro,NN,Einheit,<NA>,Abkuerzungen
1648,<NA>,EUR,Euro,NN,Einheit,<NA>,Abkuerzungen


In [133]:
# Partizipien falsch definiert?
mask1  = lexeme_manuell.tag_lex.str.startswith('Ptz')
mask2A = lexeme_manuell.tab != 'Partizipien'
mask2B = lexeme_manuell.tag != 'VVFIN'
mask = mask1  &  (mask2A | mask2B)
check_mask(lexeme_manuell,mask,0)
#lexeme_manuell[mask]

check_mask:  0 rows


In [134]:
# Abkürzungen falsch definiert?
# Das Schlüsselwort 'Abkürzung' darf nirgends in lexeme_manuell vorkommen.
mask1A = lexeme_manuell.member.str.contains('Abkürzung')
mask1B = lexeme_manuell.member.str.contains('Abkürzung')
mask = (mask1A | mask1B) 
check_mask(lexeme_manuell,mask,0)
#lexeme_manuell[mask]

check_mask:  0 rows


In [135]:
# score
lexeme_manuell['score'] = wiktionary.score.quantile(0.8)

In [136]:
#pak.sample(zahlwörter_all)

In [137]:
s = zahlwörter_all.columns
t = lexeme_manuell.columns
bpy.minivenn(s,t, format='print2')

left_only:  {'def'}
right_only: {}



In [138]:
# assert subset
assert set(zahlwörter_all.columns) - set(['def']) <= set(lexeme_manuell.columns)

In [139]:
# zahlwörter_all anfügen
lexeme_manuell = pak.add_rows(lexeme_manuell, zahlwörter_all)

678 rows added, now a total of 2472


In [140]:
# tab analysieren
pak.analyse_freqs( lexeme_manuell, cols='tab')

,tab,tab_count,tab_percent,graph
0,esszett,729,29.5,##############
1,Zahlwort,678,27.4,#############
2,Partizipien,413,16.7,########
3,Diverses,308,12.5,######
4,Abkuerzungen,156,6.3,###
5,FM,102,4.1,##
6,Symbole,48,1.9,
7,Partikel,36,1.5,
8,Namen,2,0.1,


In [141]:
# NaN

# def
lexeme_manuell['def'] = lexeme_manuell['def'].fillna('')

# member
mask = lexeme_manuell.member.isnull()   |   lexeme_manuell.member.isin(['nan'])
lexeme_manuell.loc[mask,'member'] = ''
lexeme_manuell.member += ' manuell'

# lemma und lex
mask = lexeme_manuell.lemma.isnull()   |   lexeme_manuell.lemma.isin(['nan',''])
lexeme_manuell.loc[mask,'lemma'] = lexeme_manuell[mask].lex
lexeme_manuell.loc[mask,'member'] += ' ergänzt'

mask = lexeme_manuell.lex.isnull()   |   lexeme_manuell.lex.isin(['nan',''])
lexeme_manuell.loc[mask,'lex'] = lexeme_manuell[mask].lemma
lexeme_manuell.loc[mask,'member'] += ' ergänzt'

# genus
mask = lexeme_manuell.genus.isnull()   |   lexeme_manuell.genus.isin(['nan'])
lexeme_manuell.loc[mask,'genus'] = ''

# tag_lex
mask = lexeme_manuell.tag_lex.isnull()   |   lexeme_manuell.tag_lex.isin(['nan'])
lexeme_manuell.loc[mask,'tag_lex'] = ''

mask = (lexeme_manuell.lex != lexeme_manuell.lemma)   &   (lexeme_manuell.tag_lex == '')
lexeme_manuell.loc[mask,'tag_lex'] = 'lex'

mask = (lexeme_manuell.lex == lexeme_manuell.lemma)   &   (  (lexeme_manuell.tag_lex == 'lex')  | (lexeme_manuell.tag_lex == '')  )
lexeme_manuell.loc[mask,'tag_lex'] = 'lemma'

# tag
mask = lexeme_manuell.tag.isnull()   |   lexeme_manuell.tag.isin(['nan',''])
check_mask( lexeme_manuell, mask, 0, 0)

# Abschluss
lexeme_manuell.member = lexeme_manuell.member.str.strip()
lexeme_manuell = pak.rename_col(lexeme_manuell, 'genus','Genus')

#lexeme_manuell = pak.move_cols(lexeme_manuell, ['Genus','abk'], -1)
#lexeme_manuell['abk'] = ''

check_mask:  0 rows


In [142]:
# Dups löschen
mask = lexeme_manuell.duplicated()
lexeme_manuell, df = pak.move_rows(lexeme_manuell,mask)
grid(df)

Move 76 rows from 2472
76 rows


,tag_lex,lex,lemma,tag,member,Genus,tab,score,def,msg
897,lex,einflußreiche,einflussreich,ADJA,lexAlt manuell,,esszett,2.239462,,7
910,lex,gewiße,gewiss,ADJA,lexAlt manuell,,esszett,2.239462,,7
915,lex,gräßlicher,grässlich,ADJA,lexAlt manuell,,esszett,2.239462,,7
917,lex,gräßliches,grässlich,ADJA,lexAlt manuell,,esszett,2.239462,,7
937,lex,läßliche,lässlich,ADJA,lexAlt manuell,,esszett,2.239462,,7
...,...,...,...,...,...,...,...,...,...,...
1581,lex,wüßten,wissen,VVFIN,lexAlt manuell,,esszett,2.239462,,7
1584,lex,zerfrißt,zerfressen,VVFIN,lexAlt manuell,,esszett,2.239462,,7
1587,lex,zuläßt,zulassen,VVFIN,lexAlt manuell,,esszett,2.239462,,7
1590,lex,zusammenfaßt,zusammenfassen,VVFIN,lexAlt manuell,,esszett,2.239462,,7


In [143]:
pak.sample(lexeme_manuell)

,tag_lex,lex,lemma,tag,member,Genus,tab,score,def
0,Ptz2,abberufen,abberufen,VVFIN,manuell,,Partizipien,2.239462,
487,lex,Kids,Kid,NN,manuell,n,Diverses,2.239462,
502,lex,musses,müssen,VMFIN,manuell,,Diverses,2.239462,
561,ADV,zu viele,zu viel,ADV,manuell,,Diverses,2.239462,
721,lex,❀,_,$(,manuell,,Symbole,2.239462,
723,syn,[,(,$(,manuell,,Symbole,2.239462,
756,lemma,!,!,$.,manuell ergänzt,,Symbole,2.239462,
759,lemma,…,…,$(,manuell ergänzt,,Symbole,2.239462,
1022,lex,gewiß,gewiss,ADV,modal lexAlt manuell,,esszett,2.239462,
1611,lex,a.Chr.,vor Christus,NN,Einheit Wortverbindung manuell,,Abkuerzungen,2.239462,


In [144]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

,tag_lex,lex,lemma,tag,member,Genus,tab,score,def
561,ADV,zu viele,zu viel,ADV,manuell,,Diverses,2.239462,
564,ADV,zuviele,zu viel,ADV,manuell,,Diverses,2.239462,
1608,lex,€,Euro,NN,Einheit manuell,,Abkuerzungen,2.239462,
1648,lex,EUR,Euro,NN,Einheit manuell,,Abkuerzungen,2.239462,


In [145]:
# strip
for c in pak.col_names(lexeme_manuell, only='str'):
    lexeme_manuell[c] = lexeme_manuell[c].str.replace('  ',' ').str.strip()

In [146]:
# member analysieren
pak.analyse_freqs( lexeme_manuell, cols='member', splits=' ')

,member,member_count,member_percent,graph
0,manuell,2396,57.0,############################
1,Zahlwort,678,16.1,########
2,lexAlt,653,15.5,#######
3,ergänzt,345,8.2,####
4,Einheit,56,1.3,
5,PRE,42,1.0,
6,Wortverbindung,28,0.7,
7,Ort,2,0.0,
8,modal,1,0.0,
9,Land,1,0.0,


In [147]:
# Derzeit nicht nötig: Ersetzungen im member
r = {  'Partizip1'          :  'Ptz1',        
       'Partizip_1'         :  'Ptz1',       
       'Partizip2'          :  'Ptz2',        
       'Partizip_2'         :  'Ptz2',   
       'Abkürzungen'        :  'Abkürzung',      
       'Abkürzung Abkürzung':  'Abkürzung',        
    }
lexeme_manuell['member'] = pak.replace_str(lexeme_manuell.member, r)
#grid(lexeme_manuell)

In [148]:
# Partizipien: tag_lex setzen 
mask = lexeme_manuell.member.str.contains('Ptz1')
lexeme_manuell.loc[mask,'tag_lex'] = 'Ptz1'

mask = lexeme_manuell.member.str.contains('Ptz2')
lexeme_manuell.loc[mask,'tag_lex'] = 'Ptz2'

In [149]:
# These: Auch member muss gesetzt werden, weil Partizipien ja doppelt eingetragen werden
# NEIN: Das Partizip wandert ja nach flexion
#mask1 =  lexeme_manuell.tag_lex.str.startswith('Ptz')
#mask2 = ~lexeme_manuell.member.str.contains('Ptz1')
#mask3 = ~lexeme_manuell.member.str.contains('Ptz2')
#mask = mask1  &  mask2  &  mask3
#lexeme_manuell.loc[mask,'member'] = lexeme_manuell[mask].member + ' ' + lexeme_manuell[mask].tag_lex
#grid(lexeme_manuell[mask])

In [150]:
# Tab Esszett: tag_lex setzen
mask1 = lexeme_manuell.tab == 'esszett'
mask2A = lexeme_manuell.tag_lex == ''
mask2B = lexeme_manuell.tag_lex == 'lex'
mask = mask1  &  (mask2A | mask2B)
check_mask(lexeme_manuell,mask,653)
lexeme_manuell.loc[mask,'tag_lex'] = 'lexAlt'

check_mask:  653 rows


In [151]:
# Tab Esszett: member setzen
mask1 =  lexeme_manuell.tab == 'esszett'
mask2 = ~lexeme_manuell.member.str.contains('lexAlt')
mask = mask1  &  mask2
check_mask(lexeme_manuell,mask)
lexeme_manuell.loc[mask,'member'] += 'lexAlt'

check_mask:  0 rows


In [152]:
# Tab Abkuerzungen: tag_lex setzen
mask = (lexeme_manuell.tab == 'Abkuerzungen')
lexeme_manuell.loc[mask,'tag_lex'] = 'abk'

mask = lexeme_manuell.member.str.contains('Abkürzung')
lexeme_manuell.loc[mask,'tag_lex'] = 'abk'

In [153]:
# anschauen
pak.sample(lexeme_manuell)

,tag_lex,lex,lemma,tag,member,Genus,tab,score,def
0,Ptz2,abberufen,abberufen,VVFIN,manuell,,Partizipien,2.239462,
487,lex,Kids,Kid,NN,manuell,n,Diverses,2.239462,
502,lex,musses,müssen,VMFIN,manuell,,Diverses,2.239462,
561,ADV,zu viele,zu viel,ADV,manuell,,Diverses,2.239462,
721,lex,❀,_,$(,manuell,,Symbole,2.239462,
723,syn,[,(,$(,manuell,,Symbole,2.239462,
756,lemma,!,!,$.,manuell ergänzt,,Symbole,2.239462,
759,lemma,…,…,$(,manuell ergänzt,,Symbole,2.239462,
1022,lexAlt,gewiß,gewiss,ADV,modal lexAlt manuell,,esszett,2.239462,
1611,abk,a.Chr.,vor Christus,NN,Einheit Wortverbindung manuell,,Abkuerzungen,2.239462,


In [154]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

,tag_lex,lex,lemma,tag,member,Genus,tab,score,def
561,ADV,zu viele,zu viel,ADV,manuell,,Diverses,2.239462,
564,ADV,zuviele,zu viel,ADV,manuell,,Diverses,2.239462,
1608,abk,€,Euro,NN,Einheit manuell,,Abkuerzungen,2.239462,
1648,abk,EUR,Euro,NN,Einheit manuell,,Abkuerzungen,2.239462,


In [155]:
# tag_lex analysieren
r = pak.analyse_freqs(lexeme_manuell, ['tag_lex', 'lex', 'member'], splits=[None,None,' '])
grid(r)

10 rows


,tag_lex,tag_lex_count,tag_lex_percent,lex,lex_count,member,member_count
0,lemma,785,32.8,"[dies, grade, zehntausend, zwanzigtausend, dreißigtausend, vierzigtausend, fünfzigtausend, sechzigtausend, siebzigtausend, achtzigtausend, neunzigtausend, Zehntausend, Zwanzigtausend, Dreißigtausend, Vierzigtausend, Fünfzigtausend, Sechzigtausend, Siebzigtausend, Achtzigtausend, Neunzigtausend]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[manuell, Zahlwort, ergänzt, PRE, Einheit, Vorname, Toponym]","[785, 452, 329, 30, 11, 1, 1]"
1,lexAlt,653,27.3,"[muß, müßen, mußt, müßt, mußte, müßte, mußten, müßten, mußtest, müßtest, mußtet, müßtet, aufschlußreiche, aufschlußreichen, bewußt, bewußte, bewußtem, bewußten, bewußter, bewußtes]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[lexAlt, manuell, Ort, modal, Land]","[653, 653, 2, 1, 1]"
2,lex,372,15.5,"[10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, allem, allermeisten, allerwenigsten, allerwenigstens, als Erstes, als Letztes, beidem, bins, biste, brannt, brauchs]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[manuell, Zahlwort, PRE]","[372, 226, 6]"
3,Ptz1,253,10.6,"[abfahrend, abschreckend, abstoßend, abtörnend, abwertend, achtend, adstringierend, affigierend, agglutinierend, alarmierend, alleinstehend, alternierend, amtierend, andauernd, angrenzend, anhaltend, anliegend, anmaßend, anregend, anschließend]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",manuell,253
4,Ptz2,160,6.7,"[abberufen, abgepaßt, anbehalten, aneinandergepreßt, angepaßt, aufrechterhalten, auserlesen, autogestoppt, befahren, befallen, begraben, behauen, beladen, belassen, belaufen, bemessen, bepißt, beraten, berufen, beschrieen]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",manuell,160
5,abk,156,6.5,"[&, %, °, ° C, ° F, °C, °F, +, €, 5G, 9/11, a.Chr., Abb., Abs., Abschn., AI, Anm. d. Red., Anm.d.Red., Apr., Aufl.]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[manuell, Einheit, Wortverbindung, ergänzt, PRE]","[156, 45, 28, 16, 6]"
6,VVIZU,8,0.3,"[anzugucken, anzustaunen, aufzurütteln, einzureden, herauszulassen, hereinzulassen, herumzupimpeln, loszuwerden]","[1, 1, 1, 1, 1, 1, 1, 1]",manuell,8
7,syn,6,0.3,"[[, {, <, ], }, >]","[1, 1, 1, 1, 1, 1]",manuell,6
8,ADV,2,0.1,"[zu viele, zuviele]","[1, 1]",manuell,2
9,VVIMP,1,0.0,vergleiche,1,manuell,1


In [156]:
# Kopie erzeugen, verwenden wir später um alle Partizipien als Adjektive einpflegen zu können
lexeme_manuell_copy = lexeme_manuell.copy()

In [157]:
# Spalte flexion erstellen
lexeme_manuell['lex_bak'] = lexeme_manuell.lex.copy()
lexeme_manuell = pak.implode_to_defaultdict(lexeme_manuell, col_key='tag_lex', col_value='lex', col_result='flexion')

0 rows less, now 2396 rows


In [158]:
# suche_debug
#search_str(lexeme_manuell, suche_debug)

In [159]:
# lex == lemma: flexion soll leeres defaultdict sein
mask = (lexeme_manuell.lex_bak == lexeme_manuell.lemma) 
lexeme_manuell.loc[mask,'flexion'] = lexeme_manuell[mask].flexion.apply(lambda x: defaultdict(list))  # auf leeres defaultdict setzen
lexeme_manuell = pak.drop_cols(lexeme_manuell, 'lex_bak')
#grid(lexeme_manuell,~mask)

In [160]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

,lemma,tag,member,Genus,tab,score,def,flexion
235,Euro,NN,Einheit manuell,,Abkuerzungen,2.239462,,{'abk': ['EUR']}
236,Euro,NN,Einheit manuell,,Abkuerzungen,2.239462,,{'abk': ['€']}
2266,zu viel,ADV,manuell,,Diverses,2.239462,,{'ADV': ['zu viele']}
2267,zu viel,ADV,manuell,,Diverses,2.239462,,{'ADV': ['zuviele']}


In [161]:
# Weitere Spalten in flexion verpacken (mit cols_to_defaultdict
lexeme_manuell = pak.cols_to_defaultdict(  lexeme_manuell, col_dict='flexion', cols_add=['Genus','def']   )  

In [162]:
# Spalte tab brauchen wir nicht mehr >> löschen
lexeme_manuell = pak.drop_cols(lexeme_manuell, ['tab'])

In [163]:
# suche_debug
#search_str(lexeme_manuell, suche_debug)

In [164]:
# gruppieren
lexeme_manuell = pak.group_and_agg(lexeme_manuell,
                               ['lemma', 'tag',   'member',              'flexion',            'score', ],
                               ['group', 'group',  pak.agg_words_nodup,  pak.agg_defaultdicts, 'max',   ],
                               ['lemma', 'tag',   'member',              'flexion',            'score', ]
                              )

604 rows less, now 1792 rows


In [165]:
# Felder ergänzen
lexeme_manuell['title']       = lexeme_manuell.lemma
lexeme_manuell['lemma_lower'] = lexeme_manuell.lemma.str.lower() 
lexeme_manuell = pak.move_cols( lexeme_manuell, 'lemma_lower','lemma')

In [166]:
# tagZ und tagZZ ermitteln
lexeme_manuell = pflege_tag_felder(lexeme_manuell, translate_tagZ)

In [167]:
# anschauen
grid(lexeme_manuell, pak.sample_100)

100 rows out of 1792


,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,flexion,score,title
0,!,!,$.,$.,V8I9aAXDJJ,SYM,Y,manuell ergänzt,{},2.239462,!
12,24/7,24/7,ADJA,ADJA,PUu8jiZsQg,ADJ,A,manuell ergänzt,{},2.239462,24/7
15,4D,4d,ADJA,ADJA,WHCyWk7Ykw,ADJ,A,manuell ergänzt,{},2.239462,4D
28,Abschlussarbeit,abschlussarbeit,NN,NN,lYOzN47FZo,NOM,N,lexAlt manuell,"{'lexAlt': ['Abschlußarbeit', 'Abschlußarbeiten'], 'Genus': ['f']}",2.239462,Abschlussarbeit
32,Abschlussprüfung,abschlussprüfung,NN,NN,92vRkkxFBJ,NOM,N,lexAlt manuell,"{'lexAlt': ['Abschlußprüfung', 'Abschlußprüfungen'], 'Genus': ['f']}",2.239462,Abschlussprüfung
...,...,...,...,...,...,...,...,...,...,...,...
1767,überfahren,überfahren,VVFIN,VVFIN,z5RNztRgLK,VER,V,manuell,{},2.239462,überfahren
1774,überladen,überladen,VVFIN,VVFIN,fl5mKfU8Kt,VER,V,manuell,{},2.239462,überladen
1781,überschlagen,überschlagen,VVFIN,VVFIN,zhf9FrV2E4,VER,V,manuell,{},2.239462,überschlagen
1782,überschäumen,überschäumen,VVFIN,VVFIN,8AqHFyhw3D,VER,V,manuell,{'Ptz1': ['überschäumend']},2.239462,überschäumen


In [168]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,flexion,score,title
207,Euro,euro,NN,NN,bG593aDj9w,NOM,N,Einheit manuell,"{'abk': ['EUR', '€']}",2.239462,Euro
1698,zu viel,zu viel,ADV,ADV,tJAqgsSpAN,ADV,A,manuell,"{'ADV': ['zu viele', 'zuviele']}",2.239462,zu viel


In [169]:
#suche_debug

In [170]:
# anschauen: mehr als ein tag
mask = (lexeme_manuell.tag.str.split().str.len() > 1)
lexeme_manuell[mask]

,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,flexion,score,title
1636,welch,welch,PRELS PWAT,PRELS,PWAT,PRO,P,manuell,"{'lex': ['welche', 'welchem', 'welchen', 'welcher', 'welches']}",2.239462,welch


In [171]:
# section_id2 schreiben
lexeme_manuell['section_id2'] = lexeme_manuell.lemma.str.replace(' ','_') + '_M'
mask = lexeme_manuell.tag == 'CARD'
lexeme_manuell.loc[mask,'section_id2'] = lexeme_manuell[mask].lemma.str.replace(' ','_') + '_Z'
lexeme_manuell = pak.move_cols(lexeme_manuell, ['section_id2','title'])

In [172]:
# suche_debug
#search_str(lexeme_manuell, suche_debug)

In [173]:
# Bekannte section_id2 aus wiktionary übertragen

mask = (wiktionary.lemma == wiktionary.title)  # wichtig! Gerade die Datensätze, die nicht darunter fallen, verursachen Probleme bei Partizipien.

# tagZZ
lexeme_manuell = pak.update_col(lexeme_manuell, wiktionary[mask], on=['lemma_lower','tagZZ'], col='section_id2',  col_score='score'  )

#tagZ
lexeme_manuell = pak.update_col(lexeme_manuell, wiktionary[mask], on=['lemma_lower','tagZ'],  col='section_id2',  col_score='score'  )

# tag
lexeme_manuell = pak.update_col(lexeme_manuell, wiktionary[mask], on=['lemma_lower','tag'],   col='section_id2',  col_score='score'  )

# Wenn ein sinnvoller tag fehlt, reicht lemma
mask = (wiktionary.lemma == wiktionary.title)   &   wiktionary.tag.str.contains('TODO')
lexeme_manuell = pak.update_col(lexeme_manuell, wiktionary[mask], on=['lemma'], col='section_id2',  col_score='score' )

update_col: col_score score applied, 3336 records less!
update_col: 1194 cells written into existing column
update_col: col_score score applied, 2318 records less!
update_col: 1170 cells written into existing column
update_col: col_score score applied, 3539 records less!
update_col: 1155 cells written into existing column
update_col: col_score score applied, 4 records less!
update_col: 34 cells written into existing column


In [174]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

,section_id2,title,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,flexion,score
207,Euro,Euro,Euro,euro,NN,NN,bG593aDj9w,NOM,N,Einheit manuell,"{'abk': ['EUR', '€']}",2.239462
1698,zu_viel,zu viel,zu viel,zu viel,ADV,ADV,tJAqgsSpAN,ADV,A,manuell,"{'ADV': ['zu viele', 'zuviele']}",2.239462


In [175]:
# anschauen
#grid(lexeme_manuell, pak.sample_100)

In [176]:
bpy.minivenn(wiktionary.columns, lexeme_manuell.columns, format='print2')

left_only:  {'ipa', 'lang', 'page_id', 'page_part', 'syllables', 'lang_code', 'error', 'rhymes', 'pos', 'inflected'}
right_only: {}



In [177]:
# anfügen
wiktionary = pak.add_rows(wiktionary, lexeme_manuell, assert_subset=True)

1792 rows added, now a total of 843357


In [178]:
# section_id2 ist jetzt nicht mehr eindeutig
mask = wiktionary.duplicated(subset=['section_id2'], keep=False)
check_mask(wiktionary,mask,5507) 
#a = wiktionary[mask].sort_values(['section_id2'])
#grid(a)

check_mask:  4932 rows


In [179]:
assert pak.ntypes(wiktionary.flexion) == 1

## ipa, syllables & rhymes

In [180]:
# Datentypen sind in jeder Zeile gleich
assert pak.ntypes(wiktionary.ipa) == 1
assert pak.ntypes(wiktionary.syllables) == 1

In [181]:
# Listen zu Strings
mask = ~wiktionary.ipa.isnull()
wiktionary.loc[mask,'ipa'] = wiktionary[mask].ipa.str[0]

mask = ~wiktionary.syllables.isnull()
wiktionary.loc[mask,'syllables'] = wiktionary[mask].syllables.str.join('⇀')

mask = ~wiktionary.rhymes.isnull()
wiktionary.loc[mask,'rhymes'] = wiktionary[mask].rhymes.str[0]

In [182]:
# cols_to_defaultdict
# 1 min
wiktionary = pak.cols_to_defaultdict( wiktionary, col_dict='flexion', cols_add=['ipa','syllables','rhymes']  )

In [183]:
# wiktionary anschauen
#grid(wiktionary, pak.sample)

## flexion nach wiktionary_merkmal exploden
* braucht viel RAM
* hiernach muss die löschliste nochmal angewendet werden

In [184]:
# flexion: vorbereiten
# 5 min
mask1 = wiktionary.flexion.notnull() 
mask2 = wiktionary.flexion != {}
mask3 = wiktionary.flexion != defaultdict(list)
mask = mask1  &  mask2  &  mask3 

spalten = ['section_id2','flexion']
df = pak.explode_dict(wiktionary[spalten][mask], col_dict='flexion', col_key='node_kontext', col_value='data', from_defaultdict=True)
df['merkmal'] = 'flexion'

In [185]:
# change_datatype
wiktionary = pak.change_datatype(wiktionary, verbose=False)

In [186]:
# Beispiel anschauen
mask = df.section_id2 == suche_debug[0]
grid(df,mask)

19 rows out of 3006801


,section_id2,node_kontext,data,merkmal
6687,Euro,Genus,m,flexion
6687,Euro,Nominativ Singular,Euro,flexion
6687,Euro,Genitiv Singular,Euro,flexion
6687,Euro,Dativ Singular,Euro,flexion
6687,Euro,Akkusativ Singular,Euro,flexion
6687,Euro,ipa,ˈɔɪ̯ʁo,flexion
6687,Euro,syllables,Eu⇀ro,flexion
6687,Euro,rhymes,ɔɪ̯ʁo,flexion
6687,Euro,Nominativ Plural 1,Euros,flexion
6687,Euro,Nominativ Plural 2,Euro,flexion


In [187]:
#bpy.minivenn(wiktionary_merkmal.columns, df.columns)

In [188]:
# an wiktionary_merkmal anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, df, assert_subset=True)
wiktionary   = pak.drop_cols(wiktionary, ['flexion'])  

3006801 rows added, now a total of 6261406


In [189]:
# suche_debug
mask = wiktionary_merkmal.section_id2.isin(suche_debug[0:1])
grid(wiktionary_merkmal[mask].sort_values(['section_id2','merkmal']))

58 rows


,section_id2,merkmal,node_debug,node_kontext,data,num,meta
243538,Euro,abk,,,EUR,1,Währungszeichen
243514,Euro,def,,,Währung,1,
243515,Euro,def,,,Europäischen Währungsunion,1,
243518,Euro,down,,,Pendlereuro,1,
243519,Euro,familie,,,Euro-Abschaffung,,
243520,Euro,familie,,,Eurobanknote,,
243521,Euro,familie,,,Eurobond,,
243522,Euro,familie,,,Eurocent,,
243523,Euro,familie,,,Euro-Einführung,,
243524,Euro,familie,,,Euro-Gedenkmünze,,


## wiktionary in 3 Teile aufteilen

In [190]:
# suche_debug 
mask = wiktionary.title.isin(suche_debug)
grid(wiktionary[mask].sort_values(['section_id2',]))

7 rows


,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,tag_0,tag_1,tagZ,tagZZ,member,lang,lang_code,error,lemma_lower,score
6687,11127,0,Euro,Euro,Euro,False,{'Substantiv': []},NN,NN,Q34cMU8Jzm,NOM,N,Substantiv,Deutsch,de,,euro,153.715988
841772,<NA>,<NA>,Euro,Euro,Euro,NaN,NaN,NN,NN,bG593aDj9w,NOM,N,Einheit manuell,<NA>,<NA>,<NA>,euro,2.239461
6688,11127,1,Euro_1,Euro,Euro,False,{'Substantiv': []},NN,NN,vhokubi30Y,NOM,N,Substantiv,Deutsch,de,,euro,4.110498
128971,284496,0,tugendhaftestes,tugendhaftestes,tugendhaft,True,{'Deklinierte Form': []},LEX_A,LEX_A,Zx6kMHkYfH,<NA>,,Deklinierte_Form,Deutsch,de,,tugendhaft,1.194590
31740,89575,0,zu_viel,zu viel,zu viel,False,"{'Adverb': [], 'Wortverbindung': []}",ADV,ADV,wpQ70rl7by,ADV,A,Adverb Wortverbindung,Deutsch,de,,zu viel,14.845442
843263,<NA>,<NA>,zu_viel,zu viel,zu viel,NaN,NaN,ADV,ADV,tJAqgsSpAN,ADV,A,manuell,<NA>,<NA>,<NA>,zu viel,2.239461
558334,820148,0,zu_viele,zu viele,zu viel,True,{'Wortverbindung': []},TODO,TODO,sWEsmc8dOR,X_todo,X,Wortverbindung,Deutsch,de,,zu viel,1.020573


In [191]:
# pflege_tag_felder (zur Sicherheit noch mal)
wiktionary = pflege_tag_felder(wiktionary, translate_tagZ)

In [192]:
# LEX-Kennzeichnung sollte eineindeutig sein, außer bei FIX
mask1 =  wiktionary.tag.str.contains('LEX_')   
mask2 =  wiktionary.tag.str.contains(' ')
mask3 = ~wiktionary.tag.str.contains('FIX')
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary,mask, 0)

check_mask:  0 rows


In [193]:
# inflected >> bool
wiktionary['inflected'] = wiktionary.inflected.astype('bool')

In [194]:
pak.nnan(wiktionary)

page_id        6336
page_part      6336
pos            6340
tagZ         669686
lang           6336
lang_code      6336
error          6336
dtype: int64

In [195]:
# score
mask = wiktionary.score.isnull()
check_mask(wiktionary,mask,0)
#wiktionary['score'] = wiktionary.score.fillna(wiktionary.score.quantile(0.95)).astype('float')

check_mask:  0 rows


In [196]:
pak.analyse_cols(wiktionary)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,int64,np.int64,int64,np.int32,True,False,True,False,80.0 B,1,843357,0,0,843357,0.0,421678.00,421678.00,843356.0,3.556251e+11
1,page_id,int32,pd.Int32,Int32,,True,False,True,True,50.0 B,1,801553,6336,35468,843357,0.0,622283.84,623269.00,1118318.0,5.208646e+11
2,page_part,int8,pd.Int8,Int8,,True,False,True,True,20.0 B,1,7,6336,837014,843357,0.0,0.04,0.00,6.0,3.743400e+04
3,section_id2,str,pd.string,string,,False,True,True,True,736.0 B,1,840889,0,2468,843357,!_M,NaN,NaN,…_M,NaN
4,title,str,pd.string,string,,False,True,True,True,730.0 B,1,805228,0,38129,843357,!,NaN,NaN,…,NaN
5,lemma,str,pd.string,string,,False,True,True,True,720.0 B,1,163093,0,680264,843357,!,NaN,NaN,…,NaN
6,inflected,bool_,bool,bool,,True,False,True,True,10.0 B,1,2,0,843355,843357,0.0,0.83,NaN,1.0,7.032070e+05
7,pos,dict,object,object,,False,True,False,True,1.8 KB,1,159,6340,836858,843357,NaN,NaN,NaN,NaN,NaN
8,tag,str,pd.string,string,,False,True,True,True,611.0 B,1,64,0,843293,843357,$(,NaN,NaN,ZFIX TRUNC,NaN
9,tag_0,str,pd.string,string,,False,True,True,True,611.0 B,1,43,0,843314,843357,$(,NaN,NaN,ZFIX,NaN


In [197]:
#pak.sample(wiktionary)

### wiktionary_partizip
* Enthält Partizipien in Form von Partizipialadjektiven
* Aber keine Verben mit Partizip-Lexem!

In [198]:
# Partizipien nach wiktionary_partizip abspalten
mask1 = wiktionary.tag == 'ADJA'
mask2 = wiktionary.member.str.contains('Ptz')  
mask = mask1  &  mask2
wiktionary, wiktionary_partizip = pak.move_rows( wiktionary, mask, msg=None )  

Move 27156 rows from 843357


In [199]:
## verblieben in wiktionary
mask1 = (wiktionary.lemma == wiktionary.title)  
mask2 = wiktionary.lemma.isin(suche_debug) 
mask = mask1 & mask2
wiktionary[mask]

,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,tag_0,tag_1,tagZ,tagZZ,member,lang,lang_code,error,lemma_lower,score
6687,11127,0,Euro,Euro,Euro,False,{'Substantiv': []},NN,NN,OYXC6QnBDy,NOM,N,Substantiv,Deutsch,de,,euro,153.715988
6688,11127,1,Euro_1,Euro,Euro,False,{'Substantiv': []},NN,NN,HZs41oOmIz,NOM,N,Substantiv,Deutsch,de,,euro,4.110498
31740,89575,0,zu_viel,zu viel,zu viel,False,"{'Adverb': [], 'Wortverbindung': []}",ADV,ADV,D9DicGsKDl,ADV,A,Adverb Wortverbindung,Deutsch,de,,zu viel,14.845442
841772,<NA>,<NA>,Euro,Euro,Euro,True,NaN,NN,NN,P0rXszKHZY,NOM,N,Einheit manuell,<NA>,<NA>,<NA>,euro,2.239461
843263,<NA>,<NA>,zu_viel,zu viel,zu viel,True,NaN,ADV,ADV,nuhsBel9aV,ADV,A,manuell,<NA>,<NA>,<NA>,zu viel,2.239461


In [200]:
## jetzt in wiktionary_partizip
mask = wiktionary_partizip.lemma.isin(suche_debug) 
wiktionary_partizip[mask]

,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,tag_0,tag_1,tagZ,tagZZ,member,lang,lang_code,error,lemma_lower,score


In [201]:
mask1 = wiktionary.tag == 'ADJA'
mask2 = wiktionary.lemma.isin(wiktionary_partizip.lemma) 
mask = mask1  &  mask2
pak.sample(wiktionary[mask])
#gridt(wiktionary,mask)

,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,tag_0,tag_1,tagZ,tagZZ,member,lang,lang_code,error,lemma_lower,score
2016,2864,2,wollen_2,wollen,wollen,False,{'Adjektiv': []},ADJA,ADJA,0fd5vOynw1,ADJ,A,Adjektiv,Deutsch,de,,wollen,4.586331
5803,9281,0,übertragen,übertragen,übertragen,False,{'Adjektiv': []},ADJA,ADJA,4xvLdQtcPq,ADJ,A,Adjektiv,Deutsch,de,,übertragen,49.847878
15286,25944,0,albern,albern,albern,False,{'Adjektiv': []},ADJA,ADJA,Lkect96BJP,ADJ,A,Adjektiv,Deutsch,de,,albern,36.796616
21384,57003,0,vergeben,vergeben,vergeben,False,{'Adjektiv': []},ADJA,ADJA,61gN8AAeOr,ADJ,A,Adjektiv,Deutsch,de,,vergeben,158.687744
36173,101324,0,erwachsen,erwachsen,erwachsen,False,{'Adjektiv': []},ADJA,ADJA,3L2jjK4REb,ADJ,A,Adjektiv,Deutsch,de,,erwachsen,121.997086
86973,200956,2,überlaufen_2,überlaufen,überlaufen,False,{'Adjektiv': []},ADJA,ADJA,YLz9EwQMdc,ADJ,A,Adjektiv,Deutsch,de,,überlaufen,0.978308
95883,240121,0,durchwachsen,durchwachsen,durchwachsen,False,{'Adjektiv': []},ADJA,ADJA,zkhdvdPLAf,ADJ,A,Adjektiv,Deutsch,de,,durchwachsen,37.651863
430493,641508,0,korken,korken,korken,False,{'Adjektiv': []},ADJA,ADJA,0XEwT3S51U,ADJ,A,Adjektiv,Deutsch,de,,korken,11.109015
747327,1017955,0,auserlesen,auserlesen,auserlesen,False,{'Adjektiv': []},ADJA,ADJA,3Gf2UH6NVF,ADJ,A,Adjektiv,Deutsch,de,,auserlesen,20.579056
843177,<NA>,<NA>,vorenthalten_M,vorenthalten,vorenthalten,True,NaN,ADJA,ADJA,xfHPmXn1mx,ADJ,A,manuell ergänzt,<NA>,<NA>,<NA>,vorenthalten,2.239461


### wiktionary_nolemma

In [202]:
wiktionary_nolemma       = wiktionary.head(0)
wiktionary_nolemma_trash = wiktionary.head(0)

In [203]:
# LEX-Datensätze nach wiktionary_nolemma verschieben
mask = wiktionary.tag.str.contains('LEX_')    
check_mask(wiktionary,mask,639734)
wiktionary, wiktionary_nolemma = pak.move_rows( df_from=wiktionary, df_to=wiktionary_nolemma, mask=mask, msg='tag_LEX' )  

check_mask:  669775 rows
Move 669775 rows from 816201


In [204]:
# auch nach wiktionary_nolemma verschieben
mask = wiktionary.member.str.contains('Deklinierte_Form')   # 2 Datensätze
check_mask(wiktionary, mask, 0, 10)
wiktionary, wiktionary_nolemma = pak.move_rows( df_from=wiktionary, df_to=wiktionary_nolemma, mask=mask, msg='member_Deklinierte_Form' )  

check_mask:  1 rows
Move 1 rows from 146426


In [205]:
# auch nach wiktionary_nolemma verschieben
mask = (wiktionary.title != wiktionary.lemma)               
check_mask(wiktionary, mask, 10)
wiktionary, wiktionary_nolemma = pak.move_rows( df_from=wiktionary, df_to=wiktionary_nolemma, mask=mask, msg='title_ungleich_lemma' )  

check_mask:  10 rows
Move 10 rows from 146425


In [206]:
# wiktionary suche_debug
grid(search_str(wiktionary_nolemma, suche_debug))

3 rows


,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,tag_0,tag_1,tagZ,tagZZ,member,lang,lang_code,error,lemma_lower,score,msg
63565,149864,0,Euros,Euros,Euro,True,{'Deklinierte Form': []},LEX_A,LEX_A,4Lg01EzDZP,<NA>,,Deklinierte_Form,Deutsch,de,,euro,0.908025,tag_LEX
128971,284496,0,tugendhaftestes,tugendhaftestes,tugendhaft,True,{'Deklinierte Form': []},LEX_A,LEX_A,cJwt4iRVUK,<NA>,,Deklinierte_Form,Deutsch,de,,tugendhaft,1.194590,tag_LEX
558334,820148,0,zu_viele,zu viele,zu viel,True,{'Wortverbindung': []},TODO,TODO,fvpCzBBWAJ,X_todo,X,Wortverbindung,Deutsch,de,,zu viel,1.020573,title_ungleich_lemma


In [207]:
# wiktionary_nolemma: Spalten pflegen
wiktionary_nolemma = pak.rename_col(wiktionary_nolemma, 'title', 'lex')
wiktionary_nolemma = pak.rename_col(wiktionary_nolemma, 'section_id2','nolemma_id')
wiktionary_nolemma = pak.rename_col(wiktionary_nolemma, 'score', 'data_score')
wiktionary_nolemma = pak.drop_cols(wiktionary_nolemma, ['pos','tag_anz','ipa','syllables','msg','page_id','page_part','inflected','rhymes'])
#wiktionary_nolemma = pak.rename_col( wiktionary_nolemma, 'tag_0', 'tag') # Jetzt gibt es definitiv keine Mehrfachtags mehr

In [208]:
# anschauen
pak.sample(wiktionary_nolemma)

,nolemma_id,lex,lemma,tag,tag_0,tag_1,tagZ,tagZZ,member,lang,lang_code,error,lemma_lower,data_score
29,lieben_1,lieben,lieb,LEX_A,LEX_A,WfA2BQe1xj,<NA>,,Deklinierte_Form,Deutsch,de,,lieb,0.311824
58,Polen_1,Polen,Pole,LEX_A,LEX_A,LcLpF2GQL1,<NA>,,Deklinierte_Form,Deutsch,de,,pole,0.113761
30558,Hupen,Hupen,Hupe,LEX_A,LEX_A,dolmZw9GFp,<NA>,,Deklinierte_Form,Deutsch,de,,hupe,0.794388
37341,Fachleute,Fachleute,Fachleute,LEX_A,LEX_A,2sWOYFAj2t,<NA>,,Deklinierte_Form Substantiv,Deutsch,de,,fachleute,63.546085
110139,β-Ketobuttersäuren,β-Ketobuttersäuren,β-Ketobuttersäure,LEX_A,LEX_A,9vbc4mkd6E,<NA>,,Deklinierte_Form,Deutsch,de,,β-ketobuttersäure,0.899760
414781,Körpermasse_1,Körpermasse,Körpermasse,LEX_A,LEX_A,yUSdnEgOhm,<NA>,,Deklinierte_Form,Deutsch,de,,körpermasse,0.002013
497450,durchzufallen,durchzufallen,durchfallen,LEX_V,LEX_V,s6b4opIZBv,<NA>,,Erweiterter_Infinitiv Verb,Deutsch,de,,durchfallen,0.484471
558334,zu_viele,zu viele,zu viel,TODO,TODO,fvpCzBBWAJ,X_todo,X,Wortverbindung,Deutsch,de,,zu viel,1.020573
562464,umgebildetes,umgebildetes,umgebildet,LEX_A,LEX_A,zzzYy0qKCV,<NA>,,Deklinierte_Form,Deutsch,de,,umgebildet,1.111256
648221,daraufkam,daraufkam,daraufkommen,LEX_V,LEX_V,zYcJlzO5SA,<NA>,,Konjugierte_Form Verb,Deutsch,de,,daraufkommen,0.839843


### wiktionary_lemma

In [209]:
# wiktionary_lemma: erstellen und Spalten pflegen
wiktionary_lemma  = pak.drop_cols(wiktionary, ['title','pos'])
wiktionary_lemma = pak.rename_col(wiktionary_lemma, 'score', 'lemma_score')

In [210]:
wiktionary_lemma_trash = wiktionary_lemma.head(0)

In [211]:
#del wiktionary

In [212]:
# wiktionary_lemma suche_debug
grid(search_str(wiktionary_lemma, suche_debug))

5 rows


,page_id,page_part,section_id2,lemma,inflected,tag,tag_0,tag_1,tagZ,tagZZ,member,lang,lang_code,error,lemma_lower,lemma_score
6687,11127,0,Euro,Euro,False,NN,NN,OYXC6QnBDy,NOM,N,Substantiv,Deutsch,de,,euro,153.715988
6688,11127,1,Euro_1,Euro,False,NN,NN,HZs41oOmIz,NOM,N,Substantiv,Deutsch,de,,euro,4.110498
31740,89575,0,zu_viel,zu viel,False,ADV,ADV,D9DicGsKDl,ADV,A,Adverb Wortverbindung,Deutsch,de,,zu viel,14.845442
841772,<NA>,<NA>,Euro,Euro,True,NN,NN,P0rXszKHZY,NOM,N,Einheit manuell,<NA>,<NA>,<NA>,euro,2.239461
843263,<NA>,<NA>,zu_viel,zu viel,True,ADV,ADV,nuhsBel9aV,ADV,A,manuell,<NA>,<NA>,<NA>,zu viel,2.239461


## Partizipien
Die Partizipien werden<br> 
a) an wiktionary_merkmal angefügt (als Partizip-Flexion des Verbs)<br> 
b) an wiktionary_lemma angefügt   (als Adjektiv mit member Partizip)

Danach findet sich alles aus wiktionary in wiktionary_lemma und wiktionary_nolemma

### wiktionary_partizip pflegen

In [213]:
pak.sample(wiktionary_partizip)

,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,tag_0,tag_1,tagZ,tagZZ,member,lang,lang_code,error,lemma_lower,score
419,668,1,sehen_1,sehen,sehen,False,{'Adjektiv': ['Partizip II']},ADJA,ADJA,aGOAp8wPmZ,ADJ,A,Adjektiv Ptz2,Deutsch,de,,sehen,1.055928
4891,7368,3,verschieden_3,verschieden,verscheiden,True,{'Adjektiv': ['Partizip II']},ADJA,ADJA,vjvATM8vwt,ADJ,A,Adjektiv Ptz2,Deutsch,de,,verscheiden,0.003958
29334,78413,1,umgehend_1,umgehend,umgehen,True,{'Adjektiv': ['Partizip I']},ADJA,ADJA,z8P1eElIIn,ADJ,A,Adjektiv Ptz1,Deutsch,de,,umgehen,0.064691
32489,91988,4,überdacht_4,überdacht,überdenken,True,{'Adjektiv': ['Partizip II']},ADJA,ADJA,Uc84kYNblW,ADJ,A,Adjektiv Ptz2,Deutsch,de,,überdenken,0.028046
35035,98623,0,indiziert,indiziert,indizieren,True,{'Adjektiv': ['Partizip II']},ADJA,ADJA,eH90oCPmsj,ADJ,A,Adjektiv Ptz2,Deutsch,de,,indizieren,0.459678
61219,144189,0,bedienend,bedienend,bedienend,False,{'Adjektiv': ['Partizip I']},ADJA,ADJA,BWM0fdvj1n,ADJ,A,Adjektiv Ptz1,Deutsch,de,,bedienend,2.194083
249813,428293,0,kredenzend,kredenzend,kredenzen,True,{'Adjektiv': ['Partizip I']},ADJA,ADJA,005KBXAC64,ADJ,A,Adjektiv Ptz1,Deutsch,de,,kredenzen,0.668355
280596,463118,0,geaalt,geaalt,aalen,True,{'Adjektiv': ['Partizip II']},ADJA,ADJA,nGCPm2Uccm,ADJ,A,Adjektiv Ptz2,Deutsch,de,,aalen,0.436950
368665,555699,0,Acht_gebend,Acht gebend,Acht geben,True,{'Adjektiv': ['Partizip I']},ADJA,ADJA,drIyJi8lQk,ADJ,A,Adjektiv Ptz1,Deutsch,de,,acht geben,0.209178
402320,594074,0,harangierend,harangierend,harangieren,True,{'Adjektiv': ['Partizip I']},ADJA,ADJA,00pNrk5d8k,ADJ,A,Adjektiv Ptz1,Deutsch,de,,harangieren,0.503066


In [214]:
# rename_col
wiktionary_partizip = pak.rename_col(wiktionary_partizip,  'section_id2', 'adjektiv_id' )
wiktionary_partizip = pak.rename_col(wiktionary_partizip,  'title',       'adjektiv'    )
wiktionary_partizip = pak.rename_col(wiktionary_partizip,  'lemma',       'verb'        )

#wiktionary_partizip = pak.move_cols(wiktionary_partizip,'verb_lower','verb')

In [215]:
# drop_cols
lösche_spalten = ['pos','error','lang','lang_code','lemma_lower','inflected','tag_0','tag_1']
wiktionary_partizip = pak.drop_cols(wiktionary_partizip,lösche_spalten)

In [216]:
# change_datatype
wiktionary_partizip = pak.change_datatype(wiktionary_partizip)

change_datatype 
change_datatype before: 12.5 MB after: 12.5 MB



In [217]:
# falsche Partizipien löschen
falsche_partizipien = ['sehen','hören','wollen','lassen','mögen','sollen',]
mask = wiktionary_partizip.adjektiv.isin(falsche_partizipien) 
check_mask(wiktionary_partizip, mask, 0, 6)

wiktionary_partizip, df = pak.move_rows(wiktionary_partizip, mask)
df

check_mask:  6 rows
Move 6 rows from 27156


,page_id,page_part,adjektiv_id,adjektiv,verb,tag,tagZ,tagZZ,member,score,msg
419,668,1,sehen_1,sehen,sehen,ADJA,ADJ,A,Adjektiv Ptz2,1.055928,10
1098,1467,1,hören_1,hören,hören,ADJA,ADJ,A,Adjektiv Ptz2,0.747702,10
2015,2864,1,wollen_1,wollen,wollen,ADJA,ADJ,A,Adjektiv Ptz2,0.409591,10
2024,2873,1,lassen_1,lassen,lassen,ADJA,ADJ,A,Adjektiv Ptz2,0.481364,10
6186,10069,1,mögen_1,mögen,mögen,ADJA,ADJ,A,Adjektiv Ptz2,0.341409,10
10318,17830,1,sollen_1,sollen,sollen,ADJA,ADJ,A,Adjektiv Ptz2,0.414756,10


In [218]:
# wiktionary suche_debug
grid(search_str(wiktionary_partizip, suche_debug))

No rows


In [219]:
# So sehen normale Partizipien 1 aus
mask1 = wiktionary_partizip.verb.str.endswith(verbendungen)
mask2 = wiktionary_partizip.member.str.contains('Ptz1')
mask = mask1 & mask2
check_mask(wiktionary_partizip,mask,13554)

pak.sample(wiktionary_partizip[mask])

check_mask:  13543 rows


,page_id,page_part,adjektiv_id,adjektiv,verb,tag,tagZ,tagZZ,member,score
2186,3071,1,bedeutend_1,bedeutend,bedeuten,ADJA,ADJ,A,Adjektiv Ptz1,0.059268
20750,54869,3,überwiegend_3,überwiegend,überwiegen,ADJA,ADJ,A,Adjektiv Ptz1,0.051715
29334,78413,1,umgehend_1,umgehend,umgehen,ADJA,ADJ,A,Adjektiv Ptz1,0.064691
56866,134898,0,Rad_fahrend,Rad fahrend,Rad fahren,ADJA,ADJ,A,Adjektiv Ptz1,0.364487
124504,279022,0,absorbierend,absorbierend,absorbieren,ADJA,ADJ,A,Adjektiv Ptz1,0.503066
261717,443029,0,aufziehend,aufziehend,aufziehen,ADJA,ADJ,A,Adjektiv Ptz1,1.435615
299864,483152,1,leer_stehend_1,leer stehend,leer stehen,ADJA,ADJ,A,Adjektiv Ptz1,0.032452
368665,555699,0,Acht_gebend,Acht gebend,Acht geben,ADJA,ADJ,A,Adjektiv Ptz1,0.209178
436893,653374,0,auspowernd,auspowernd,auspowern,ADJA,ADJ,A,Adjektiv Ptz1,0.414223
452113,675746,0,Bescheid_sagend,Bescheid sagend,Bescheid sagen,ADJA,ADJ,A,Adjektiv Ptz1,0.267029


In [220]:
# Dies sind kaputte Partizipien 1: Das verb ist falsch, kann aber leicht korrigiert werden
mask1 = ~wiktionary_partizip.verb.str.endswith(verbendungen)
mask2 =  wiktionary_partizip.member.str.contains('Ptz1')
mask3 =  wiktionary_partizip.adjektiv == wiktionary_partizip.verb
mask4 =  wiktionary_partizip.verb.str.endswith(('end','ernd'))
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_partizip,mask,11)

wiktionary_partizip.loc[mask,'verb'] = wiktionary_partizip[mask].verb.str[:-1]
wiktionary_partizip[mask]

check_mask:  11 rows


,page_id,page_part,adjektiv_id,adjektiv,verb,tag,tagZ,tagZZ,member,score
34900,98181,2,betreffend_2,betreffend,betreffen,ADJA,ADJ,A,Adjektiv Ptz1,0.129810
46086,120123,0,lachend,lachend,lachen,ADJA,ADJ,A,Adjektiv Ptz1,2.053587
52759,129786,0,denkend,denkend,denken,ADJA,ADJ,A,Adjektiv Ptz1,1.408959
57135,135210,0,bedenkend,bedenkend,bedenken,ADJA,ADJ,A,Adjektiv Ptz1,1.822182
61211,144143,0,dienend,dienend,dienen,ADJA,ADJ,A,Adjektiv Ptz1,1.756066
61212,144145,0,achtend,achtend,achten,ADJA,ADJ,A,Adjektiv Ptz1,1.970942
61219,144189,0,bedienend,bedienend,bedienen,ADJA,ADJ,A,Adjektiv Ptz1,2.194083
61220,144191,0,bedingend,bedingend,bedingen,ADJA,ADJ,A,Adjektiv Ptz1,1.805653
61222,144197,0,bejagend,bejagend,bejagen,ADJA,ADJ,A,Adjektiv Ptz1,2.012264
61223,144199,0,beklagend,beklagend,beklagen,ADJA,ADJ,A,Adjektiv Ptz1,1.830446


In [221]:
# So sehen normale Partizipien 2 aus
mask1 = wiktionary_partizip.verb.str.endswith(verbendungen)
mask2 = wiktionary_partizip.member.str.contains('Ptz2')
mask = mask1 & mask2
check_mask(wiktionary_partizip,mask,13592)

pak.sample(wiktionary_partizip[mask])

check_mask:  13592 rows


,page_id,page_part,adjektiv_id,adjektiv,verb,tag,tagZ,tagZZ,member,score
476,749,2,dezidiert_2,dezidiert,dezidieren,ADJA,ADJ,A,Adjektiv Ptz2,0.057074
4891,7368,3,verschieden_3,verschieden,verscheiden,ADJA,ADJ,A,Adjektiv Ptz2,0.003958
19780,49520,0,gedacht,gedacht,denken,ADJA,ADJ,A,Adjektiv Ptz2,0.548521
32489,91988,4,überdacht_4,überdacht,überdenken,ADJA,ADJ,A,Adjektiv Ptz2,0.028046
35035,98623,0,indiziert,indiziert,indizieren,ADJA,ADJ,A,Adjektiv Ptz2,0.459678
63029,149154,0,geschrien,geschrien,schreien,ADJA,ADJ,A,Adjektiv Ptz2,2.057463
239754,415704,0,geheißen,geheißen,heißen,ADJA,ADJ,A,Adjektiv Ptz2,0.945197
319807,503505,0,Acht_gegeben,Acht gegeben,Acht geben,ADJA,ADJ,A,Adjektiv Ptz2,0.214343
373013,560657,0,abgenickt,abgenickt,abnicken,ADJA,ADJ,A,Adjektiv Ptz2,0.401826
467077,697570,0,instand_gesetzt,instand gesetzt,instand setzen,ADJA,ADJ,A,Adjektiv Wortverbindung Ptz2,0.491591


In [222]:
# Dies sind kaputte Partizipien 2: Das verb ist falsch, kann auch nicht gut korrigiert werden.
mask1 = ~wiktionary_partizip.verb.str.endswith(verbendungen)
mask2 = wiktionary_partizip.member.str.contains('Ptz2')
mask3 = wiktionary_partizip.adjektiv == wiktionary_partizip.verb
mask4 = wiktionary_partizip.verb.str.endswith('t')
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_partizip,mask,0,10)

wiktionary_partizip[mask]

check_mask:  4 rows


,page_id,page_part,adjektiv_id,adjektiv,verb,tag,tagZ,tagZZ,member,score
87490,209588,1,unterminiert_1,unterminiert,unterminiert,ADJA,ADJ,A,Adjektiv Ptz2,1.658692
121851,275617,0,verspeist,verspeist,verspeist,ADJA,ADJ,A,Adjektiv Ptz2,1.813917
467076,697568,1,instandgesetzt_1,instandgesetzt,instandgesetzt,ADJA,ADJ,A,Adjektiv Ptz2,0.098127
491822,734226,1,weiterverhandelt_1,weiterverhandelt,weiterverhandelt,ADJA,ADJ,A,Adjektiv Ptz2,0.295955


In [223]:
# Kaputte Partizipien 2 löschen
wiktionary_partizip = pak.drop_rows(wiktionary_partizip,mask)

Delete 4 rows from 27150


In [224]:
# Gibt es weitere kaputte Partizipien?
mask = ~wiktionary_partizip.verb.str.endswith(verbendungen)
check_mask(wiktionary_partizip,mask,0)
#grid(wiktionary_partizip,mask)

check_mask:  0 rows


In [225]:
# Wenn es verb nicht auch als Wort gibt, dann ist das Müll und kann weg.
# Wenn man das nicht will: Als Partizipien in lexeme_manuell aufnehmen!

mask = ~wiktionary_partizip.verb.isin(wiktionary_lemma.lemma)
check_mask(wiktionary_partizip,mask,0,10)
wiktionary_partizip,df = pak.move_rows(wiktionary_partizip,mask)
df

check_mask:  5 rows
Move 5 rows from 27146


,page_id,page_part,adjektiv_id,adjektiv,verb,tag,tagZ,tagZZ,member,score,msg
7300,12329,2,versiert_2,versiert,versieren,ADJA,ADJ,A,Adjektiv Ptz2,0.060001,10
27620,72640,1,gesamt_1,gesamt,samen,ADJA,ADJ,A,Adjektiv Ptz2,0.065466,10
450762,674038,0,wullackend,wullackend,wullacken,ADJA,ADJ,A,Adjektiv Ptz1,0.459678,10
450763,674039,0,gewullackt,gewullackt,wullacken,ADJA,ADJ,A,Adjektiv Ptz2,0.405959,10
529726,789253,0,mechend,mechend,mechen,ADJA,ADJ,A,Adjektiv Ptz1,0.443149,10


In [226]:
mask = wiktionary_lemma.tagZZ == 'V'
pak.analyse_freqs( wiktionary_lemma[mask], ['tag', 'member'])
#wiktionary_lemma[mask]

,tag,tag_count,tag_percent,member,member_count
0,VVFIN,14336,99.9,"[Verb, manuell, lexAlt manuell, Verb Wortverbindung, manuell ergänzt, Verb Redewendung Wortverbindung, Verb Redewendung]","[13796, 402, 84, 41, 11, 1, 1]"
1,VAFIN,7,0.0,"[Verb Hilfsverb, manuell]","[4, 3]"
2,VMFIN,7,0.0,"[manuell, lexAlt manuell]","[6, 1]"


In [227]:
# verb_id ermitteln
mask = wiktionary_lemma.tagZZ == 'V'
wiktionary_partizip = pak.update_col( wiktionary_partizip,  wiktionary_lemma[mask], 
                                      left_on='verb',       right_on='lemma', 
                                      col_rename='verb_id', col='section_id2',    
                                                            col_score='lemma_score' 
                                    )
wiktionary_partizip = pak.move_cols(wiktionary_partizip, 'verb_id','adjektiv')

update_col: col_score lemma_score applied, 877 records less!
update_col: 27140 cells written into new column


In [228]:
# keine verb_id? Dann ist das Müll und kann weg.
# Wenn man das nicht will: Als Partizipien in lexeme_manuell aufnehmen!

mask = wiktionary_partizip.verb_id.isnull()
check_mask(wiktionary_partizip,mask,0,5)
wiktionary_partizip,df = pak.move_rows(wiktionary_partizip,mask)
df

check_mask:  1 rows
Move 1 rows from 27141


,page_id,page_part,adjektiv_id,adjektiv,verb_id,verb,tag,tagZ,tagZZ,member,score,msg
39390,108025,1,betroffen_1,betroffen,<NA>,betroffen,ADJA,ADJ,A,Adjektiv Ptz2,0.181285,10


In [229]:
# member pflegen. Enthält danach genau Ptz1 x-oder Ptz2.
wiktionary_partizip['member'] = pak.remove_str(wiktionary_partizip.member,['Adjektiv','Wortverbindung'])
mask = ~wiktionary_partizip.member.isin(['Ptz1','Ptz2'])
check_mask(wiktionary_partizip,mask,0)
mask = wiktionary_partizip.member.isnull()  |  (wiktionary_partizip.member=='')
check_mask(wiktionary_partizip,mask,0)

check_mask:  0 rows
check_mask:  0 rows


In [230]:
# Dups sollten eigentlich nicht drin sein
mask = wiktionary_partizip.duplicated()
check_mask(wiktionary_partizip,mask,0)

check_mask:  0 rows


In [231]:
# wiktionary_partizip.member analysieren
#r = pak.analyse_freqs( wiktionary_partizip, ['member', 'verb','adjektiv'])
#r

In [232]:
mask = wiktionary_partizip.adjektiv.isin(['betroffen','dienend'])
wiktionary_partizip[mask]

,page_id,page_part,adjektiv_id,adjektiv,verb_id,verb,tag,tagZ,tagZZ,member,score
61211,144143,0,dienend,dienend,dienen,dienen,ADJA,ADJ,A,Ptz1,1.756066


In [233]:
pak.sample(wiktionary_partizip)

,page_id,page_part,adjektiv_id,adjektiv,verb_id,verb,tag,tagZ,tagZZ,member,score
476,749,2,dezidiert_2,dezidiert,dezidieren,dezidieren,ADJA,ADJ,A,Ptz2,0.057074
4891,7368,3,verschieden_3,verschieden,verscheiden,verscheiden,ADJA,ADJ,A,Ptz2,0.003958
28720,75262,0,gehört,gehört,hören,hören,ADJA,ADJ,A,Ptz2,0.451413
32489,91988,4,überdacht_4,überdacht,überdenken,überdenken,ADJA,ADJ,A,Ptz2,0.028046
40673,110695,0,geleitet,geleitet,geleiten,geleiten,ADJA,ADJ,A,Ptz2,0.470008
61219,144189,0,bedienend,bedienend,bedienen,bedienen,ADJA,ADJ,A,Ptz1,2.194083
368665,555699,0,Acht_gebend,Acht gebend,Acht_geben,Acht geben,ADJA,ADJ,A,Ptz1,0.209178
373013,560657,0,abgenickt,abgenickt,abnicken,abnicken,ADJA,ADJ,A,Ptz2,0.401826
564476,826523,0,umnähend,umnähend,umnähen,umnähen,ADJA,ADJ,A,Ptz1,0.412157
591819,854478,0,Brust_schwimmend,Brust schwimmend,Brust_schwimmen,Brust schwimmen,ADJA,ADJ,A,Ptz1,0.402472


### Partizipien als Flexion des Verbs eintragen

In [234]:
# wiktionary_merkmal_partizip
wiktionary_merkmal_partizip = wiktionary_partizip.copy()
wiktionary_merkmal_partizip = pak.rename_col(wiktionary_merkmal_partizip, 'score',       'data_score')
wiktionary_merkmal_partizip = pak.rename_col(wiktionary_merkmal_partizip, 'member',      'merkmal')
wiktionary_merkmal_partizip = pak.rename_col(wiktionary_merkmal_partizip, 'verb_id',     'section_id2')
wiktionary_merkmal_partizip = pak.rename_col(wiktionary_merkmal_partizip, 'adjektiv_id', 'data_id')
wiktionary_merkmal_partizip = pak.rename_col(wiktionary_merkmal_partizip, 'adjektiv',    'data')
wiktionary_merkmal_partizip = pak.drop_cols(wiktionary_merkmal_partizip,['tag','tagZ','tagZZ','score','verb','page_id','page_part'])
wiktionary_merkmal_partizip = pak.move_cols(wiktionary_merkmal_partizip,['section_id2','merkmal'])
#assert set(wiktionary_merkmal_partizip.columns) == set(['section_id2','merkmal','data_id','data'])
pak.sample(wiktionary_merkmal_partizip)

,section_id2,merkmal,data_id,data,data_score
476,dezidieren,Ptz2,dezidiert_2,dezidiert,0.057074
4891,verscheiden,Ptz2,verschieden_3,verschieden,0.003958
39506,hinreißen,Ptz2,hingerissen,hingerissen,0.420421
61219,bedienen,Ptz1,bedienend,bedienend,2.194083
368665,Acht_geben,Ptz1,Acht_gebend,Acht gebend,0.209178
436893,auspowern,Ptz1,auspowernd,auspowernd,0.414223
591818,Brust_schwimmen,Ptz2,Brust_geschwommen,Brust geschwommen,0.411238
666493,durchsieben,Ptz2,durchgesiebt,durchgesiebt,0.430752
752953,übriglassen,Ptz1,übriglassend,übriglassend,0.565050
836939,übertölpeln,Ptz2,übertölpelt,übertölpelt,0.389430


In [235]:
# wiktionary_partizip suche_debug
search_str(wiktionary_merkmal_partizip, suche_debug)

,section_id2,merkmal,data_id,data,data_score


In [236]:
# Spalten vergleichen
bpy.minivenn(wiktionary_merkmal.columns,wiktionary_merkmal_partizip.columns, format='print2')

left_only:  {'num', 'meta', 'node_kontext', 'node_debug'}
right_only: {'data_score', 'data_id'}



In [237]:
# wiktionary_merkmal_partizip als merkmal anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, wiktionary_merkmal_partizip)

27140 rows added, now a total of 6288546


### Partizipien als Adjektiv einpflegen

In [238]:
#wiktionary_partizip.head(1)

In [239]:
#wiktionary_lemma.head(1)

In [240]:
# wiktionary_lemma_partizip
wiktionary_lemma_partizip = wiktionary_partizip.copy()
wiktionary_lemma_partizip = pak.rename_col(wiktionary_lemma_partizip, 'score',           'lemma_score')
wiktionary_lemma_partizip = pak.rename_col(wiktionary_lemma_partizip, 'adjektiv_id',     'section_id2')
wiktionary_lemma_partizip = pak.rename_col(wiktionary_lemma_partizip, 'adjektiv',        'lemma')
wiktionary_lemma_partizip = pak.rename_col(wiktionary_lemma_partizip, 'verb_id',         'data_id')
wiktionary_lemma_partizip = pak.drop_cols(wiktionary_lemma_partizip, ['verb'] )
wiktionary_lemma_partizip = ergänze_felder(wiktionary_lemma_partizip)
wiktionary_lemma_partizip = pflege_tag_felder(wiktionary_lemma_partizip, translate_tagZ)

pak.sample(wiktionary_lemma_partizip)

check_mask:  0 rows
Keine Doppeltags gefunden


,page_id,page_part,section_id2,lemma,lemma_lower,isupper,lemma_len,data_id,tag,tag_0,tag_1,tagZ,tagZZ,member,lemma_score
0,749,2,dezidiert_2,dezidiert,dezidiert,False,1,dezidieren,ADJA,ADJA,AzOs1lPYMO,ADJ,A,Ptz2,0.057074
14,7368,3,verschieden_3,verschieden,verschieden,False,1,verscheiden,ADJA,ADJA,NseuYVi9UD,ADJ,A,Ptz2,0.003958
180,78413,1,umgehend_1,umgehend,umgehend,False,1,umgehen,ADJA,ADJA,glSJIuNqLA,ADJ,A,Ptz1,0.064691
192,91988,4,überdacht_4,überdacht,überdacht,False,1,überdenken,ADJA,ADJA,VVfvV9tSFw,ADJ,A,Ptz2,0.028046
337,110695,0,geleitet,geleitet,geleitet,False,1,geleiten,ADJA,ADJA,Gb9OkZM4d2,ADJ,A,Ptz2,0.470008
1716,144189,0,bedienend,bedienend,bedienend,False,1,bedienen,ADJA,ADJA,JEIFNxXnlX,ADJ,A,Ptz1,2.194083
1818,149585,0,bedacht,bedacht,bedacht,False,1,bedachen,ADJA,ADJA,hNTSlxPH2Y,ADJ,A,Ptz2,0.492736
3447,417030,0,piepend,piepend,piepend,False,1,piepen,ADJA,ADJA,zzz6cVQZ8b,ADJ,A,Ptz1,0.542322
4977,463119,0,aalend,aalend,aalend,False,1,aalen,ADJA,ADJA,vlcqW98C0o,ADJ,A,Ptz1,0.523727
5953,503505,0,Acht_gegeben,Acht gegeben,acht gegeben,True,2,Acht_geben,ADJA,ADJA,z3lf7kI6kh,ADJ,A,Ptz2,0.214343


In [241]:
# Einige ungeeignete Partizipien löschen
mask1 = ~wiktionary_lemma_partizip.lemma.str.endswith(('d','t','en'))
mask2 = ~wiktionary_lemma_partizip.lemma.isin(['vertan','angetan','zugetan'])
mask = mask1  &  mask2
check_mask(wiktionary_lemma_partizip,mask,35)
wiktionary_lemma_partizip, df = pak.move_rows(wiktionary_lemma_partizip,mask)
df

check_mask:  35 rows
Move 35 rows from 27140


,page_id,page_part,section_id2,lemma,lemma_lower,isupper,lemma_len,data_id,tag,tag_0,tag_1,tagZ,tagZZ,member,lemma_score,msg
283,107579,0,getan,getan,getan,False,1,tun,ADJA,ADJA,k6dKXwYdGb,ADJ,A,Ptz2,1.053309,10
15238,672369,0,notgetan,notgetan,notgetan,False,1,nottun,ADJA,ADJA,GZUEVHHu3p,ADJ,A,Ptz2,0.391496,10
16340,714589,0,dargetan,dargetan,dargetan,False,1,dartun,ADJA,ADJA,NUlPHiKXws,ADJ,A,Ptz2,0.711443,10
17423,759022,0,abgetan,abgetan,abgetan,False,1,abtun,ADJA,ADJA,GDoig5PHXJ,ADJ,A,Ptz2,0.496868,10
17428,759226,0,betan,betan,betan,False,1,betun,ADJA,ADJA,IZmLGFELZC,ADJ,A,Ptz2,0.536124,10
17452,760264,0,wehgetan,wehgetan,wehgetan,False,1,wehtun,ADJA,ADJA,bbg4k0ohCk,ADJ,A,Ptz2,0.391496,10
17453,760265,0,gleichgetan,gleichgetan,gleichgetan,False,1,gleichtun,ADJA,ADJA,8GnjWC12kV,ADJ,A,Ptz2,0.519595,10
17454,760266,0,gutgetan,gutgetan,gutgetan,False,1,guttun,ADJA,ADJA,PEww7gY80l,ADJ,A,Ptz2,0.393562,10
17455,760267,0,kundgetan,kundgetan,kundgetan,False,1,kundtun,ADJA,ADJA,1XvnOYKUBN,ADJ,A,Ptz2,0.401826,10
17456,760271,0,leidgetan,leidgetan,leidgetan,False,1,leidtun,ADJA,ADJA,16GHidPqrm,ADJ,A,Ptz2,0.403893,10


In [242]:
# wiktionary_lemma suche_debug
grid(search_str(wiktionary_lemma_partizip, suche_debug))

No rows


In [243]:
# Spalten vergleichen
bpy.minivenn( wiktionary_lemma.columns, wiktionary_lemma_partizip.columns, format='print2')

left_only:  {'error', 'lang', 'inflected', 'lang_code'}
right_only: {'lemma_len', 'isupper', 'data_id'}



In [244]:
# wiktionary_lemma_partizip als lemma anfügen
wiktionary_lemma = pak.add_rows(wiktionary_lemma, wiktionary_lemma_partizip)

27105 rows added, now a total of 173520


### Manuelle Partizipien als Adjektiv einpflegen

In [245]:
# lexeme_manuell_copy als Basis
mask = lexeme_manuell_copy.tab == 'Partizipien'
ptz_manuell = lexeme_manuell_copy[mask]

In [246]:
# id des Verbs holen (dirty)
ptz_manuell = pak.update_col(ptz_manuell, 
                             lexeme_manuell,
                             on=['lemma','tag'],  
                             col='section_id2',
                             col_rename='data_id'
                            )

update_col: 413 cells written into new column


In [247]:
pak.sample(ptz_manuell)

,tag_lex,lex,lemma,tag,member,Genus,tab,score,def,data_id
0,Ptz2,abberufen,abberufen,VVFIN,manuell,,Partizipien,2.239462,,abberufen
137,Ptz1,erstaufführend,erstaufführen,VVFIN,manuell,,Partizipien,2.239462,,erstaufführen
266,Ptz1,Schlange stehend,Schlange stehen,VVFIN,manuell,,Partizipien,2.239462,,Schlange_stehen
304,Ptz1,überzeugend,überzeugen,VVFIN,manuell,,Partizipien,2.239462,,überzeugen
412,Ptz2,zwangsverschickt,zwangsverschicken,VVFIN,manuell,,Partizipien,2.239462,,zwangsverschicken


In [248]:
# Weitere Spalten
ptz_manuell['section_id2'] = ptz_manuell.lex.str.replace(' ','_') + '_MADJ'
spalten = ['section_id2','lex','tag_lex','data_id','score']
ptz_manuell = ptz_manuell[spalten]
ptz_manuell = pak.rename_col(ptz_manuell, 'score',  'lemma_score')
ptz_manuell = pak.rename_col(ptz_manuell, 'lex',    'lemma')
ptz_manuell = pak.rename_col(ptz_manuell, 'tag_lex','member')

ptz_manuell['member'] += ' manuell'
ptz_manuell['tag'] = 'ADJA'

ptz_manuell = ergänze_felder(ptz_manuell)
ptz_manuell = pflege_tag_felder(ptz_manuell, translate_tagZ)

#grid(ptz_manuell)

check_mask:  0 rows
Keine Doppeltags gefunden


In [249]:
# ptz_manuell suche_debug
grid(search_str(ptz_manuell, suche_debug))

No rows


In [250]:
# section_id2 ermitteln
mask = wiktionary_lemma.tagZ == 'ADJ'
ptz_manuell = pak.update_col( ptz_manuell,  
                              wiktionary_lemma[mask], 
                              on='lemma',     
                              col='section_id2',    
                              col_score='lemma_score' 
                                    )

update_col: col_score lemma_score applied, 1254 records less!
update_col: 232 cells written into existing column


In [251]:
pak.sample(ptz_manuell)

,section_id2,lemma,lemma_lower,isupper,lemma_len,member,data_id,lemma_score,tag,tag_0,tag_1,tagZ,tagZZ
0,abberufen_MADJ,abberufen,abberufen,False,1,Ptz2 manuell,abberufen,2.239462,ADJA,ADJA,oL7rvrhQ47,ADJ,A
125,ergraben,ergraben,ergraben,False,1,Ptz2 manuell,ergraben_1,2.239462,ADJA,ADJA,zxcXzyjcJ2,ADJ,A
137,erstaufführend_MADJ,erstaufführend,erstaufführend,False,1,Ptz1 manuell,erstaufführen,2.239462,ADJA,ADJA,3Z8ljm5khm,ADJ,A
266,Schlange_stehend_MADJ,Schlange stehend,schlange stehend,True,2,Ptz1 manuell,Schlange_stehen,2.239462,ADJA,ADJA,0ope0iKkJ8,ADJ,A
304,überzeugend,überzeugend,überzeugend,False,1,Ptz1 manuell,überzeugen,2.239462,ADJA,ADJA,DbWGtP70dq,ADJ,A
309,umfahren_MADJ,umfahren,umfahren,False,1,Ptz2 manuell,umfahren,2.239462,ADJA,ADJA,0AJJDxd9dh,ADJ,A
412,zwangsverschickt_MADJ,zwangsverschickt,zwangsverschickt,False,1,Ptz2 manuell,zwangsverschicken,2.239462,ADJA,ADJA,8v8XmJSxR2,ADJ,A


In [252]:
bpy.minivenn(wiktionary_lemma.columns,wiktionary_lemma_partizip.columns, format='print2')

left_only:  {'error', 'inflected', 'lang', 'lang_code'}
right_only: {}



In [253]:
# ptz_manuell als lemma anfügen
wiktionary_lemma = pak.add_rows(wiktionary_lemma, ptz_manuell, assert_subset=True)

413 rows added, now a total of 173933


In [254]:
mask = wiktionary_merkmal.data_id != ''
check_mask(wiktionary_merkmal,mask,27140)
pak.sample(wiktionary_merkmal[mask])

check_mask:  27140 rows


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,data_id,data_score
6261406,dezidieren,Ptz2,<NA>,<NA>,dezidiert,<NA>,<NA>,dezidiert_2,0.057074
6261420,verscheiden,Ptz2,<NA>,<NA>,verschieden,<NA>,<NA>,verschieden_3,0.003958
6261743,geleiten,Ptz2,<NA>,<NA>,geleitet,<NA>,<NA>,geleitet,0.470008
6263122,bedienen,Ptz1,<NA>,<NA>,bedienend,<NA>,<NA>,bedienend,2.194083
6267251,abwägen,Ptz2,<NA>,<NA>,abgewogen,<NA>,<NA>,abgewogen,0.661770
6270470,Acht_geben,Ptz1,<NA>,<NA>,Acht gebend,<NA>,<NA>,Acht_gebend,0.209178
6278886,überstreichen,Ptz2,<NA>,<NA>,übergestrichen,<NA>,<NA>,übergestrichen,0.649760
6286228,übriglassen,Ptz1,<NA>,<NA>,übriglassend,<NA>,<NA>,übriglassend,0.565050
6288545,übertölpeln,Ptz2,<NA>,<NA>,übertölpelt,<NA>,<NA>,übertölpelt,0.389430


## wiktionary_lemma weiterbearbeiten

### Grundpflege

In [255]:
# reset_index
wiktionary_lemma = pak.reset_index(wiktionary_lemma)

In [256]:
# drop_cols
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,['msg','inflected','error','lang','lang_code'])

In [257]:
# ergänze_felder
#from x32_Funktionen import *
wiktionary_lemma = ergänze_felder(wiktionary_lemma)

check_mask:  309 rows


In [258]:
mask = wiktionary_lemma.lemma_score.isnull()
wiktionary_lemma[mask]

,page_id,page_part,section_id2,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,lemma_score,data_id


In [259]:
# NaN
#wiktionary_lemma['lemma_score'] = wiktionary_lemma.lemma_score.fillna(0.45)
assert not pak.any_nan(wiktionary_lemma, without=['page_id','page_part','data_id',])
#pak.nnan(wiktionary_lemma)

## wiktionary_merkmal_text

In [260]:
# ist schon geladen
# wiktionary_merkmal_text = pak.load_pickle(wiktionary_merkmal_text_filename)
assert not pak.any_nan(wiktionary_merkmal_text)
# pak.nnan(wiktionary_merkmal_text)

### Grundpflege

In [261]:
# drop_cols
wiktionary_merkmal_text = pak.drop_cols( wiktionary_merkmal_text, ['section_id'])

In [262]:
# wiktionary_merkmal_text: list_to_string
wiktionary_merkmal_text['num'] = pak.list_to_string(wiktionary_merkmal_text.num)

In [263]:
# wiktionary_merkmal_text: drop_duplicates
vorher = wiktionary_merkmal_text.shape[0]
wiktionary_merkmal_text = wiktionary_merkmal_text.drop_duplicates()
nachher = wiktionary_merkmal_text.shape[0]
print(vorher-nachher,'Datensätze gelöscht')

41 Datensätze gelöscht


In [264]:
pak.sample(wiktionary_merkmal_text)

,section_id2,merkmal,num,meta,data
0,Hallo,def,1,,"lautes Rufen; fröhliches, lautes Durcheinander"
7,Subfamilia,bsp,1,Beispiele,Beispiele
1520,Haus,def,1,,zu einem bestimmten Zweck erbautes Gebäude
9615,Virus,bsp,2,,""" John von Neumann entwickelte 1949 die Theorie der ""selbstreplizierenden Automaten"" und bildete damit die theoretische Basis für alle Viren. Der Begriff Virus wurde 1981 in einem Gespräch zwischen Dr. Fred Cohen und Professor Leonhard M. Adlemann das erste Mal verwendet. In seiner Doktorarbeit ""Computer Viruses - Theory and Experiment"" aus dem Jahr 1983 definierte Cohen erstmals die grundlegenden Funktionen von Viren. Im Jahr 1986 verbreitete ein PC-Händler mit ""Brain"" den ersten MS-DOS- Virus. "" nach Axel Schoen: Brennpunkt Gefahr aus dem Web, Angriff der Schädlinge, in: Chip 04/2008, Seite 38"
70161,Kanji,bsp,2,,"父 , 水 und 家 sind Kanji."
100054,Cadmiumsulfit,bsp,"9,15",,und α-MnSO 3
292715,μm,def,1,Mikrometer,Einheitensymbol für Mikrometer
294557,&c.,def,1,,et cetera
353288,Bukkake,herkunft,,ぶっかけ,"Entliehen aus dem japanischen ぶっかけ , das Nomen des japanischen Verbs „Wasser spritzen“. «bukkake»,"
654676,Überreichweite,bsp,1,Überreichweite,"„Hinderlich sind Vogls Worten zufolge auch Signale von Sendern mit Überreichweite . Demnach haben die Rettungsdienste in den Landkreisen Erding, Ebersberg und Freising Signale eines Fernsehsenders aus Kiew in der Leitung statt die Signale aus der Leitstelle.“"


In [265]:
# Grobreinigung 

r = { ' !':'!',  ' ?':'?',  ' ,':',',  ' en ':'en ',  ' e ':'e ',  ' en,':'en,',  ' e,':'e,',  ' es,':'es,',  '  ':' ', }
wiktionary_merkmal_text['data'] = pak.replace_str( wiktionary_merkmal_text.data, r )

r = ['[1]', '[2]', '[3]', '[4]', '[5]', '[6]', '[7]', '[8]', '[9]', '[10]', '[11]', '[12]', '[1a]', '[*]']
wiktionary_merkmal_text['data'] = pak.remove_str( wiktionary_merkmal_text.data, r )

In [266]:
pak.nnan(wiktionary_lemma)

page_id        6749
page_part      6749
data_id      146415
dtype: int64

### Aufzählungen in Einzeldatensätze aufsplitten
* Die merkmale bsp_re und bsp_sp enthalten Aufzählungen. Diese werden in Einzeldatensätze aufgesplittet. 
* Das Feld rank_merkmal erhält die Reihenfolge.

In [267]:
# Splitten vorbereiten

mask = wiktionary_merkmal_text.merkmal.isin(['bsp_re','bsp_sp'])
r = { '–':'➤',  '—':'➤',  ';':'➤',  '/':'➤',  }
wiktionary_merkmal_text.loc[mask,'data'] = pak.replace_str( wiktionary_merkmal_text[mask].data, r )

grid( wiktionary_merkmal_text, mask )

6756 rows out of 654636


,section_id2,merkmal,num,meta,data
79,Januar,bsp_sp,1,,"der Januar baut eine Brücke, der Februar reißt sie in Stücke"
80,Januar,bsp_sp,1,,ein schöner Januar bringt ein gutes Jahr
81,Januar,bsp_sp,1,,gelinder Januar bringt ein spätes Frühjahr
82,Januar,bsp_sp,1,,"gibt’s im Januar viel Regen, bringt’s den Früchten keinen Segen"
83,Januar,bsp_sp,1,,"ist der Januar feucht und lau, wird das Frühjahr trocken und rau"
...,...,...,...,...,...
649876,Zwanni,bsp_re,1,,Haste➤Hasse mal ’nen Zwanni?
653063,Eilzugtempo,bsp_re,1,,im Eilzugtempo ➤ etwas wird schnell erledigt
653254,Daus_1,bsp_re,,,ei der Daus! potz Daus! was der Daus!
653375,beieinanderliegen,bsp_re,,,Genie und Wahnsinn liegen dicht beieinander


In [268]:
# rank_merkmal
wiktionary_merkmal_text = wiktionary_merkmal_text.reset_index() # spalte 'index' erzeugen
wiktionary_merkmal_text = pak.rank( wiktionary_merkmal_text, cols_group=['section_id2','merkmal'], col_score='index', find='min', col_target='rank_merkmal')
wiktionary_merkmal_text = pak.rename_col(wiktionary_merkmal_text,'index','i')
wiktionary_merkmal_text = pak.force_index(wiktionary_merkmal_text,'i')

rank: 0 rows less, now 654636 rows


In [269]:
# zu splittende Datensätze abtrennen
mask1 = wiktionary_merkmal_text.merkmal.isin(['bsp_re','bsp_sp'])
mask2 = wiktionary_merkmal_text.data.str.contains('➤')
mask  = mask1  &  mask2

wiktionary_merkmal_text, df = pak.move_rows( wiktionary_merkmal_text, mask, msg=None )

Move 1451 rows from 654636


In [270]:
# splitten
df['data'] = df.data.str.split('➤')
df = df.explode('data')

In [271]:
df.head(5)

,section_id2,merkmal,num,meta,data,rank_merkmal
i,,,,,,
202,ich,bsp_re,1,,"ich denk, ich hör nicht recht!",5
202,ich,bsp_re,1,,"ich denk, ich seh nicht recht!",5
207,ich,bsp_re,1,,"wasch mich, aber mach mich nicht nass!",10
207,ich,bsp_re,1,,"wasch mir den Pelz, aber mach mich nicht nass!",10
207,ich,bsp_re,1,,"wasch mir den Pelz, und mach mich nicht nass!",10


In [272]:
# rank_merkmal2
df = df.reset_index()
df = pak.rank(df, cols_group=['section_id2','merkmal','rank_merkmal'], col_score='i', find='min', col_target='rank_merkmal2')
df = pak.force_index(df,'i')

rank: 0 rows less, now 4323 rows


In [273]:
df.head(5)

,section_id2,merkmal,num,meta,data,rank_merkmal,rank_merkmal2
i,,,,,,,
202,ich,bsp_re,1,,"ich denk, ich hör nicht recht!",5,1
202,ich,bsp_re,1,,"ich denk, ich seh nicht recht!",5,2
207,ich,bsp_re,1,,"wasch mich, aber mach mich nicht nass!",10,1
207,ich,bsp_re,1,,"wasch mir den Pelz, aber mach mich nicht nass!",10,2
207,ich,bsp_re,1,,"wasch mir den Pelz, und mach mich nicht nass!",10,3


In [274]:
bpy.minivenn(wiktionary_merkmal_text.columns, df.columns, format='print')

left_only:  {}
both:       {'data', 'meta', 'section_id2', 'num', 'merkmal', 'rank_merkmal'}
right_only: {'rank_merkmal2'}



In [275]:
# wieder anfügen
wiktionary_merkmal_text = pak.add_rows(wiktionary_merkmal_text, df)
wiktionary_merkmal_text['rank_merkmal2'] = wiktionary_merkmal_text.rank_merkmal2.fillna(0) 

4323 rows added, now a total of 657508


In [276]:
# rank_merkmal endgültig schreiben
assert wiktionary_merkmal_text.rank_merkmal2.max() < 100
assert not pak.any_nan(wiktionary_merkmal_text.rank_merkmal)
assert not pak.any_nan(wiktionary_merkmal_text.rank_merkmal2)
wiktionary_merkmal_text['rank_merkmal'] = (wiktionary_merkmal_text.rank_merkmal * 100 + wiktionary_merkmal_text.rank_merkmal2).astype('int')
wiktionary_merkmal_text = pak.drop_cols(wiktionary_merkmal_text,['rank_merkmal2'])

In [277]:
#mask = wiktionary_merkmal_text.merkmal.isin( ['bsp_re'] )
#grid(wiktionary_merkmal_text,mask)

## Preprocessing

In [278]:
text = bj_nlp.TESTTEXT_0
text

"Hällö, „Welt“! <i>Hel 9lo</i> <b>Wo9 rld</b>! Das Wetter ist gut. Wie könnte`s not? A Öder nicht? Nic nemůže letět Rychlostí vyšší, než 300 tisíc kilometrů za sekundu! Hel 9lo Wo9rld! Th3     weather_is really g00d today, isn't it? Das geht    super, nöß?   Riß_2"

In [279]:
# Standardkonfiguration
bj_nlp.preprocess(text)

"Hällö, »Welt«! <i>Hel 9lo</i> <b>Wo9 rld</b>! Das Wetter ist gut. Wie könntes not? A Öder nicht? Nic nemuze letet Rychlosti vyssi, nez 300 tisic kilometru za sekundu! Hel 9lo Wo9rld! Th3 weather_is really g00d today, isn't it? Das geht super, nöß? Riß_2"

In [280]:
a = 'Hej U+1F600 världen'
print(a)

Hej U+1F600 världen


In [281]:
pak.analyse_datatypes(wiktionary_merkmal)

,col_name,datatype_instance,datatype,datatype_short,is_numeric,is_string,is_hashable,nan_allowed,mem_usage
col_no,,,,,,,,,
0,__index__,int64,np.int64,int64,True,False,True,False,80.0 B
1,section_id2,str,pd.string,string,False,True,True,True,791.0 B
2,merkmal,str,pd.string,string,False,True,True,True,708.0 B
3,node_debug,str,pd.string,string,False,True,True,True,557.0 B
4,node_kontext,str,pd.string,string,False,True,True,True,582.0 B
5,data,str,pd.string,string,False,True,True,True,918.0 B
6,num,str,pd.string,string,False,True,True,True,451.0 B
7,meta,str,pd.string,string,False,True,True,True,425.0 B
8,data_id,str,pd.string,string,False,True,True,True,534.0 B


In [282]:
# wiktionary_lemma
cols = pak.col_names(wiktionary_lemma, only='str')
cols = [ c for c in cols if c not in ['error']  and not c.startswith('tag')] 
#print(cols)

for feld in cols:
    print(feld)
    wiktionary_lemma[feld] = wiktionary_lemma[feld].fillna('').apply(   lambda x: bj_nlp.preprocess(x)       )

section_id2
lemma
lemma_lower
member
data_id


In [283]:
# wiktionary_nolemma
cols = pak.col_names(wiktionary_nolemma, only='str')
cols = [ c for c in cols if c not in ['error']  and not c.startswith('tag')] 
#print(cols)

for feld in cols:
    print(feld)
    wiktionary_nolemma[feld] = wiktionary_nolemma[feld].fillna('').apply(   lambda x: bj_nlp.preprocess(x)   )

nolemma_id
lex
lemma
member
lang
lang_code
lemma_lower


In [284]:
# wiktionary_merkmal

if gründlich:
    cols = pak.col_names(wiktionary_merkmal, only='str')
    cols = [ c for c in cols if c not in ['error']  and not c.startswith('tag')] 
    # print(cols)
    
    for feld in cols:
        print(feld)
        wiktionary_merkmal[feld] = wiktionary_merkmal[feld].fillna('').apply(   lambda x: bj_nlp.preprocess(x)   )

section_id2
merkmal
node_debug
node_kontext
data
num
meta
data_id


In [285]:
# Doppelspaces prüfen
if gründlich:
    mask = wiktionary_merkmal.data.str.contains('  ')
    check_mask( wiktionary_merkmal, mask, 0)
    #wiktionary_merkmal[mask]

check_mask:  0 rows


In [286]:
# wiktionary_merkmal_text

if gründlich:
    cols = pak.col_names(wiktionary_merkmal_text, only='str')
    cols = [ c for c in cols if c not in ['error']  and not c.startswith('tag')] 
    #print(cols)
    
    for feld in cols:
        print(feld)
        wiktionary_merkmal_text[feld] = wiktionary_merkmal_text[feld].fillna('').apply(   lambda x: bj_nlp.preprocess(x)   )

section_id2
merkmal
num
meta
data


## Sonderzeichen

In [287]:
# Sonderfälle
#mask = wiktionary_merkmal.data.str.contains('{{')
r = { '{{NNBSP}}'   :   ' '  ,   
             "''"   :   ''   ,
    }
wiktionary_merkmal['data'] = pak.replace_str(wiktionary_merkmal.data, r)

In [288]:
# Beispiel anschauen
mask = wiktionary_merkmal.section_id2 == suche_debug[0]
grid(wiktionary_merkmal,mask)

58 rows out of 6288546


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,data_id,data_score
243500,Euro,übersicht,Deutsch Substantiv Übersicht,Genus,m,,,,NaN
243501,Euro,übersicht,Deutsch Substantiv Übersicht,Nominativ Singular,Euro,,,,NaN
243502,Euro,übersicht,Deutsch Substantiv Übersicht,Nominativ Plural 1,Euros,,,,NaN
243503,Euro,übersicht,Deutsch Substantiv Übersicht,Nominativ Plural 2,Euro,,,,NaN
243504,Euro,übersicht,Deutsch Substantiv Übersicht,Genitiv Singular,Euro,,,,NaN
243505,Euro,übersicht,Deutsch Substantiv Übersicht,Genitiv Singular*,Euros,,,,NaN
243506,Euro,übersicht,Deutsch Substantiv Übersicht,Genitiv Plural 1,Euros,,,,NaN
243507,Euro,übersicht,Deutsch Substantiv Übersicht,Genitiv Plural 2,Euro,,,,NaN
243508,Euro,übersicht,Deutsch Substantiv Übersicht,Dativ Singular,Euro,,,,NaN
243509,Euro,übersicht,Deutsch Substantiv Übersicht,Dativ Plural 1,Euros,,,,NaN


In [289]:
# wiktionary_lemma enthält nicht_erlaubte_sonderzeichen (siehe Steuertabellen)
mask = wiktionary_lemma.lemma.str.contains(nicht_erlaubte_sonderzeichen_lemma)   &   ~wiktionary_lemma.section_id2.str.endswith('_M')
check_mask(wiktionary_lemma, mask, 0, 50)

msg = 'lemma_nicht_erlaubte_sonderzeichen'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(df_from=wiktionary_lemma, df_to=wiktionary_lemma_trash, mask=mask, msg=msg)
grid(wiktionary_lemma_trash,msg, color='green')

check_mask:  9 rows
Move 9 rows from 173933
9 rows


,page_id,page_part,section_id2,lemma,inflected,tag,tag_0,tag_1,tagZ,tagZZ,member,lang,lang_code,error,lemma_lower,lemma_score,isupper,lemma_len,data_id,msg
17309,42163,0,sa!,sa!,NaN,ITJ,ITJ,dUijRaDCrL,X_itj,X,Interjektion Partikel,<NA>,<NA>,<NA>,sa!,11.591488,False,1,,lemma_nicht_erlaubte_sonderzeichen
46067,136392,0,B$,B$,NaN,TODO,TODO,x3RU3UrMWk,X_todo,X,Abkürzung,<NA>,<NA>,<NA>,b$,2.830446,True,1,,lemma_nicht_erlaubte_sonderzeichen
51341,213354,0,Wasser_marsch!,Wasser marsch!,NaN,ITJ,ITJ,y5qJLOa69h,X_itj,X,Interjektion Partikel,<NA>,<NA>,<NA>,wasser marsch!,5.494268,True,2,,lemma_nicht_erlaubte_sonderzeichen
52552,231375,0,"April,_April!","April, April!",NaN,ITJ,ITJ,mywVGWeWaX,X_itj,X,Interjektion Partikel,<NA>,<NA>,<NA>,"april, april!",5.350616,True,2,,lemma_nicht_erlaubte_sonderzeichen
74855,402225,0,X!,X!,NaN,TODO,TODO,QKPWGSRfwv,X_todo,X,Abkürzung,<NA>,<NA>,<NA>,x!,2.392430,True,1,,lemma_nicht_erlaubte_sonderzeichen
74856,402226,0,Y!,Y!,NaN,TODO,TODO,kdbj5736Cs,X_todo,X,Abkürzung,<NA>,<NA>,<NA>,y!,2.392430,True,1,,lemma_nicht_erlaubte_sonderzeichen
89068,494955,0,beim_Teutates!,beim Teutates!,NaN,TODO,TODO,3Eq0C7xTgb,X_todo,X,Wortverbindung,<NA>,<NA>,<NA>,beim teutates!,0.239687,False,2,,lemma_nicht_erlaubte_sonderzeichen
108256,764892,0,T$,T$,NaN,TODO,TODO,JIBpjoCRZG,X_todo,X,Abkürzung,<NA>,<NA>,<NA>,t$,3.313280,True,1,,lemma_nicht_erlaubte_sonderzeichen
111820,823670,0,cui_bono?,cui bono?,NaN,TODO,TODO,zWBZalENW7,X_todo,X,Wortverbindung,<NA>,<NA>,<NA>,cui bono?,66.018257,False,2,,lemma_nicht_erlaubte_sonderzeichen


In [290]:
# wiktionary_nolemma enthält nicht_erlaubte_sonderzeichen
mask = wiktionary_nolemma.lemma.str.contains(nicht_erlaubte_sonderzeichen_lemma) # siehe Steuertabellen
check_mask(wiktionary_nolemma, mask, 0, 100)

msg='lemma_nicht_erlaubte_sonderzeichen'
wiktionary_nolemma, wiktionary_nolemma_trash = pak.move_rows(df_from=wiktionary_nolemma, df_to=wiktionary_nolemma_trash, mask=mask, msg=msg)
grid(wiktionary_nolemma_trash,msg, color='green')

check_mask:  0 rows
No rows moved
No rows


In [291]:
# wiktionary_nolemma enthält nicht_erlaubte_sonderzeichen
mask = wiktionary_nolemma.lex.str.contains(nicht_erlaubte_sonderzeichen_lemma) # siehe Steuertabellen
check_mask(wiktionary_nolemma, mask, 0, 100)

msg='lex_nicht_erlaubte_sonderzeichen'
wiktionary_nolemma, wiktionary_nolemma_trash = pak.move_rows(df_from=wiktionary_nolemma, df_to=wiktionary_nolemma_trash, mask=mask, msg=msg)
grid(wiktionary_nolemma_trash,msg, color='green')

check_mask:  0 rows
No rows moved
No rows


In [292]:
# data remove_str
r = ['Verzeichnis:Deutsch/','Verzeichnis:','<sub>','</sub>','<sup>','</sup>','Wikipedia:de:','Wikipedia:',]
wiktionary_merkmal['data'] = pak.remove_str(wiktionary_merkmal.data, r )

In [293]:
# data enthält Reste von Tags >> absplitten
mask = wiktionary_merkmal.data.str.contains('<',regex=False)
wiktionary_merkmal.loc[mask,'data'] = wiktionary_merkmal[mask].data.str.split('<').str[0]
#wiktionary_merkmal[mask]

In [294]:
# data enthält Reste von Tags >> absplitten
mask = wiktionary_merkmal.data.str.contains('{',regex=False)
wiktionary_merkmal.loc[mask,'data'] = wiktionary_merkmal[mask].data.str.split('{').str[0]
#wiktionary_merkmal[mask]

In [295]:
# wiktionary_merkmal data belöschen
mask = wiktionary_merkmal.data.str.startswith(('http:','https:'))
check_mask(wiktionary_merkmal, mask, 65)

msg='data_startswith'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg, color='green')

check_mask:  65 rows
Move 65 rows from 6288546
65 rows out of 13955


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,msg,data_id,data_score
3255394,Hand,flexion,,"mini|1|Gewiss [[strecken]] sich [[Gott]]es und des [[Mensch]]en ''Hand'' [[einander]] [[entgegen]], gewiss [[weisen]] ihre [[Fingerspitze]]n [[aufeinander]] zu - doch da ist eine [[Lücke]], ein [[Abstand]] zwischen ihnen. Er [[mögen|mag]] winzig [[erscheinen]], und dennoch ist er da.<ref>{{Per-Deutschlandradio | Online",http://www.deutschlandfunk.de/michelangelos-fresko-die-erschaffung-adams-die-luecke.2540.de.html?dram:article_id=401677,,,data_startswith,,NaN
3332947,Geothermie,flexion,,Online,https://p.dw.com/p/46SBc,,,data_startswith,,NaN
3378449,täuschen,flexion,,mini|1|<nowiki>[</nowiki>[[aufblasbar|Aufblasbarer]] [[Panzer]]:<nowiki>]</nowiki> »Den [[Feind]] zu ''täuschen'' ist [[grundlegend]]e [[Kriegstaktik]]. Im Laufe der [[Geschichte]] wurden [[hierzu]] immer wieder [[Attrappe]]n [[einsetzen|eingesetzt]].«<ref>{{Per-Deutsche Welle | Online,https://p.dw.com/p/4GLuJ,,,data_startswith,,NaN
3388477,Gewinn,flexion,,mini|1|Nach dem ''Gewinn'' des vierten [[WM-Titel]]s macht die [[deutsch]]e [[Fußball-Nationalmannschaft]] die [[die Nacht zum Tage machen|Nacht]] von Rio zum Tage. Und [[feiern|feiert]]. Und feiert. Und feiert.<ref>{{Per-Deutsche Welle | Online,http://dw.de/p/1Ccnj,,,data_startswith,,NaN
3432365,Versuchung,flexion,,"mini|1|''Versuchung'' bedeutet im christlichen Glauben, dass Gott die Menschen und ihren Gehorsam auf die Probe stellt. So haben zum Beispiel Adam und Eva der ''Versuchung'' nicht widerstehen können, die Äpfel vom Baum der Erkenntnis zu essen.<ref>{{Per-Deutsche Welle | Online",http://p.dw.com/p/1D5xg,,,data_startswith,,NaN
...,...,...,...,...,...,...,...,...,...,...
6216722,reparaturbedürftig,flexion,,mini|1|[Malzfabrik/Ering:] »Das [[alt]]e [[Gebäude]] hat seit seiner [[Stilllegung]] [[mehrmals]] die [[Besitzer]] [[wechseln|gewechselt]] und ist in einen [[stark]] ''reparaturbedürftigen'' [[Zustand]] mit [[Sanierungsstau]] [[kommen|gekommen]].«<ref>{{Per-Bayerischer Rundfunk | Online,"https://www.br.de/nachrichten/bayern/verein-will-eringer-malzfabrik-zu-kulturzentrum-umbauen,T5LAwKQ",,,data_startswith,,NaN
6217453,Drittland,flexion,,"mini|1|»Aus [[Sicht]] der [[EU]] ist die [[Türkei]] [[für]] [[Flüchtling]]e ein [[sicher]]es ''Drittland,'' dabei [[flüchten]] selbst [[Türken]] aus Erdogans [[Staat]].«<ref>{{Per-Deutsche Welle | Online",https://p.dw.com/p/42TWt,,,data_startswith,,NaN
6227637,Eckturm,flexion,,mini|1|»Im Gebäude mit dem L-förmigen Grundriss wurden diese zwei Bereiche in zwei Flügeln untergebracht. Verbunden miteinander sind sie durch das Stiegenhaus im markanten ''Eckturm.''«<ref>{{Internetquelle | url,https://oe1.orf.at/artikel/644795/Grazer-Stadtwerke-Haus-Steiermark,,,data_startswith,,NaN
6234618,Faltkarton,flexion,,mini|1|»[[Umzugsunternehmen]] [[einsetzen|setzen]] [[üblicherweise]] ''Faltkartons'' ein.«<ref>{{Per-Süddeutsche Online | Online,https://www.sueddeutsche.de/wirtschaft/immobilien-tipps-fuer-einen-stressarmen-umzug-dpa.urn-newsml-dpa-com-20090101-190211-99-938897,,,data_startswith,,NaN


In [296]:
# wiktionary_merkmal data belöschen
mask = wiktionary_merkmal.data.str.lower().str.endswith(('.jpg','.png','.gif'))
check_mask(wiktionary_merkmal, mask, 6)

msg='data_endswith'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg, color='green')

check_mask:  6 rows
Move 6 rows from 6288481
6 rows out of 13961


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,msg,data_id,data_score
3287486,fahl,flexion,,Bild,"Silvestro Lega - Mazzini morente, 1873.jpg",,,data_endswith,,NaN
3441480,Ramme,flexion,,200px|1|''Ramme'' mit ihrem Lamm|Bild 2,Kafar_na_Odrze.jpg,,,data_endswith,,NaN
3444567,Brache,flexion,,mini|1|landwirtschaftliche Brache|Bild 2,Frachtenbahnhof_Linz_Back.JPG,,,data_endswith,,NaN
3483103,knollig,flexion,,Bild,Illustration Sagittaria sagittifolia0.jpg,,,data_endswith,,NaN
3567749,apokalyptisch,flexion,,Bild,Durer Revelation Four Riders.jpg,,,data_endswith,,NaN
3604992,ungeschoren,flexion,,Bild,"Sheep, London Wetland Centre.jpg",,,data_endswith,,NaN


In [297]:
# wiktionary_merkmal node_kontext belöschen
mask = wiktionary_merkmal.node_kontext.str.startswith(('mini|','thumb|','|'))
check_mask(wiktionary_merkmal, mask, 40)

msg='node_kontext_startswith'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg, color='green')

check_mask:  40 rows
Move 40 rows from 6288475
40 rows out of 14001


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,msg,data_id,data_score
3281650,zeichnen,flexion,,mini|Bildbezug 3,5a,,,node_kontext_startswith,,NaN
3283121,Zimmermann,flexion,,"mini|1|»Druckt Leute, als Collectivum eine unbestimmte Menge aus, deren Theile nicht gezählt werden. Eigentlich kann es also kein bestimmendes Zahlwort vor sich haben, und weil man nicht sagen kann: sechs Messinge, sechs Silber u. sollte man auch nicht sagen: sechs Zimmerleute, sondern sechs ''Zimmermänner''.«<ref>{{Literatur|Autor",Johann Ernst Stutz,,,node_kontext_startswith,,NaN
3283319,Moskau,flexion,,"mini|1|''Moskau,'' [[Roter Platz]] mit Basiliuskathedrale und Spasski-Turm|Bild 2",Moscow_Oblast_in_Russia.svg,,,node_kontext_startswith,,NaN
3284751,Romantik,flexion,,mini|1|Caspar David Friedrich: Der Wanderer über dem Nebelmeer - In der [[modern]]en [[medial]]en [[Rezeption]] wurde das [[Bild]] zu einem [[vielfach]] [[verwendet]]en [[Symbol]] der ''Romantik'' und zu einer [[Trivial-Ikone]] des [[deutsch]]en [[Bewusstsein]]s.<ref>{{Wikipedia|Der Wanderer über dem Nebelmeer|oldid,161800745,,,node_kontext_startswith,,NaN
3288867,leben,flexion,,"mini|1|»Der [[groß|größte]] [[Vertreter]] der [[Mammut]]e und eines der größten [[bekannt]]en [[Rüsseltier]]e [[überhaupt]] war jedoch das [[Steppenmammut]] (Mammuthus trogontherii), das im [[Altpleistozän|Alt-]] und [[Mittelpleistozän]] in [[Eurasien]] ''lebte.''«<ref>{{Wikipedia|Mammute|oldid",229929288#Allgemein und Körpergröße,,,node_kontext_startswith,,NaN
3332946,Geothermie,flexion,,mini|1|»Bohrung für eine Erdsonde: Oberflächennahe ''Geothermie'' nutzt andere Technologien als tiefe ''Geothermie''«<ref name,"""dw_01"" >",,,node_kontext_startswith,,NaN
3336094,Humor,flexion,,thumb|2|Der Philosoph und Literaturkritiker Walter Benjamin hob bereits 1929 die besondere Rolle des ''Humors'' im Film […] hervor […].<ref>{{Wikipedia|Charlie Chaplin|oldid,166673968,,,node_kontext_startswith,,NaN
3353459,illegal,flexion,,"mini|1|»Kein Mensch ist ''illegal''« <nowiki>[</nowiki>auf [[Demonstration]]en gegen [[Flüchtlingsabschiebung]], [[Wohnungsnot]], [[etc.]], [[brauchen|gebrauchter]] [[Spruch]], der auf [[w:Elie Wiesel|Elie Wiesel]] [[zurückgehen|zurückgeht]]<nowiki>]</nowiki><ref>{{Wikipedia|kein mensch ist illegal|kein mensch ist illegal|spr",de,,,node_kontext_startswith,,NaN
3372098,Äquipotentialfläche,flexion,,mini|1|''Äquipotentialflächen'' (rot) für zwei punktförmig konzentrierte Ladungen gleichen Vorzeichens in 3D<ref name,WP>,,,node_kontext_startswith,,NaN
3409395,fangen,flexion,,"mini|5|<nowiki>[</nowiki>[[Grizzlybär]]en:<nowiki>]</nowiki> »Mit seinen [[Tatze]]n [[jagen|jagt]] er, ''fängt'' [[Lachs]]e, [[wenden|wendet]] [[Stein]]e zur [[Nahrungssuche]] und [[graben|gräbt]] [[Höhle]]n.«<ref>{{Wikipedia|Grizzlybär|oldid",170785322,,,node_kontext_startswith,,NaN


In [298]:
# wiktionary_merkmal node_kontext belöschen
mask = wiktionary_merkmal.node_kontext.str.contains('px|', regex=False)
check_mask(wiktionary_merkmal, mask, 3)

msg='node_kontext_contains'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg, color='green')

check_mask:  3 rows
Move 3 rows from 6288435
3 rows out of 14004


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,msg,data_id,data_score
3527548,Immendorf,flexion,,200px|1|Ersterwähnungsurkunde von ''Immendorf'' ([[w:Koblenz-Immendorf|Koblenz-Immendorf]]) <ref name,"""im""> abgedruckt auf der Titelseite zu einer Festschrift ""1100 Jahre Immendorf"" im Jahr 1980: Ortsring Immendorf (Herausgeber) / Hans Rudolf-Perschbach et al. (Redaktion), 1100 Jahre Immendorf 880-1980, ALFA-Verlag (Koblenz-Neuendorf), 1980, no ISBN.",,,node_kontext_contains,,NaN
3805689,Flederhund,flexion,,200px|1|ein Vertreter der ''Flederhunde'' (''Ptenochirus jagori'')|Genus,m,,,node_kontext_contains,,NaN
5224325,Stundenglas,flexion,,120px|Bildbezug 1,1,,,node_kontext_contains,,NaN


In [299]:
# wiktionary_merkmal enthält nicht_erlaubte_sonderzeichen
# Das weist auf Fehler beim Parsen hin >> löschen

felder = ['node_debug','node_kontext','meta','data'] 
for feld in felder: 
    mask = wiktionary_merkmal[feld].str.contains(nicht_erlaubte_sonderzeichen_gnädig) 
    print(feld, wiktionary_merkmal[mask].shape[0], 'Datensätze')
    wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg= feld + '_sonderzeichen', verbose=False)

node_debug 14 Datensätze
node_kontext 0 Datensätze
meta 1610 Datensätze
data 419 Datensätze


In [300]:
# Häufigkeiten
mask = wiktionary_merkmal_trash.msg.str.endswith('_sonderzeichen')
a = pak.analyse_freqs(wiktionary_merkmal_trash[mask], ['msg','merkmal'])
a

,msg,msg_count,msg_percent,merkmal,merkmal_count
0,meta_sonderzeichen,1610,78.8,"[def, familie, syn, herkunft]","[1592, 11, 5, 2]"
1,data_sonderzeichen,419,20.5,"[übersicht, flexion, def, down, nurWortart, herkunft, abk, kombi, gegen, syn, lateral, alt_2, familie, alt_1]","[149, 118, 64, 19, 17, 14, 11, 8, 5, 5, 4, 2, 2, 1]"
2,node_debug_sonderzeichen,14,0.7,übersicht,14


In [301]:
# Anzeigen
#mask = wiktionary_merkmal_trash.msg.str.endswith('_sonderzeichen')
#gridt(wiktionary_merkmal_trash[mask])

In [302]:
assert not pak.any_nan(wiktionary_merkmal.data) 

In [303]:
#a = pak.analyse_freqs(wiktionary_merkmal_trash,['msg','data','meta'])
#grid(a)

In [304]:
#mask = (wiktionary_merkmal_trash.msg == 'data_endswith')
#grid(wiktionary_merkmal_trash,mask)

In [305]:
# Beispiel anschauen
#mask = wiktionary_merkmal.section_id2 == suche_debug[0]
#grid(wiktionary_merkmal,mask)

## wiktionary_merkmal

### node_debug >> member
* member durch Daten aus der Spalte node_debug ergänzen

In [306]:
# übersicht reinigen mit remove_str und replace_str
mask = (wiktionary_merkmal.merkmal == 'übersicht')
r = ['-sch', 'Bairisch','Deutsch', ' f',' m',' n', 'Übersicht']
wiktionary_merkmal.loc[mask, 'node_debug'] = pak.remove_str(wiktionary_merkmal[mask].node_debug, r)

r = {'Pronomina-Tabelle': 'Pronomen', 'adjektivisch':'adjNN', 'Eigenname':'Name'}
wiktionary_merkmal.loc[mask, 'node_debug'] = pak.replace_str(wiktionary_merkmal[mask].node_debug, r)

In [307]:
# Was enthält node_debug?
mask = (wiktionary_merkmal.merkmal == 'übersicht')
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['node_debug', 'section_id2'], splits=[' ',None])
a

,node_debug,node_debug_count,node_debug_percent,section_id2,section_id2_count
0,Substantiv,899401,80.8,"[Eponym, Iftar, Gams, Avis, Dschungel, Omuramba, Mündel, Halfter, Cicerone, Quargel, Chor_1, Sandwich, Agens, Match, Staat, Index, Haube, Paprika, Triangel, Wimpel]","[30, 29, 29, 28, 27, 27, 27, 27, 27, 27, 26, 26, 26, 26, 23, 23, 23, 23, 23, 23]"
1,Verb,138224,12.4,"[anbellen, schraffieren, unterschieben, durchkämmen_1, wahrsagen, auferstehen, anerkennen, übersiedeln, anerbieten, downloaden, obliegen, obwalten, festfragen, wertschätzen, entbehren, dreschen, melken, liken, zusammenschrecken, flechten]","[25, 19, 19, 19, 19, 18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 16, 16, 16, 16, 15]"
2,Adjektiv,42483,3.8,"[langohrig, dunkelrot, erdnah, praxisnah, hohlwangig, wirtschaftsnah, lebensnah, rot, gesund, schmal, kirschrot, glatt, nass, blass, doof, bang, bange, karg, krumm, ungesund]","[9, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]"
3,adjNN,18978,1.7,"[weißer_Rauch, Großer_Zapfenstreich, menschlicher_Schutzschild, offener_Brief, falscher_Freund, amerikanischer_Traum, alter_Fahrensmann, kalter_Entzug, totaler_Krieg, absoluter_Krieg, frommer_Wunsch, gebranntes_Kind, ewiges_Talent, elektrischer_Stuhl, falscher_Hase, Schwarzes_Loch, Gewöhnlicher_Schweinswal, Fette_Henne, Grünes_Heupferd, Armenische_Brombeere]","[32, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 28, 28, 28, 28, 28, 28]"
4,Vorname,11795,1.1,"[Asbirg, Abel, Adelheid, Eschel, Ketel, Angert, Marrin, Tatt, Gerlich, Herlich, Ing, Marret, Moild, Gerd, Gertrud, Irmgard, Ingeborg, Christel_1, Brit, Eitel]","[17, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]"
5,Pronomen,1270,0.1,"[anderer, unser_1, euer, son, irgendein, manch, aller, der_1, ein_2, dies, welcher, all, jeglicher, jedweder, irgendwelche, kein, solch, ebensolcher, dieser, ihr_1]","[38, 31, 29, 21, 19, 19, 18, 18, 18, 18, 18, 18, 18, 18, 18, 17, 17, 17, 16, 16]"
6,Name,579,0.1,"[Michel, Pidgin, Patois, Ottilie_Normalverbraucherin, Mars, Otto_Normalverbraucher, Sol, Andromeda, Selection, Classic, Judas, Jesus, Mühle_1, Miraculix, Majestix, Idefix, Undine, Steno, Britannia, Katharevousa]","[14, 10, 10, 10, 9, 9, 9, 9, 9, 9, 5, 4, 2, 2, 2, 2, 2, 2, 2, 2]"
7,Nachname,453,0.0,"[Aab, Wolf_4, Engel_3, Hain_1, Adler_3, Huber_1, Ackermann_1, Aichinger, Achleitner, Acher, Achner, Beck_1, Berger, Buchegger, Brunner, Baumert, Auer, Holzapfel_1, Bichler, Aal_1]","[10, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2]"
8,Toponym,119,0.0,"[Annaberg, Rioja, Saporischschja, Krywyj_Rih, Karien, Jenseits_1, Irak, Iran, Montserrat_1, Niger_1, Tschad, Aschersleben, Stall_1, Sudan, Bruay-la-Buissiere, Kongo, Senegal_1, Oman, Jemen, Libanon]","[10, 8, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
9,Adverb,31,0.0,"[wohl, oft, gerne, forte, staccato, wahrlich, wohlweislich, hoch_1]","[5, 5, 5, 4, 4, 3, 3, 2]"


In [308]:
# member_plus festlegen
mask = (wiktionary_merkmal.merkmal == 'übersicht')   &   (wiktionary_merkmal.node_debug.str.len() > 0)
spalten = ['section_id2','node_debug']
df = wiktionary_merkmal[mask][spalten].drop_duplicates()
df.columns = ['section_id2','member_plus']
#grid(df,sample)

In [309]:
# an member anhängen
wiktionary_lemma = pak.update_col(wiktionary_lemma, df, on='section_id2',col='member_plus')
mask = (wiktionary_lemma.member_plus.str.len() > 0)
wiktionary_lemma.loc[mask,'member'] = wiktionary_lemma[mask].member.str.strip() + ' ' + wiktionary_lemma[mask].member_plus.str.strip()
wiktionary_lemma = pak.drop_cols(wiktionary_lemma, 'member_plus')

update_col: 127595 cells written into new column
update_col: WARNING: df_from identifier not unique. I call this again with func="max"
update_col: func max applied, 1 records less!
update_col: 127594 cells written into new column


In [310]:
# node_debug leeren
mask = (wiktionary_merkmal.merkmal == 'übersicht')
wiktionary_merkmal.loc[mask,'node_debug'] = ''

In [311]:
# Letzte Löschungen
r = ['Wortart fehlt']
wiktionary_merkmal.node_debug = pak.remove_str( wiktionary_merkmal.node_debug, r )

In [312]:
# node_debug löschen (müsste jetzt überall leer sein)
mask = (wiktionary_merkmal.node_debug.str.len() > 0)
if version == '_all':
    check_mask(wiktionary_merkmal, mask, 0, 0)
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal, 'node_debug')   

check_mask:  0 rows


### übersicht und flexion
* aus wiktionary_merkmal herausnehmen, pflegen und wieder anfügen

In [313]:
# Beispiel anschauen
#mask = wiktionary_merkmal.section_id2 == suche_debug[0]
#grid(wiktionary_merkmal,mask)

In [314]:
# nach flexion auslagern
mask = wiktionary_merkmal.merkmal.isin(['übersicht','flexion']) 
wiktionary_merkmal, flexion = pak.move_rows( wiktionary_merkmal, mask, msg=None ) 
flexion_trash = flexion.head(0)

Move 4119902 rows from 6286389


In [315]:
# In wiktionary_merkmal ist node_kontext jetzt überflüssig
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal, 'node_kontext') 

In [316]:
flexion = pak.rename_col(flexion, 'merkmal',      'quelle')
flexion = pak.rename_col(flexion, 'node_kontext', 'merkmal_lang')
flexion = pak.drop_cols(flexion, ['num','meta','msg','section_id','quelle'])
flexion['merkmal'] = flexion.merkmal_lang.copy()
flexion = pak.move_cols( flexion, ['section_id2', 'merkmal_lang', 'merkmal', 'data'])

In [317]:
# Dups entfernen
flexion = pak.reset_index(flexion.drop_duplicates())

In [318]:
# meta schreiben
# dazu lexeme_manuell nochmal aufklappen, um an die lexeme heranzukommen
df = pak.explode_dict(lexeme_manuell, col_dict='flexion', col_key='key', col_value='data', from_defaultdict=True)
mask = df.section_id2 == 'und'
df[mask]

,section_id2,title,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,score,key,data
1496,und,und,und,und,KON,KON,YRB0tCmU8R,X_konj,X,manuell,2.239462,abk,&
1496,und,und,und,und,KON,KON,YRB0tCmU8R,X_konj,X,manuell,2.239462,abk,+
1496,und,und,und,und,KON,KON,YRB0tCmU8R,X_konj,X,manuell,2.239462,abk,u.


In [319]:
# meta schreiben
flexion['meta'] = ''
mask = pak.isin( flexion, df, on=['section_id2','data'])
check_mask(flexion,mask)
flexion.loc[mask,'meta'] = 'manuell'

check_mask:  2808 rows


In [320]:
# abspalten
flexion, df = pak.move_rows(flexion, mask, msg=None)

# auswählen
df = df.drop_duplicates(subset=['section_id2','merkmal'])

# wieder anfügen
flexion = pak.add_rows(flexion, df, assert_subset=True)

Move 2808 rows from 3094268
2526 rows added, now a total of 3093986


In [321]:
# merkmal reinigen

# 'Ptz1' und 'Ptz2' maskieren
flexion['merkmal'] = pak.replace_str(flexion.merkmal, {'Ptz1':'PtzA', 'Ptz2':'PtzB'})

# merkmal reinigen
r = ['*','1','2','3','4','stark','schwach','gemischt']
flexion['merkmal'] = pak.remove_str(flexion.merkmal, r)

# 'Ptz1' und 'Ptz2' demaskieren
flexion['merkmal'] = pak.replace_str(flexion.merkmal, {'PtzA':'Ptz1', 'PtzB':'Ptz2'})

In [322]:
# Beispiel anschauen
#mask = flexion.section_id2 == suche_debug[0]
#grid(flexion,mask)

In [323]:
# translate_first

print(translate_first1)
print(translate_first2)

for quelle, ziel in translate_first1:
    flexion.merkmal = flexion.merkmal.str.replace( quelle, ziel )

for quelle, ziel in translate_first2:
    flexion.merkmal = flexion.merkmal.str.replace( quelle, ziel )

[('  ', ' '), ('Partizip II', 'Ptz2'), ('Partizip_II', 'Ptz2'), ('PartizipII', 'Ptz2'), ('Konjunktiv II', 'Konj2'), ('Konjunktiv_II', 'Konj2'), ('KonjunktivII', 'Konj2'), ('  ', ' ')]
[('  ', ' '), ('Partizip I', 'Ptz1'), ('Partizip_I', 'Ptz1'), ('PartizipI', 'Ptz1'), ('Konjunktiv I', 'Konj1'), ('Konjunktiv_I', 'Konj1'), ('KonjunktivI', 'Konj1'), ('  ', ' ')]


In [324]:
translate_merkmal

{'kein Plural': 'noPl',
 'kein Singular': 'noSg',
 'Singular': 'Sg',
 'Pluralformen': 'Pl',
 'Plural': 'Pl',
 'Nominativ': 'Nom',
 'Genitiv': 'Gen',
 'Dativ': 'Dat',
 'Akkusativ': 'Akk',
 'Imperativ': 'Imp',
 'Gegenwart': 'Präsens',
 'Weitere_Konjugationen': 'Weitere',
 'keine weiteren Formen': 'noFlex',
 '_ich': ' ich',
 '_du': ' du',
 '_er,sie,es': ' ersiees',
 '_er, sie, es': ' ersiees',
 '_wir': ' wir',
 '_ihr': ' ihr',
 'Konjunktiv': 'Konj'}

In [325]:
# translate_merkmal: merkmal reinigen
flexion['merkmal'] = pak.replace_str(flexion.merkmal, translate_merkmal)

In [326]:
#a = analyse_freqs(flexion,'merkmal','data', split_A=' ', style='top') # 
#grid(a)

In [327]:
# strip
flexion.merkmal = flexion.merkmal.str.replace('  ',' ').str.strip()

In [328]:
# Einige Merkmale löschen
r = ['','Weitere Konjugationen','beschreibung','bezug','breite','Jahr','Titel','am','Zugriff','Monat','Tag','Autor','Bild','Flexion','unpersönlich',
     'Kommentar','zugriff','hrsg','Artikel','größe','titel','Ort','spr','Verlag','Seiten','autor','datum','Weitere','kein-s', 'Kein-ens','Datum',
     'EW','ISBN','Originalsprache','TitelErg','dim','kommentar','name','region','sprache','text','Keine weiteren Formen',   
    ]
mask = flexion.merkmal.isin(r)
flexion, flexion_trash = pak.move_rows(df_from=flexion, df_to=flexion_trash, mask=mask, msg='ungenutzte_Merkmale')
grid(df)

Move 1500 rows from 3093986
2526 rows


,section_id2,merkmal_lang,merkmal,data,data_id,data_score,meta
121,Dezember,Genus,Genus,m,,NaN,manuell
130,Januar,Genus,Genus,m,,NaN,manuell
140,Februar,Genus,Genus,m,,NaN,manuell
150,März,Genus,Genus,m,,NaN,manuell
162,April,Genus,Genus,m,,NaN,manuell
...,...,...,...,...,...,...,...
3094263,überraschen,Ptz1,Ptz1,überraschend,,NaN,manuell
3094264,überschäumen,Ptz1,Ptz1,überschäumend,,NaN,manuell
3094265,überwiegen,Ptz1,Ptz1,überwiegend,,NaN,manuell
3094266,überwältigen,Ptz1,Ptz1,überwältigend,,NaN,manuell


In [329]:
# merkmale sortieren
flexion = sort_merkmale(flexion)

In [330]:
# Müll löschen (obsolet: Das wurde weiter oben schon gelöscht)
mask = flexion.merkmal.str.contains('|', regex=False)
check_mask(flexion,mask,0)
#flexion, flexion_trash = pak.move_rows(df_from=flexion, df_to=flexion_trash, mask=mask, msg='Müll1')

check_mask:  0 rows


In [331]:
# Müll löschen (obsolet: Das wurde weiter oben schon gelöscht)
mask = flexion.data.str.contains('<', regex=False)
check_mask(flexion,mask,0)
#flexion, flexion_trash = pak.move_rows(df_from=flexion, df_to=flexion_trash, mask=mask, msg='Müll2') 

check_mask:  0 rows


In [332]:
# Striche vereinheitlichen
r = {'—':'-', '–':'-'}
flexion.data = pak.replace_str(flexion.data, r)

In [333]:
#a = pak.analyse_freqs(flexion,['merkmal','data'], splits=[' ', None]) # 
#gridt(a)

In [334]:
#a = pak.analyse_freqs(flexion,['merkmal','data']) 
#gridt(a)

In [335]:
#a = pak.analyse_freqs(flexion,['sort','merkmal']) # 
#gridt(a)

In [336]:
# flexion wieder an wiktionary_merkmal anfügen

df = pak.drop_cols(flexion, ['merkmal_lang','sort'])
df = df.drop_duplicates()
#grid(df)
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, df, assert_subset=True)

3078221 rows added, now a total of 5244708


In [337]:
# Beispiel anzeigen 
mask = (wiktionary_merkmal.section_id2 == suche_debug[0])
grid(wiktionary_merkmal,mask)

43 rows out of 5244708


,section_id2,merkmal,data,num,meta,data_id,data_score
195244,Euro,def,Währung,1,,,NaN
195245,Euro,def,Europäischen Währungsunion,1,,,NaN
195246,Euro,syn,Teuro,1,scherzhaft,,NaN
195247,Euro,up,Währung,1,,,NaN
195248,Euro,down,Pendlereuro,1,,,NaN
195249,Euro,familie,Euro-Abschaffung,,,,NaN
195250,Euro,familie,Eurobanknote,,,,NaN
195251,Euro,familie,Eurobond,,,,NaN
195252,Euro,familie,Eurocent,,,,NaN
195253,Euro,familie,Euro-Einführung,,,,NaN


In [338]:
# Speicher sparen
if speicher_freigeben:
    del flexion

### wiktionary_merkmal pflegen
* Diverse Pflegeschritte

In [339]:
# drop_cols
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,['rhymes','inflected','lang','lang_code','lemma_lower'])

In [340]:
mask = wiktionary_merkmal.merkmal == 'rhymes'
pak.sample(wiktionary_merkmal[mask])

,section_id2,merkmal,data,num,meta,data_id,data_score
2167817,speicherten,rhymes,aɪcɐtn,NaN,,,NaN
2167941,exploitierten,rhymes,iːɐtn,NaN,,,NaN
2176627,ABC-Trinkers,rhymes,eːtʁɪŋkɐs,NaN,,,NaN
2478092,gepresstem,rhymes,ɛstəm,NaN,,,NaN


In [341]:
# change_datatype
wiktionary_merkmal = pak.change_datatype(wiktionary_merkmal)

change_datatype 
section_id2          --> pd.string 
merkmal              --> pd.string 
data                 --> pd.string 
num                  --> pd.string 
meta                 --> pd.string 
data_id              --> pd.string 
change_datatype before: 1.9 GB after: 1.9 GB



In [342]:
# Keine nan in data
assert not pak.any_nan(wiktionary_merkmal.data) 

In [343]:
# herkunft: meta leeren
mask = wiktionary_merkmal.merkmal.isin(['herkunft']) 
wiktionary_merkmal.loc[mask,'meta'] = ''
#grid(wiktionary_merkmal, mask)

In [344]:
#grid(wiktionary_merkmal_trash)

In [345]:
# noPl
mask = (wiktionary_merkmal.data.str.len() <= 1)  &   wiktionary_merkmal.merkmal.str.endswith('Pl')   &   (wiktionary_merkmal.merkmal != 'noPl')
wiktionary_merkmal.loc[mask,'merkmal'] = 'noPl¼'
wiktionary_merkmal.loc[mask,'data']    = '1'

In [346]:
# noSg
mask = (wiktionary_merkmal.data.str.len() <= 1)  &   wiktionary_merkmal.merkmal.str.endswith('Sg')   &   (wiktionary_merkmal.merkmal != 'noSg')
wiktionary_merkmal.loc[mask,'merkmal'] = 'noSg¼'
wiktionary_merkmal.loc[mask,'data']    = '1'

In [347]:
# mindestens 2 Zeichen deuten auf noSg hin

# finden
mask = (wiktionary_merkmal.merkmal == 'noSg¼')
df = wiktionary_merkmal[mask].groupby('section_id2')['merkmal'].agg('count').reset_index()
mask = (df.merkmal > 1)
markiere = df[mask]['section_id2']

# nach wiktionary_lemma übertragen
mask = wiktionary_lemma.section_id2.isin(markiere)
wiktionary_lemma.loc[mask,'member'] += ' noSg'

In [348]:
# mindestens 2 Zeichen deuten auf noPl hin

# finden
mask = (wiktionary_merkmal.merkmal == 'noPl¼')
df = wiktionary_merkmal[mask].groupby('section_id2')['merkmal'].agg('count').reset_index()
mask = (df.merkmal > 1)
markiere = df[mask]['section_id2']

# nach wiktionary_lemma übertragen
mask = wiktionary_lemma.section_id2.isin(markiere)
wiktionary_lemma.loc[mask,'member'] += ' noPl'

In [349]:
# entfernen
mask = wiktionary_merkmal.merkmal.isin(['noSg¼','noPl¼'])
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='noSg¼ noPl¼')

Move 55846 rows from 5244708


In [350]:
# entfernen
mask = wiktionary_merkmal.merkmal.isin(['beschreibung 5','bezug 5','breite 5'])
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='Artefakte')

Move 3 rows from 5188862


In [351]:
# entfernen
mask = wiktionary_merkmal.merkmal.isin(['Weitere Formen'])
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='Weitere Formen')

Move 6 rows from 5188859


In [352]:
#a = pak.analyse_freqs(wiktionary_merkmal,['merkmal','data'], splits=[' ', None]) # 
#grid(a)

In [353]:
# data enthält nicht_erlaubte_sonderzeichen
mask = wiktionary_merkmal.data.str.contains(nicht_erlaubte_sonderzeichen_gnädig) # siehe Steuertabellen
check_mask(wiktionary_merkmal, mask, 0, 10000)

msg='Sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [354]:
# E-Merkmale 
mask = (plan_merkmal.collect == 'E')
merkmale_E = list(plan_merkmal[mask].merkmal)
merkmale_E 

['noSteig',
 'noSing',
 'istName',
 'Wikispecies',
 'Wikivoyage',
 'Taxonomie',
 'Präfix',
 'Suffix',
 'VMaßeinheiten',
 'noSg',
 'noPl',
 'noFlex']

In [355]:
# data ist zu kurz 
mask1 =  wiktionary_merkmal.data.str.len() == 1
mask2 = ~wiktionary_merkmal.merkmal.isin( merkmale_E + ['Genus','ipa','rhymes','def','lex','abk'] )
mask3 = ~wiktionary_merkmal.data.isin(['-'])
mask = mask1  &  mask2  &  mask3

check_mask(wiktionary_merkmal, mask, 0, 30)
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='zu kurz')

#grid(wiktionary_merkmal,mask)
#pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal', 'data'])

check_mask:  20 rows
Move 20 rows from 5188853


In [356]:
mask = wiktionary_merkmal.data == '€'
wiktionary_merkmal[mask]

,section_id2,merkmal,data,num,meta,data_id,data_score


In [357]:
# wiktionary_merkmal belöschen: Wortbildungen
mask = wiktionary_merkmal.data.str.contains('Wortbildungen/')
check_mask(wiktionary_merkmal,mask,201)

msg = 'Wortbildungen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  201 rows
Move 201 rows from 5188833
201 rows out of 72123


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,msg,data_id,data_score
44596,mit,familie,NaN,NaN,Wortbildungen/mit,,ausschließlich,Wortbildungen,,NaN
44598,mit_1,familie,NaN,NaN,Wortbildungen/mit,,,Wortbildungen,,NaN
73951,Phobie,down,NaN,NaN,Wortbildungen/-phobie,1,siehe,Wortbildungen,,NaN
73953,Phobie,familie,NaN,NaN,Wortbildungen/-phobie,,siehe,Wortbildungen,,NaN
87429,eins,familie,NaN,NaN,Wortbildungen/ein,,,Wortbildungen,,NaN
...,...,...,...,...,...,...,...,...,...,...
1658667,-algie,familie,NaN,NaN,Wortbildungen/-algie,,,Wortbildungen,,NaN
1777935,Vorkriegs-,familie,NaN,NaN,Wortbildungen/Vorkriegs,,ausschließlich,Wortbildungen,,NaN
1985887,zurück-,familie,NaN,NaN,Wortbildungen/zurück,,,Wortbildungen,,NaN
2029337,raus-,familie,NaN,NaN,Wortbildungen/heraus,,,Wortbildungen,,NaN


In [358]:
# meta remove_str replace_str
wiktionary_merkmal['meta'] = pak.remove_str(  wiktionary_merkmal.meta, '()'         )
wiktionary_merkmal['meta'] = pak.replace_str( wiktionary_merkmal.meta, {'/':' '}    )

In [359]:
# data Klammer am Ende schließen
mask1 =  wiktionary_merkmal.data.str.contains('(', regex=False)
mask2 = ~wiktionary_merkmal.data.str.contains(')', regex=False)
mask = mask1  &  mask2
wiktionary_merkmal.loc[mask,'data'] += ')'
# grid(wiktionary_merkmal, mask)

In [360]:
# Beispiel anzeigen 
mask = (wiktionary_merkmal.section_id2 == suche_debug[0])
grid(wiktionary_merkmal,mask)

43 rows out of 5188632


,section_id2,merkmal,data,num,meta,data_id,data_score
195244,Euro,def,Währung,1,,,NaN
195245,Euro,def,Europäischen Währungsunion,1,,,NaN
195246,Euro,syn,Teuro,1,scherzhaft,,NaN
195247,Euro,up,Währung,1,,,NaN
195248,Euro,down,Pendlereuro,1,,,NaN
195249,Euro,familie,Euro-Abschaffung,,,,NaN
195250,Euro,familie,Eurobanknote,,,,NaN
195251,Euro,familie,Eurobond,,,,NaN
195252,Euro,familie,Eurocent,,,,NaN
195253,Euro,familie,Euro-Einführung,,,,NaN


### Kontrolle wiktionary_merkmal
* Diverse Kontrollen

In [361]:
# plan_merkmal Steuertabelle neu erstellen
try:
    del plan_merkmal
except:
    pass

from Steuertabellen.s01_Steuertabellen import *
plan_merkmal = plan_merkmal_erstellen()
#pak.sample(plan_merkmal)
grid(plan_merkmal)

103 rows


,merkmal,template,collect,plan,sort,is_lex
0,noSteig,kSt.,E,1,,False
1,noSing,kSg.,E,1,,False
2,nurWortart,Wortart fehlt,P,1,,False
3,istName,Navigationsleiste Anthroponyme,E,1,,False
4,Wikispecies,Wikispecies,E,1,,False
...,...,...,...,...,...,...
98,Superlativ,,,3,x3 Superlativ,True
99,Stamm,,,3,z1 Stamm,False
100,ipa,,,3,z1 ipa,False
101,noFlex,,E,3,z1 noFlex,True


In [362]:
# Merkmale lt. Steuertabelle und lt. Realität
mask = (plan_merkmal.plan != 2)
merkmale_plan = set(plan_merkmal[mask].merkmal)
merkmale_data = set(wiktionary_merkmal.merkmal.unique())

In [363]:
# left_only:  Merkmale, die in er Realität nicht auftreten
# right_only: Merkmale, die nicht geplant sind
bpy.minivenn(merkmale_plan,merkmale_data,format='print2')

left_only:  {'Konj1', 'PTKVZ', 'Konj2', 'übersicht', 'Konj', 'lemma', 'Gerundivum', 'down_part', 'Imp'}
right_only: {'Sg', 'genitiv Sg', 'ADV', 'VVIMP'}



In [364]:
# Fast alle in der Steuertabelle plan_merkmal erfassten Merkmale kommen in wiktionary_merkmal tatsächlich vor.
ausnahmen = {'PTKVZ', 'Konj2', 'Konj', 'lemma', 'down_part', 'Konj1', 'Imp', 'Gerundivum', 'übersicht'}
fehlende_merkmale = bpy.minivenn(merkmale_plan - ausnahmen,merkmale_data,format='dict')['left_only']
assert len(fehlende_merkmale) == 0

In [365]:
# Tauchen die Ausnahmen in wiktionary_merkmal_trash auf?
# Sollten sie nicht.
mask1 =  wiktionary_merkmal_trash.merkmal.isin(  ausnahmen - {'nurWortart'}  )
mask2 = ~wiktionary_merkmal_trash.merkmal.isin(['übersicht'])
mask = mask1  & mask2
check_mask(wiktionary_merkmal_trash,mask,0)
#wiktionary_merkmal_trash[mask]

check_mask:  0 rows


In [366]:
# alle Merkmale in wiktionary_merkmal sind in der Steuertabelle plan_merkmal erfasst
alle_tags = set(translate_tagZ.tag) # tags werden als merkmal akzeptiert
unregistrierte_merkmale = bpy.minivenn(merkmale_plan, merkmale_data - alle_tags, format='dict')['right_only']
print(unregistrierte_merkmale)

{'Sg', 'genitiv Sg'}


In [367]:
# Probleme anschauen
mask = wiktionary_merkmal.merkmal.isin(unregistrierte_merkmale)
check_mask(wiktionary_merkmal,mask,0,3)
grid(wiktionary_merkmal, mask, color='green')

check_mask:  2 rows
2 rows out of 5188632


,section_id2,merkmal,data,num,meta,data_id,data_score
2579757,North_Carolina_1,Sg,North Carolina,<NA>,,,NaN
2579758,Zinkweiß,genitiv Sg,Zinkweißes,<NA>,,,NaN


In [368]:
# Keine nan in data
assert pak.nnan(wiktionary_merkmal.data) == 0

In [369]:
# Eindeutige Datentypen
for col in wiktionary_merkmal.columns:
    print(col)
    if col == 'num':
        continue
    assert pak.ntypes(wiktionary_merkmal[col]) == 1

section_id2
merkmal
data
num
meta
data_id
data_score


In [370]:
wiktionary_merkmal.shape

(5188632, 7)

## wiktionary_lemma

In [371]:
# suche_debug
mask = wiktionary_lemma.lemma.isin(suche_debug) #== 'gefriergetrocknet'
wiktionary_lemma[mask]

,page_id,page_part,section_id2,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,lemma_score,data_id
6418,11127,0,Euro,Euro,euro,True,1,NN,NN,OYXC6QnBDy,NOM,N,Substantiv Substantiv,153.715988,
6419,11127,1,Euro_1,Euro,euro,True,1,NN,NN,HZs41oOmIz,NOM,N,Substantiv Substantiv,4.110498,
29461,89575,0,zu_viel,zu viel,zu viel,False,2,ADV,ADV,D9DicGsKDl,ADV,A,Adverb Wortverbindung,14.845442,
144830,<NA>,<NA>,Euro,Euro,euro,True,1,NN,NN,P0rXszKHZY,NOM,N,Einheit manuell Substantiv,2.239461,
146321,<NA>,<NA>,zu_viel,zu viel,zu viel,False,2,ADV,ADV,nuhsBel9aV,ADV,A,manuell,2.239461,


### member erweitern
* Einige Merkmale aus wiktionary_merkmal nach member in wiktionary_lemma verschieben
* und zwar die E-Merkmale. das sind die, bei denen es einzig auf die Existenz ankommt. Diese Merkmale können keine verschiedenen Werte annehmen.

In [372]:
# noSing >> noSg
mask = (wiktionary_merkmal.merkmal == 'noSing')
wiktionary_merkmal.loc[mask,'merkmal'] = 'noSg'

In [373]:
# E-Merkmale 
print(merkmale_E)

['noSteig', 'noSing', 'istName', 'Wikispecies', 'Wikivoyage', 'Taxonomie', 'Präfix', 'Suffix', 'VMaßeinheiten', 'noSg', 'noPl', 'noFlex']


In [374]:
# anschauen
mask = wiktionary_merkmal.merkmal.isin(merkmale_E)
a = pak.analyse_freqs(wiktionary_merkmal[mask],['merkmal','data']) # 
grid(a)

11 rows


,merkmal,merkmal_count,merkmal_percent,data,data_count
0,noSteig,8098,65.6,exists,8098
1,istName,1801,14.6,exists,1801
2,noSg,690,5.6,"[exists, 1, ja]","[650, 32, 8]"
3,noPl,457,3.7,"[1, ja]","[431, 26]"
4,noFlex,390,3.2,"[ja, 1, j]","[388, 1, 1]"
5,Wikivoyage,243,2.0,exists,243
6,Wikispecies,202,1.6,exists,202
7,Taxonomie,171,1.4,exists,171
8,Präfix,111,0.9,exists,111
9,Suffix,105,0.9,exists,105


In [375]:
# update-Tabelle erstellen
mask = wiktionary_merkmal.merkmal.isin(merkmale_E)
df = pak.group_and_agg(wiktionary_merkmal[mask], 
              col_origins=['section_id2', 'merkmal',             'merkmal',       ],
              col_funcs  =['group',       pak.agg_words_nodup,   'count',         ],
              col_names  =['',            'member_plus',         'count',         ],  
             )
#df

582 rows less, now 11759 rows


In [376]:
# member_plus schreiben
wiktionary_lemma = pak.update_col(wiktionary_lemma, df, on='section_id2', col='member_plus')

update_col: 12445 cells written into new column


In [377]:
# member schreiben
mask = (wiktionary_lemma.member_plus.str.len() > 0)
wiktionary_lemma.loc[mask,'member'] = wiktionary_lemma[mask].member + ' ' + wiktionary_lemma[mask].member_plus
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'member_plus')
#grid(wiktionary_lemma, mask)

In [378]:
# wiktionary_merkmal belöschen
mask = wiktionary_merkmal.merkmal.isin(merkmale_E)
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='merkmale_E')

Move 12341 rows from 5188632


### member pflegen

In [379]:
# werte_anz
df = pak.group_and_agg(wiktionary_merkmal, 
              col_origins=['merkmal',    'data',                  ],
              col_funcs  =['group',       pak.nunique,            ],
              col_names  =['',           'werte_anz',             ],              
             )
#df

5176212 rows less, now 79 rows


In [380]:
# merkmale mit kleiner werte_anz
mask = (df.werte_anz < 3)   &   ~df.merkmal.str.startswith(('Nom','Gen','Dat','Akk', 'Konj', 'Ptz1'))
m = list(df[mask].merkmal)

mask = wiktionary_merkmal.merkmal.isin(m)
assert wiktionary_merkmal[mask].shape[0] <= 10  

a = pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','data'])
a 

#wiktionary_merkmal[mask]

,merkmal,merkmal_count,merkmal_percent,data,data_count
0,ADV,1,16.7,zu viele,1
1,Befehl du,1,16.7,hetz umeinander!,1
2,Befehl ihr,1,16.7,hetzt umeinander!,1
3,VVIMP,1,16.7,vergleiche,1
4,Sg,1,16.7,North Carolina,1
5,genitiv Sg,1,16.7,Zinkweißes,1


In [381]:
wiktionary_merkmal[mask]

,section_id2,merkmal,data,num,meta,data_id,data_score
2166487,zu_viel,ADV,zu viele,<NA>,manuell,,NaN
2166488,umeinanderhetzen,Befehl du,hetz umeinander!,<NA>,,,NaN
2166489,umeinanderhetzen,Befehl ihr,hetzt umeinander!,<NA>,,,NaN
2166490,vergleichen,VVIMP,vergleiche,<NA>,manuell,,NaN
2579757,North_Carolina_1,Sg,North Carolina,<NA>,,,NaN
2579758,Zinkweiß,genitiv Sg,Zinkweißes,<NA>,,,NaN


In [382]:
# In list wandeln
wiktionary_lemma.member = wiktionary_lemma.member.str.split()

# nach Priorität sortieren (laut Steuertabellen)
wiktionary_lemma.member = member_sortieren(wiktionary_lemma.member) 

# In string wandeln
wiktionary_lemma.member = wiktionary_lemma.member.str.join(' ')

In [383]:
# Seltene member finden
member_selten = pak.analyse_freqs(wiktionary_lemma, ['member', 'section_id2'], splits=[' ',None])
mask = (member_selten.member_count == 1)
member_selten = member_selten[mask]
member_selten_member      = list(member_selten.member)
member_selten_section_id2 = list(member_selten.section_id2)

# und entfernen
mask = wiktionary_lemma.section_id2.isin(member_selten_section_id2)
wiktionary_lemma.loc[mask,'member'] = pak.remove_str(wiktionary_lemma[mask].member, member_selten_member)
#wiktionary_lemma[mask]

In [384]:
member_selten

,member,member_count,member_percent,section_id2,section_id2_count
76,Formel,1,0.0,wohl_bekomms,1
77,Sprichwort,1,0.0,variatio_delectat,1
78,Reziprokpronomen,1,0.0,einander,1
79,Englisch,1,0.0,North_Carolina_1,1
80,Vergleichspartikel,1,0.0,wie_1,1
81,modal,1,0.0,gewiss,1


In [385]:
# anschauen wiktionary_lemma
a = pak.analyse_freqs(wiktionary_lemma,['member','tag'], splits=[' ', None])
gridt(a)

76 rows


In [386]:
# anschauen wiktionary_nolemma
b = pak.analyse_freqs(wiktionary_nolemma,['member','tag'], splits=[' ', None])
gridt(b)

17 rows


In [387]:
# suche_debug
mask = wiktionary_lemma.lemma.isin(suche_debug) #== 'gefriergetrocknet'
wiktionary_lemma[mask]

,page_id,page_part,section_id2,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,lemma_score,data_id
6418,11127,0,Euro,Euro,euro,True,1,NN,NN,OYXC6QnBDy,NOM,N,Substantiv Substantiv,153.715988,
6419,11127,1,Euro_1,Euro,euro,True,1,NN,NN,HZs41oOmIz,NOM,N,Substantiv Substantiv,4.110498,
29461,89575,0,zu_viel,zu viel,zu viel,False,2,ADV,ADV,D9DicGsKDl,ADV,A,Adverb Wortverbindung,14.845442,
144830,<NA>,<NA>,Euro,Euro,euro,True,1,NN,NN,P0rXszKHZY,NOM,N,Substantiv Einheit manuell,2.239461,
146321,<NA>,<NA>,zu_viel,zu viel,zu viel,False,2,ADV,ADV,nuhsBel9aV,ADV,A,manuell,2.239461,


### member weiter pflegen

In [388]:
# Ersetzungen im member laut member_replaces (definiert in Steuertabellen)
print(member_replaces)
wiktionary_lemma.loc[:,'member'] = pak.replace_str(wiktionary_lemma.member, member_replaces)

{'Eigenname': '', 'Straßenname': 'Geo_straße', 'Ortsnamengrundwort': 'Geo_ort', 'Ort': 'Geo_ort', 'Stadt': 'Geo_stadt', 'Land': 'Geo_land', 'Toponym': 'Geo', 'Wikivoyage': 'Geo', 'Vorname': 'Person_vorname', 'Nachname': 'Person_nachname', 'PRE': 'Person_pre', 'adjektivische_Deklination': 'adjNN', 'Wikispecies': 'Spezies', 'VMaßeinheiten': 'Einheit'}


In [389]:
# redundantes Substantiv
mask = wiktionary_lemma.member.str.contains('Substantiv')   &   (wiktionary_lemma.tagZZ == 'N') 
check_mask( wiktionary_lemma, mask, 104832 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Substantiv','')

check_mask:  104618 rows


In [390]:
# redundantes Adjektiv
mask = wiktionary_lemma.member.str.contains('Adjektiv')   &   (wiktionary_lemma.tagZ == 'ADJ') 
check_mask( wiktionary_lemma, mask, 14312 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Adjektiv','')

check_mask:  14312 rows


In [391]:
# redundantes Verb
mask = wiktionary_lemma.member.str.contains('Verb')   &   (wiktionary_lemma.tagZZ == 'V') 
check_mask( wiktionary_lemma, mask, 14333 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Verb','')

check_mask:  14335 rows


In [392]:
# redundantes istName, außerdem 1 falsches istName das NN ist
mask = wiktionary_lemma.member.str.contains('istName')   &   ( wiktionary_lemma.tagZZ == 'N' ) 
check_mask( wiktionary_lemma, mask, 2339 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('istName','')

check_mask:  2337 rows


In [393]:
# redundantes Name
mask = wiktionary_lemma.member.str.contains('Name')   &   ( wiktionary_lemma.tag_0.isin(['NE']) | wiktionary_lemma.tag_1.isin(['NE']) )
check_mask( wiktionary_lemma, mask, 177 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Name','')

check_mask:  177 rows


In [394]:
# redundantes Konjunktion
#mask = wiktionary_lemma.member.str.contains('Konjunktion')   &   ~wiktionary_lemma.member.str.contains('Konjunktionaladverb')   &   \
#     ( wiktionary_lemma.tag_0.isin(['KOUS','KON','KOKOM','KOUI']) | wiktionary_lemma.tag_1.isin(['KOUS','KON','KOKOM','KOUI']) ) 

mask1 =   wiktionary_lemma.member.str.contains('Konjunktion') 
mask2 =  ~wiktionary_lemma.member.str.contains('Konjunktionaladverb') 
mask3A =  wiktionary_lemma.tag_0.isin(['KOUS','KON','KOKOM','KOUI'])
mask3B =  wiktionary_lemma.tag_1.isin(['KOUS','KON','KOKOM','KOUI'])
mask = mask1  &  mask2  &  (mask3A | mask3B)
check_mask( wiktionary_lemma, mask, 104 )

##wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Konjunktion','')
#grid(wiktionary_lemma[mask])

check_mask:  104 rows


In [395]:
# redundantes Indefinitpronomen
mask = wiktionary_lemma.member.str.contains('Indefinitpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PIAT','PIS']) | wiktionary_lemma.tag_1.isin(['PIAT','PIS']) )
check_mask( wiktionary_lemma, mask, 66 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Indefinitpronomen','')

check_mask:  66 rows


In [396]:
# redundantes Interjektion
mask = wiktionary_lemma.member.str.contains('Interjektion')   &   ( wiktionary_lemma.tag_0.isin(['ITJ']) | wiktionary_lemma.tag_1.isin(['ITJ']) )
check_mask( wiktionary_lemma, mask, 245 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Interjektion','')

check_mask:  245 rows


In [397]:
# redundantes Adverb
mask = wiktionary_lemma.member.str.contains('Adverb')   &   ( wiktionary_lemma.tag_0.isin(['ADV']) | wiktionary_lemma.tag_1.isin(['ADV']) )
check_mask( wiktionary_lemma, mask, 1524 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Adverb','')

check_mask:  1524 rows


In [398]:
# redundantes Subjunktion
mask = wiktionary_lemma.member.str.contains('Subjunktion')   &   ( wiktionary_lemma.tag_0.isin(['KOUS']) | wiktionary_lemma.tag_1.isin(['KOUS']) )
check_mask( wiktionary_lemma, mask, 58 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Subjunktion','')

check_mask:  58 rows


In [399]:
# redundantes Personalpronomen
mask = wiktionary_lemma.member.str.contains('Personalpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PPER']) | wiktionary_lemma.tag_1.isin(['PPER']) )
check_mask( wiktionary_lemma, mask, 33 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Personalpronomen','')

check_mask:  33 rows


In [400]:
# redundantes Demonstrativpronomen
mask = wiktionary_lemma.member.str.contains('Demonstrativpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PDAT']) | wiktionary_lemma.tag_1.isin(['PDAT']) )
check_mask( wiktionary_lemma, mask, 32 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Demonstrativpronomen','')

check_mask:  32 rows


In [401]:
# redundantes Artikel
mask = wiktionary_lemma.member.str.contains('Artikel')   &   ( wiktionary_lemma.tag_0.isin(['ART']) | wiktionary_lemma.tag_1.isin(['ART']) )
check_mask( wiktionary_lemma, mask, 16 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Artikel','')

check_mask:  16 rows


In [402]:
# redundantes Postposition
mask = wiktionary_lemma.member.str.contains('Postposition')   &   ( wiktionary_lemma.tag_0.isin(['APPO']) | wiktionary_lemma.tag_1.isin(['APPO']) )
check_mask( wiktionary_lemma, mask, 13 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Postposition','')

check_mask:  13 rows


In [403]:
# redundantes Relativpronomen
mask = wiktionary_lemma.member.str.contains('Relativpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PRELAT']) | wiktionary_lemma.tag_1.isin(['PRELAT']) )
check_mask( wiktionary_lemma, mask, 10 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Relativpronomen','')

check_mask:  10 rows


In [404]:
# redundantes Possessivpronomen
mask = wiktionary_lemma.member.str.contains('Possessivpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PPOSS']) | wiktionary_lemma.tag_1.isin(['PPOSS']) )
check_mask( wiktionary_lemma, mask, 9 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Possessivpronomen','')

check_mask:  9 rows


In [405]:
# redundantes Negationspartikel
mask = wiktionary_lemma.member.str.contains('Negationspartikel')   &   ( wiktionary_lemma.tag_0.isin(['PTKNEG']) | wiktionary_lemma.tag_1.isin(['PTKNEG']) )
check_mask( wiktionary_lemma, mask, 4 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Negationspartikel','')

check_mask:  4 rows


In [406]:
# redundantes Interrogativpronomen
mask = wiktionary_lemma.member.str.contains('Interrogativpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PWAT']) | wiktionary_lemma.tag_1.isin(['PWAT']) )
check_mask( wiktionary_lemma, mask, 4 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Interrogativpronomen','')

check_mask:  4 rows


In [407]:
# falsches noPl (das betrifft vorwiegend NE-Datensätze)
mask = wiktionary_lemma.member.str.contains('noPl')   &   ~wiktionary_lemma.tag_0.isin(['NN','TODO',])   &   ~wiktionary_lemma.tagZZ.isin(['P'])
check_mask(wiktionary_lemma,mask, 1452)
wiktionary_lemma.loc[mask,'member'] = wiktionary_lemma[mask].member.str.replace('noPl','')
#grid( wiktionary_lemma, mask )

check_mask:  1452 rows


In [408]:
# falsches noSg
mask = wiktionary_lemma.member.str.contains('noSg')   &   ~wiktionary_lemma.tag_0.isin(['NN','NE','TODO','PPER'])
check_mask(wiktionary_lemma,mask,3)
wiktionary_lemma.loc[mask,'member'] = wiktionary_lemma[mask].member.str.replace('noSg','')
#grid( wiktionary_lemma, mask )

check_mask:  3 rows


In [409]:
# falsches noSteig
mask = wiktionary_lemma.member.str.contains('noSteig')   &   ~wiktionary_lemma.tag_0.isin(['ADJA'])
check_mask(wiktionary_lemma,mask, 22)
wiktionary_lemma.loc[mask,'member'] = wiktionary_lemma[mask].member.str.replace('noSteig','')

check_mask:  22 rows


In [410]:
# falsches noFlex
mask = wiktionary_lemma.member.str.contains('noFlex')   &   ~wiktionary_lemma.tagZ.isin(['ADJ',])
check_mask( wiktionary_lemma, mask, 2 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('noFlex','')

check_mask:  2 rows


In [411]:
# falsches Interjektion
mask = wiktionary_lemma.member.str.contains('Interjektion')  &   wiktionary_lemma.tag_0.isin(['PTKANT',])
check_mask( wiktionary_lemma, mask, 31 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Interjektion','')

check_mask:  31 rows


In [412]:
# falsches Präfix
mask = wiktionary_lemma.member.str.contains('Präfix')   &   ~wiktionary_lemma.tagZ.isin(['FIX',])
check_mask( wiktionary_lemma, mask, 3 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Präfix','')

check_mask:  3 rows


In [413]:
# Taxonomie >> Spezies
mask = wiktionary_lemma.member.str.contains('Taxonomie')   &   (wiktionary_lemma.tagZZ == 'N') 
check_mask( wiktionary_lemma, mask, 169 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Taxonomie','Spezies')

check_mask:  169 rows


In [414]:
# falsches Vorname
mask = wiktionary_lemma.member.str.contains('Person_vorname')   &   ~wiktionary_lemma.tag_0.isin(['NE'])
check_mask( wiktionary_lemma, mask, 1 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Person_vorname','')
#grid( wiktionary_lemma, mask )

check_mask:  1 rows


In [415]:
# Streichungen im member       
member_removes = [ 'Numerale', 'Pronomen', 'noWortart', 'Hilfsverb', 'Taxonomie', 'Geflügeltes_Wort', ] 

wiktionary_lemma.loc[:,'member'] = pak.remove_str(wiktionary_lemma.member, member_removes)

In [416]:
# wiktionary_lemma_bak = wiktionary_lemma.copy()
#wiktionary_lemma = wiktionary_lemma_bak.copy()

In [417]:
# Substantiv aber kein NN
mask1 =  wiktionary_lemma.member.str.contains('Substantiv')
mask2 = ~wiktionary_lemma.tag.str.contains(' ')
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,3)
wiktionary_lemma.loc[mask,'tag'] = 'NN ' + wiktionary_lemma[mask].tag
wiktionary_lemma[mask]

check_mask:  3 rows


,page_id,page_part,section_id2,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,lemma_score,data_id
18958,50386,0,Bussi,Bussi,bussi,True,1,NN PTKANT,PTKANT,I6vxvBKqKR,X_antw,X,Substantiv Substantiv Grußformel Partikel,46.624123,
27533,78268,0,zig,zig,zig,False,1,NN ADJA,ADJA,f4xaiRta3a,ADJ,A,Substantiv Zahlklassifikator noSteig,36.099144,
69356,377153,0,Salam,Salam,salam,True,1,NN PTKANT,PTKANT,ScWEXNgirP,X_antw,X,Substantiv Substantiv Grußformel Partikel,62.145668,


In [418]:
# korrigieren
wiktionary_lemma, df = pak.move_rows(wiktionary_lemma,mask,msg=None)
df = pflege_tag_felder(df, translate_tagZ)
wiktionary_lemma = pak.add_rows(wiktionary_lemma,df)

Move 3 rows from 173924
3 rows added, now a total of 173924


In [419]:
# anschauen
mask1 =  wiktionary_lemma.member.str.contains('Substantiv')
mask2 =  wiktionary_lemma.tag.str.contains(' ')
mask = mask1  &  mask2
pak.analyse_freqs(wiktionary_lemma[mask],['tag','lemma'])

,tag,tag_count,tag_percent,lemma,lemma_count
0,ZFIX TRUNC,36,92.3,"[-meter, -ier, -in, -kunde, -schrecke, -er, -ismus, -ant, -chen, -lein, -logie, -heit, -keit, -ung, -phobie, -ling, -tum, -schaft, -loge, -skopie]","[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,NN PTKANT,2,5.1,"[Bussi, Salam]","[1, 1]"
2,NN ADJA,1,2.6,zig,1


### member prüfen

In [420]:
# alle member in den realen Daten zusammensammeln

a = pak.analyse_freqs(wiktionary_lemma, ['member','tag'], splits=[' ', None] )
#grid(a)

b = pak.analyse_freqs(wiktionary_nolemma, ['member','tag'], splits=[' ', None] )
#grid(b)

# Member lt. Steuertabelle und lt. Realität
member_plan = set(priorität_member) #- set(member_replaces.keys())

member_data1 = set(a.member)
member_data2 = set(b.member)
member_data  = member_data1.union(member_data2)
#member_data

In [421]:
# left_only:  member, die geplant sind aber in der Realität nicht vorkommen
# right_only: member, die in der Realität vorkommen aber nicht geplant sind
bpy.minivenn( member_plan, member_data, format='print2')

left_only:  {'Personalpronomen', 'Ort', 'Ortsnamengrundwort', 'PRE', 'Geflügeltes_Wort', 'Interfix', 'Eigenname', 'Wikispecies', 'Nachname', 'Stadt', 'noSing', 'Postposition', 'istName', 'Affix', 'Interjektion', 'Sprichwort', 'Toponym', 'Negationspartikel', 'Subjunktion', 'Zirkumfix', 'Land', 'Reziprokpronomen', 'Possessivpronomen', 'Vorname', 'Hilfsverb', 'VMaßeinheiten', 'Interrogativpronomen', 'Vergleichspartikel', 'Taxonomie', 'Aufzählung', 'Straßenname', 'oid', 'Formel', 'Wikivoyage', 'nurWortart'}
right_only: {''}



In [422]:
# alle Merkmale in wiktionary_merkmal sollten in der Steuertabelle plan_merkmal erfasst sein.
ungeplante_member = bpy.minivenn(member_plan,member_data)['right_only'] - {''}
assert len(ungeplante_member) == 0

In [423]:
# Beispiel anzeigen 
mask = (wiktionary_merkmal.section_id2 == suche_debug[0])
grid(wiktionary_merkmal,mask)

43 rows out of 5176291


,section_id2,merkmal,data,num,meta,data_id,data_score
195244,Euro,def,Währung,1,,,NaN
195245,Euro,def,Europäischen Währungsunion,1,,,NaN
195246,Euro,syn,Teuro,1,scherzhaft,,NaN
195247,Euro,up,Währung,1,,,NaN
195248,Euro,down,Pendlereuro,1,,,NaN
195249,Euro,familie,Euro-Abschaffung,,,,NaN
195250,Euro,familie,Eurobanknote,,,,NaN
195251,Euro,familie,Eurobond,,,,NaN
195252,Euro,familie,Eurocent,,,,NaN
195253,Euro,familie,Euro-Einführung,,,,NaN


### wiktionary_lemma pflegen und prüfen
* Diverse Pflegeschritte

In [424]:
# lemma_lower, isupper, lemma_len
wiktionary_lemma = ergänze_felder(wiktionary_lemma)

check_mask:  309 rows


In [425]:
# Eindeutige Datentypen
for col in wiktionary_lemma.columns:
    print(col)
    if col in ['num','error']:
        continue
    assert pak.ntypes(wiktionary_lemma[col]) == 1

page_id
page_part
section_id2
lemma
lemma_lower
isupper
lemma_len
tag
tag_0
tag_1
tagZ
tagZZ
member
lemma_score
data_id


In [426]:
# Keine unbekannten tags
a = pak.analyse_freqs(wiktionary_lemma, ['tag'], splits=[' '])
mask = ~a.tag.isin(alle_tags)
check_mask( a, mask, 0)
a[mask]

check_mask:  0 rows


,tag,tag_count,tag_percent,graph


In [427]:
# finden
mask = wiktionary_lemma.tag == 'Zirkumfix'
grid(wiktionary_lemma, mask, color='green')

No rows, mask filters them all away


In [428]:
# NaN
wiktionary_lemma['page_id']   = wiktionary_lemma.page_id.fillna(-1) 
wiktionary_lemma['page_part'] = wiktionary_lemma.page_part.fillna(-1) 

## Letzte Pflegeschritte

In [429]:
wiktionary_lemma        = pak.reset_index( wiktionary_lemma.sort_values('section_id2') )

In [430]:
wiktionary_nolemma      = pak.drop_cols(wiktionary_nolemma,['lang','lang_code','error'])
wiktionary_nolemma      = pak.move_cols(wiktionary_nolemma,'lemma_lower','lemma')

In [431]:
#wiktionary_merkmal_bak = wiktionary_merkmal.copy()

In [432]:
# score übertragen und ggf. ergänzen
wiktionary_merkmal = pak.update_col(df_to=wiktionary_merkmal, df_from=wiktionary_lemma,        on='section_id2',                        col='lemma_score', col_rename='data_score', func='max', cond='null')
wiktionary_merkmal = pak.update_col(df_to=wiktionary_merkmal, df_from=wiktionary_nolemma, left_on='section_id2', right_on='nolemma_id', col='data_score',                           func='max', cond='null')
wiktionary_merkmal['data_score'] = wiktionary_merkmal.data_score.fillna(wiktionary_merkmal.data_score.quantile(0.5)).astype('float')

update_col: func max applied, 2700 records less!
update_col: 2761361 cells written into existing column
update_col: func max applied, but it was pointless!
update_col: 2387352 cells written into existing column


In [433]:
# Keine Leerzeichen in der ID
mask = wiktionary_lemma.section_id2.str.contains(' ')
check_mask(wiktionary_lemma,mask,0)

mask = wiktionary_nolemma.nolemma_id.str.contains(' ')
check_mask(wiktionary_nolemma,mask,0)

mask = wiktionary_merkmal.section_id2.str.contains(' ')
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows
check_mask:  0 rows
check_mask:  0 rows


## Löschliste nochmal

In [434]:
# wiktionary_lemma belöschen nach section_id2
mask = wiktionary_lemma.section_id2.isin(löschliste_section_id2)   

msg='löschliste_B1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows( df_from=wiktionary_lemma, df_to=wiktionary_lemma_trash, mask=mask, msg=msg)       # Löschen
grid(wiktionary_lemma_trash,msg)

No rows moved
No rows, mask filters them all away


In [435]:
# wiktionary_lemma belöschen nach lex
mask = wiktionary_lemma.lemma.isin(löschliste_lex)

msg='löschliste_B2'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows( df_from=wiktionary_lemma, df_to=wiktionary_lemma_trash, mask=mask, msg=msg)       # Löschen
grid(wiktionary_lemma_trash,msg)

Move 4 rows from 173924
4 rows out of 13


,page_id,page_part,section_id2,lemma,inflected,tag,tag_0,tag_1,tagZ,tagZZ,member,lang,lang_code,error,lemma_lower,lemma_score,isupper,lemma_len,data_id,msg
3277,-1,-1,Ahu_V,Ahu,NaN,NE,NE,3FF150TR8o,NAM,N,Person_vorname,<NA>,<NA>,<NA>,ahu,0.745434,True,1,,löschliste_B2
7457,-1,-1,Assoll_V,Assoll,NaN,NE,NE,1lVaNPDFLe,NAM,N,Person_vorname,<NA>,<NA>,<NA>,assoll,0.745434,True,1,,löschliste_B2
18375,-1,-1,Chen_V,Chen,NaN,NE,NE,Hw7Dj08G3J,NAM,N,Person_vorname,<NA>,<NA>,<NA>,chen,0.745434,True,1,,löschliste_B2
45494,-1,-1,Iker_V,Iker,NaN,NE,NE,J2uG6rsMw4,NAM,N,Person_vorname,<NA>,<NA>,<NA>,iker,0.745434,True,1,,löschliste_B2


In [436]:
# wiktionary_nolemma belöschen nach lex
mask = wiktionary_nolemma.lemma.isin(löschliste_lex)   

msg='löschliste_B3'
wiktionary_nolemma, wiktionary_nolemma_trash = pak.move_rows( df_from=wiktionary_nolemma, df_to=wiktionary_nolemma_trash, mask=mask, msg=msg )
grid(wiktionary_nolemma_trash,msg)

Move 2189 rows from 669786
2189 rows


,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,tag_0,tag_1,...,member,lang,lang_code,error,lemma_lower,score,nolemma_id,lex,data_score,msg
6196,<NA>,<NA>,<NA>,<NA>,lusen,NaN,NaN,LEX_V,LEX_V,UoomvcO35B,...,Konjugierte_Form Verb,<NA>,<NA>,<NA>,lusen,NaN,lus,lus,0.726053,löschliste_B3
7299,<NA>,<NA>,<NA>,<NA>,versieren,NaN,NaN,LEX_V,LEX_V,HbJNmxP7Ou,...,Konjugierte_Form Verb,<NA>,<NA>,<NA>,versieren,NaN,versiert_1,versiert,0.113503,löschliste_B3
11117,<NA>,<NA>,<NA>,<NA>,gelt,NaN,NaN,LEX_A,LEX_A,fHxWdyACom,...,Deklinierte_Form,<NA>,<NA>,<NA>,gelt,NaN,gelten_1,gelten,0.282176,löschliste_B3
17941,<NA>,<NA>,<NA>,<NA>,russen,NaN,NaN,LEX_V,LEX_V,VTOMOZ2arN,...,Konjugierte_Form Verb,<NA>,<NA>,<NA>,russen,NaN,russe,russe,0.723131,löschliste_B3
18214,<NA>,<NA>,<NA>,<NA>,mechen,NaN,NaN,LEX_V,LEX_V,HMgmztNtaz,...,Konjugierte_Form Verb,<NA>,<NA>,<NA>,mechen,NaN,meche,meche,1.417441,löschliste_B3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826558,<NA>,<NA>,<NA>,<NA>,Norddeutscher Postbezirk,NaN,NaN,LEX_A,LEX_A,WossjZwCyQ,...,Deklinierte_Form,<NA>,<NA>,<NA>,norddeutscher postbezirk,NaN,Norddeutschem_Postbezirke,Norddeutschem Postbezirke,0.526654,löschliste_B3
826559,<NA>,<NA>,<NA>,<NA>,Norddeutscher Postbezirk,NaN,NaN,LEX_A,LEX_A,MaEjw5IFSJ,...,Deklinierte_Form,<NA>,<NA>,<NA>,norddeutscher postbezirk,NaN,Norddeutschem_Postbezirk,Norddeutschem Postbezirk,0.522271,löschliste_B3
826560,<NA>,<NA>,<NA>,<NA>,Norddeutscher Postbezirk,NaN,NaN,LEX_A,LEX_A,wfSSZLknGt,...,Deklinierte_Form,<NA>,<NA>,<NA>,norddeutscher postbezirk,NaN,Norddeutsche_Postbezirk,Norddeutsche Postbezirk,0.322814,löschliste_B3
826561,<NA>,<NA>,<NA>,<NA>,Norddeutscher Postbezirk,NaN,NaN,LEX_A,LEX_A,GtUeL7fVeV,...,Deklinierte_Form,<NA>,<NA>,<NA>,norddeutscher postbezirk,NaN,Norddeutschen_Postbezirke,Norddeutschen Postbezirke,0.424054,löschliste_B3


In [437]:
# wiktionary_nolemma belöschen nach lex
mask = wiktionary_nolemma.lex.isin(löschliste_lex)   

msg='löschliste_B4'
wiktionary_nolemma, wiktionary_nolemma_trash = pak.move_rows( df_from=wiktionary_nolemma, df_to=wiktionary_nolemma_trash, mask=mask, msg=msg )
grid(wiktionary_nolemma_trash,msg)

No rows moved
No rows, mask filters them all away


In [438]:
# wiktionary_merkmal belöschen nach section_id2
mask = wiktionary_merkmal.section_id2.isin(löschliste_section_id2) 

msg='löschliste_B5'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg)

No rows moved
No rows, mask filters them all away


In [439]:
# wiktionary_merkmal belöschen nach lex
mask = wiktionary_merkmal.data.isin(löschliste_lex) 

msg='löschliste_B6'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg)

Move 61 rows from 5176291
61 rows out of 84525


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,msg,data_id,data_score
78436,Verb,kombi,NaN,NaN,trennbares Verb,1,,löschliste_B6,,402.653046
112078,Abbe,herkunft,NaN,NaN,abbe,,,löschliste_B6,,46.024231
115115,Gesetz,kombi,NaN,NaN,Zipfsches Gesetz,2,Adjektiv,löschliste_B6,,1777.648926
145478,Dativ,kombi,NaN,NaN,freier Dativ,1,,löschliste_B6,,171.259445
199829,Infinitiv,kombi,NaN,NaN,erweiterter Infinitiv,1,,löschliste_B6,,138.694305
...,...,...,...,...,...,...,...,...,...,...
3531184,fladern_1,Ptz2,NaN,NaN,g'fladerd,<NA>,,löschliste_B6,,4.764214
3549517,humorig,Positiv,NaN,NaN,humorigg.s,<NA>,,löschliste_B6,,167.101364
3584233,gut_besucht,Superlativ,NaN,NaN,besten besucht,<NA>,,löschliste_B6,,7.850154
3584242,gut_erhalten,Superlativ,NaN,NaN,besten erhalten,<NA>,,löschliste_B6,,9.255282


In [440]:
# wiktionary_merkmal belöschen nach data
mask = wiktionary_merkmal.data.isin(löschliste_data) 

msg='löschliste_B7'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg)

Move 68 rows from 5176230
68 rows out of 84593


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,msg,data_id,data_score
19540,Salz,syn,NaN,NaN,Essen und Trinken/Küchenkräuter und Gewürze,1,,löschliste_B7,,553.452026
62205,Zucker,down,NaN,NaN,Essen und Trinken/Küchenkräuter und Gewürze,1,allgemein,löschliste_B7,,945.212646
64576,auf_Wiedersehen,syn,NaN,NaN,Grüßen/Abschied,1,siehe,löschliste_B7,,38.669815
66841,adieu,syn,NaN,NaN,Grüßen/Abschied,1,,löschliste_B7,,28.278547
106372,bis_bald,syn,NaN,NaN,Grüßen/Abschied,1,,löschliste_B7,,8.338638
...,...,...,...,...,...,...,...,...,...,...
1871017,Industrieviertel_1,down_part_1,NaN,NaN,Bruck an der Leitha,1,,löschliste_B7,,4.324754
1912253,pfiat_di,syn,NaN,NaN,Grüßen/Abschied,1,siehe,löschliste_B7,,6.970684
1946072,Selbständige,nurWortart,NaN,NaN,Deklinierte Form,,,löschliste_B7,,16.735296
1946074,Selbständige,nurWortart,NaN,NaN,Plural der maskulinen Form,,,löschliste_B7,,16.735296


## Prüfen

In [441]:
# lex_test, lemma_test
from x32_Funktionen import *
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=1 )
#grid(lemma_test)

180 rows loaded
Delete 84 rows from 180


In [442]:
# lemma_test
from x32_Funktionen import *
lemma_test = check_lemma_test(lemma_test, wiktionary_lemma)
#lemma_test

In [443]:
# Fehler
maskA = lemma_test.lemma_id == ''
maskB = lemma_test.check_tag == ''
mask = maskA | maskB
check_mask(lemma_test,mask)

probleme = list(lemma_test[mask].lemma)
lemma_test[mask]

check_mask:  0 rows


,level,lemma_id,lemma,tag_soll,tag_ist,member_soll,check_tag,member_ist,check_member


In [444]:
# Sicherstellen: Keine Fehler
check_mask(lemma_test,mask,0)

check_mask:  0 rows


## Speichern

In [445]:
# Doppelspaces prüfen
if gründlich:
    mask = wiktionary_merkmal.data.str.contains('  ')
    check_mask( wiktionary_merkmal, mask, 0)

check_mask:  0 rows


In [446]:
pak.analyse_cols(wiktionary_merkmal)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,int64,np.int64,int64,np.int32,True,False,True,False,80.0 B,1,5176162,0,0,5176162,0.0,2619377.78,2599958.50,5244707.0,1.355832e+13
1,section_id2,str,pd.string,string,,False,True,True,True,812.0 B,1,840504,0,4335658,5176162,&c.,NaN,NaN,μm,NaN
2,merkmal,str,pd.string,string,,False,True,True,True,620.0 B,1,79,0,5176083,5176162,ADV,NaN,NaN,weib,NaN
3,data,str,pd.string,string,,False,True,True,True,946.0 B,1,2324447,0,2851715,5176162,,NaN,NaN,𒄿𒌑𒃷,NaN
4,num,str,pd.string,string,,False,True,True,True,471.0 B,1,224,3021441,2154497,5176162,,NaN,NaN,"9,14",NaN
5,meta,str,pd.string,string,,False,True,True,True,591.0 B,1,17305,0,5158857,5176162,,NaN,NaN,übrige,NaN
6,data_id,str,pd.string,string,,False,True,True,True,570.0 B,1,27141,0,5149021,5176162,,NaN,NaN,übriglassend,NaN
7,data_score,float64,np.float64,float64,np.float32,True,False,True,True,80.0 B,1,77144,0,5099018,5176162,0.0,79.87,11.46,12106.77,4.134356e+08


In [447]:
# wiktionary_merkmal: drop_duplicates
wiktionary_merkmal = wiktionary_merkmal.drop_duplicates()

In [448]:
assert gründlich

In [449]:
# Speichern  
pak.dump_pickle(wiktionary_lemma,            wiktionary_lemma_filename_save)   
pak.dump_pickle(wiktionary_nolemma,          wiktionary_nolemma_filename_save)  
pak.dump_pickle(wiktionary_merkmal,          wiktionary_merkmal_filename_save) 
pak.dump_pickle(wiktionary_merkmal_text,     wiktionary_merkmal_text_filename_save)  
os.utime(verzeichnis_save)

In [450]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive

In [451]:
pak.memory_consumption(locals())

,name,rtype,size
0,wiktionary_merkmal,"(DataFrame, Series)",2.0 GB
1,flexion,"(DataFrame, Series)",1.4 GB
2,wiktionary_nolemma,"(DataFrame, Series)",449.6 MB
3,wiktionary_merkmal_text,"(DataFrame, Series)",342.3 MB
4,wiktionary,"(DataFrame, Series)",150.9 MB
5,wiktionary_lemma,"(DataFrame, Series)",116.0 MB
6,mask,"(Series,)",49.4 MB
7,wiktionary_merkmal_trash,"(DataFrame, Series)",45.8 MB
8,mask3,"(Series, bool)",44.5 MB
9,wiktionary_lemma_partizip,"(DataFrame, Series)",17.9 MB


## Trash anschauen

### wiktionary_trash

In [452]:
wiktionary_trash.msg = wiktionary_trash.msg.fillna('')

In [453]:
# Übersicht
a = pak.analyse_freqs(wiktionary_trash, ['msg','member'], splits=[None,' '])
a

,msg,msg_count,msg_percent,member,member_count
0,title zu viele Wörter,3910,82.8,NaN,3910
1,löschliste_section_id2,504,10.7,NaN,504
2,title zu kurz,206,4.4,NaN,206
3,löschliste_lex,51,1.1,NaN,51
4,lemma zu kurz,26,0.6,NaN,26
5,lang_code ist nan,10,0.2,NaN,10
6,FIX_ZU_KURZ,10,0.2,"[Präfix, Suffix, , Ortsnamengrundwort, Interfix, Pronomen, Personalpronomen]","[3, 2, 2, 1, 1, 1, 1]"
7,MFIX,4,0.1,"[Gebundenes_Lexem, Affix, Interfix]","[2, 2, 2]"
8,Zirkumfix,3,0.1,"[Affix, Zirkumfix]","[3, 3]"


In [454]:
# Datensätze
mask = (wiktionary_trash.msg == 'MFIX')
#grid( wiktionary_trash, mask )

In [455]:
# suche_debug
grid(search_str(wiktionary_trash, suche_debug))

No rows


### wiktionary_merkmal_trash

In [456]:
wiktionary_merkmal_trash.msg = wiktionary_merkmal_trash.msg.fillna('')

In [ ]:
# Übersicht
#a = pak.analyse_freqs(wiktionary_merkmal_trash, ['msg','merkmal', 'data', ])
a = pak.analyse_freqs(wiktionary_merkmal_trash, ['msg','merkmal', 'data', ])
a = pak.drop_cols(a, ['msg_percent','data_count'])
grid(a,  max_column_width=300)

In [ ]:
# Datensätze
mask = (wiktionary_merkmal_trash.msg == 'data_endswith')
grid( wiktionary_merkmal_trash)

In [ ]:
# suche_debug
maskA = wiktionary_merkmal_trash.data.isin(suche_debug)
maskB = wiktionary_merkmal_trash.section_id2.isin(suche_debug)
mask  = maskA | maskB
grid(wiktionary_merkmal_trash,mask, color='green')

### flexion_trash
* flexion_trash enthält Datensätze, die ursprünglich aus wiktionary_merkmal stammen

In [ ]:
# Übersicht
a = pak.analyse_freqs(flexion_trash, ['merkmal', 'data' , 'msg'])
a = pak.drop_cols(a, ['merkmal_percent','msg_count','data_count'])
grid(a)

### wiktionary_lemma_trash

In [ ]:
# suche_debug
mask = wiktionary_lemma_trash.lemma.isin(suche_debug) 
wiktionary_lemma_trash[mask]

In [ ]:
# Übersicht
a = pak.analyse_freqs(wiktionary_lemma_trash,['member','msg','lemma'], splits=[None,' '])
grid(a)

### wiktionary_nolemma_trash

In [ ]:
# suche_debug
mask = wiktionary_nolemma_trash.lemma.isin(suche_debug) 
wiktionary_nolemma_trash[mask]

In [ ]:
# Übersicht
a = pak.analyse_freqs(wiktionary_nolemma_trash,['member','lex'], splits=[None,' '])
grid(a)

## Nutzdaten anschauen

In [ ]:
suche_debug

### wiktionary_lemma

In [ ]:
mask = wiktionary_lemma.lemma.isin(suche_debug) #== 'gefriergetrocknet'
wiktionary_lemma[mask]

### wiktionary_merkmal

In [ ]:
mask = wiktionary_merkmal.section_id2 == suche_debug[0]
a = wiktionary_merkmal[mask].sort_values(['merkmal','section_id2','data'])
grid(a)

In [ ]:
mask = wiktionary_merkmal.data.isin(suche_debug) #== 'gefriergetrocknet'
a = wiktionary_merkmal[mask].sort_values(['merkmal','section_id2','data'])
grid(a)

### wiktionary_nolemma

In [ ]:
# wiktionary_nolemma suche_debug
grid(search_str(wiktionary_nolemma, suche_debug))

In [ ]:
raise Stop